# OBJETIVO

BASE SOLUCX - OUVIDORIA - PEDIDO CRIS - COM AS SEGUINTES COLUNAS

id_transacao (Cod. Admissao)

data_transacao (Data ultima admissão)

id_cliente (Cód. Paciente)	

nome_do_cliente	

telefone	

email	

nome_loja (UNIDADE)	

setores (coluna SETOR do QVIS)	

especialidade	

id Loja	

Grupo Hospitalar	

Jornada (Caráter admissão + Outros produtos)

Subcategoria - Todas as de produção (Excluir Não-Categorizado)

Solicitante - Coluna dos Relatórios de Produção	

medico (Classificação com base no Tipo II vs Tudo exceto Tipo II) x ver lista Cris (Relação Médica VISION ONE)	

Tec_Med (Retorna 'Técnico' se Tipo Produto = Tipo II, 'Médico' caso contrário)

# PASSO A PASSO

BASES A SEREM UTILIZADAS:

ACOMP FREQ PACIENTE, CADASTRO, RELACAO MEDICA V1 SP, E PRODUCAO (CONSULTAS, EXAMES, CIRURGIAS, LENTES)

###############################################################################################################################

ACOMP FREQ PACIENTE (USAR BASE PARA PEGAR ESPECIALIDADE, DATA PRIMEIRA ADMISSAO, DATA ULTIMA ADMISSAO, SETOR, UNIDADE, GRUPO HOSPITALAR, COLUNA DE PRIMEIRA VEZ OU RECORRENTE, CELULAR E EMAIL):

BAIXADA SEPARADA POR ESPECIALIDADE, ADICIONANDO AQUI CADA ESPECIALIDADE CORRESPONDENTE AO SEU ARQUIVO EXCEL, FILTRANDO MES ATUAL

CONCATENAR TODOS NO FINAL

###############################################################################################################################

CADASTRO (USAR PARA PEGAR GENERO E IDADE), ATUALIZAR NO DIA DA BASE

###############################################################################################################################

PRODUCAO (USAR BASE PARA PEGAR COD ADMISSAO, DATA ADMISSAO, JORNADA, SUBCATEGORIA, SOLICITANTE, MEDICO):

LER BASES SEPARADAS -> ARQUIVOS SOLTOS -> BAIXAR FILTRANDO MES ATUAL E DIA DA BASE

###############################################################################################################################

SEPARAR JORNADAS, UM DF PARA JORNADAS (AMBULATORIAL, EXAMES, LENTES E URGENCIA), OUTRO DF PARA JORNADA CIRURGICA / INJECAO

###############################################################################################################################

NEGATIVAÇÃO DAS BASES (OLHAR NOTA NA PASTA)

###############################################################################################################################

ADICIONAR COLUNA DE TEC_MED MANUALMENTE NO EXCEL -> FORMULA EM NOTA NA PASTA

LER BASE COM TEC_MED, TIRAR COLUNA TIPO PRODUTO, BAIXAR BASE FINAL

###############################################################################################################################

FILTRAR HOSPITAL (GRUPO HOSPITALAR) NO FINAL DE ACORDO COM A PESQUISA

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import os
from dateutil.relativedelta import relativedelta
import time
start_time = time.time()

In [2]:
#!/usr/bin/env python
# coding: utf-8

import smtplib
import ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Função para enviar e-mails via Gmail
def enviar_email(remetente, senha, destinatarios, assunto, corpo):
    try:
        message = MIMEMultipart()
        message['Subject'] = assunto
        message['From'] = remetente
        message['To'] = ', '.join(destinatarios)
        
        # Adicionando o corpo do e-mail
        message.add_header('Content-Type', 'text/html')
        message.set_payload(corpo)

        # Conexão com o servidor SMTP do Gmail
        context = ssl.create_default_context()
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls(context=context)  # Habilita TLS (criptografia)
            server.login(remetente, senha)    # Login com a senha de app
            server.sendmail(
                remetente, destinatarios,
                message.as_string().encode('utf_8')
            )
        print(f"E-mail enviado com sucesso de {remetente}.")
        return True
    except Exception as e:
        print(f"Falha ao enviar e-mail com {remetente}. Erro: {e}")
        return False

# Definir destinatários
email_to = ['marcelo.carvalho@visionone.com.br', 'gabrielly.vetore@visionone.com.br']

# Corpo do e-mail (mensagem fixa)
corpo = r'''
    <p>Prezado(a),</p>
    <p>INÍCIO NOTEBOOK ENVIO API</p>
    <p>Esta é uma mensagem enviada automaticamente. Por favor, não a responda.</p>
'''

# Assunto do e-mail
assunto = 'INÍCIO NOTEBOOK ENVIO API'

# Remetente e senha de aplicativo para Gmail
remetente_gmail = 'mcc.visionone@gmail.com'
senha_gmail = 'bqwk zcgm vgeh bcgp'  # Substitua com sua senha de aplicativo gerada

# Tentar enviar com Gmail
sucesso = enviar_email(remetente_gmail, senha_gmail, email_to, assunto, corpo)

if sucesso:
    print("E-mail enviado com sucesso!")
else:
    print("Falha ao enviar e-mail.")

E-mail enviado com sucesso de mcc.visionone@gmail.com.
E-mail enviado com sucesso!


# OBS: ADICIONANDO DOIS CAMINHOS PARA LEITURA DOS ARQUIVOS (HENRIQUE E GABY), 

# APENAS PARA FACILITAR QUANDO PRECISAR TROCAR OS DIAS DE EXTRAIR,

# SEMPRE DEIXAR O CAMINHO QUE NAO ESTIVER UTILIZANDO COMENTADO (RAW NB CONVERT)

# POR EXEMPLO: GABY ESTA EXTRAINDO, COMENTA O CAMINHO DO HENRIQUE, E VICE VERSA


# LER BASES (ACOMP FREQ PACIENTES)

!pip install --user xlrd

import os
import pandas as pd




# Caminho da pasta
folder_path = r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\AcompFreq"

# DataFrame vazio
df_acompFREQ = pd.DataFrame()

# Loop pelos arquivos da pasta
for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
        file_path = os.path.join(folder_path, file_name)

        try:
            # Tenta com engine padrão (openpyxl para .xlsx)
            df_temp = pd.read_excel(file_path)
        except ValueError:
            # Se falhar, tenta com outro engine
            df_temp = pd.read_excel(file_path, engine='xlrd')  # geralmente para .xls antigos

        # Concatenação
        df_acompFREQ = pd.concat([df_acompFREQ, df_temp], ignore_index=True)

# Limpeza de dados
df_acompFREQ = df_acompFREQ.drop_duplicates()
df_acompFREQ['Cód. Paciente'] = df_acompFREQ['Cód. Paciente'].astype(str)
#df_acompFREQ['Cód. Paciente'] = df_acompFREQ['Cód. Paciente'].str.removesuffix('.0')

# Remove linhas totalmente vazias
df_acompFREQ = df_acompFREQ.dropna(how='all')

# Remove linhas finais com "Total" ou "filtros aplicados"
while df_acompFREQ.shape[0] > 0 and df_acompFREQ.iloc[-1].astype(str).str.contains('Total|filtros aplicados', case=False, na=False).any():
    df_acompFREQ = df_acompFREQ.iloc[:-1]


In [3]:
import os
import pandas as pd

# Caminho da pasta
folder_path = r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\AcompFreq"

# DataFrame vazio
df_acompFREQ = pd.DataFrame()

# Loop pelos arquivos .xlsx (evita arquivos .xls que precisam de xlrd)
for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx'):  # <-- apenas .xlsx
        file_path = os.path.join(folder_path, file_name)

        try:
            df_temp = pd.read_excel(file_path, engine='openpyxl')
            df_acompFREQ = pd.concat([df_acompFREQ, df_temp], ignore_index=True)
        except Exception as e:
            print(f"Erro ao ler {file_name}: {e}")

# Limpeza de dados
df_acompFREQ = df_acompFREQ.drop_duplicates()
df_acompFREQ['Cód. Paciente'] = df_acompFREQ['Cód. Paciente'].astype(str)
#df_acompFREQ['Cód. Paciente'] = df_acompFREQ['Cód. Paciente'].str.removesuffix('.0')
df_acompFREQ['Cód. Paciente'] = df_acompFREQ['Cód. Paciente'].str.removesuffix('.0')


# Remove linhas totalmente vazias
df_acompFREQ = df_acompFREQ.dropna(how='all')

# Remove linhas finais com "Total" ou "filtros aplicados"
while df_acompFREQ.shape[0] > 0 and df_acompFREQ.iloc[-1].astype(str).str.contains('Total|filtros aplicados', case=False, na=False).any():
    df_acompFREQ = df_acompFREQ.iloc[:-1]


In [4]:
df_acompFREQ.head()

,Cód. Paciente,Unidade,Especialidade,Setor,Celular,E-mail,Primeira Admissão,Última Admissão
0,9950184,CBV MATRIZ,OFTA,CBV MATRIZ CENTRO DIAGNOSTICO,61982400323,Não deseja informar e-mail,2024-11-04 00:00:00,2025-09-08
1,11484765,CBV MATRIZ,OFTA,CBV MATRIZ CENTRO DIAGNOSTICO,61984011214,Não deseja informar e-mail,2025-09-08 00:00:00,2025-09-08
2,8034250,HOSL,OFTA,HOSL PRIMEIRO BLOCO CIRURGICO,81987829575,Não deseja informar e-mail,2019-10-01 00:00:00,2025-09-08
3,8139694,FSL,OFTA,FSL TERREO CONSULTORIOS,81984741810,Não deseja informar e-mail,2018-11-01 00:00:00,2025-09-08
4,10268326,FSL,OFTA,FSL TERREO CONSULTORIOS,81986183932,naõ@bol.com.br,2025-02-03 00:00:00,2025-09-08


In [5]:
print("Especialidades: \n", df_acompFREQ['Especialidade'].value_counts())

Especialidades: 
 Especialidade
OFTA        182476
OTOR          8158
ANES           916
CIRUR           23
FISIO           17
AUX OFTA         2
Name: count, dtype: int64


# OBTER GRUPO HOSPITALAR (ACOMP FREQ PACIENTES)

In [6]:
df_acompFREQ2 = df_acompFREQ.copy()

In [7]:
# Lista de mapeamentos
map_hospitalar = {
    'HOC': ['HOC', 'HOVG'],
    'V1 SP': ['CLINOFT', 'HABC', 'HPAUL', 'MOLI', 'VISO', 'ULC', 'CEOSP'],
    'CERPO': ['DIAD', 'SAMARO', 'SANDRE', 'SCSUL', 'MAUA'],
    'LASER': ['LASER'],
    'HOPE': ['CEOFT', 'CEOFT (CEO)', 'CEOFT (DIFUSORA)', 'HORE', 'NEO', 'UASG', 'UASP', 'UASPT', 'UASR', 'UASRM', 'COVC', 'HOPE'],
    'HRO': ['HRO (GOLDEN)', 'HRO (ILHA)', 'HRO (MATRIZ)', 'HRO (RIO ANIL)', 'HRO (SUS)', 'VMAIS', 'HRO (ELLEVE)'],
    'OFTALMOS': ['OFTALMOS BC', 'OFTALMOS ITA', 'OFTALMOS MK'],
    'HOSL': ['FSL', 'HOSL'],
    'CBV': ['CBV MATRIZ', 'CBV AGUAS CLARAS', 'CBV TAGUATINGA', 'CBV LUMINAR', 'CBV HOSPITAL DE AGUAS CLARAS'],
    'HOA': ['HOA', 'HOSPITAL ARARAQUARA'],
    'HORP': ['HORP', 'TOKUNAGA']
    
}

# Função para mapear a unidade para o grupo hospitalar
def get_grupo_hospitalar(unidade):
    for grupo, unidades in map_hospitalar.items():
        if unidade in unidades:
            return grupo
    return None

# Aplicar a função ao DataFrame
df_acompFREQ2['Grupo Hospitalar'] = df_acompFREQ2['Unidade'].apply(get_grupo_hospitalar)

In [8]:
df_acompFREQ2.head()

,Cód. Paciente,Unidade,Especialidade,Setor,Celular,E-mail,Primeira Admissão,Última Admissão,Grupo Hospitalar
0,9950184,CBV MATRIZ,OFTA,CBV MATRIZ CENTRO DIAGNOSTICO,61982400323,Não deseja informar e-mail,2024-11-04 00:00:00,2025-09-08,CBV
1,11484765,CBV MATRIZ,OFTA,CBV MATRIZ CENTRO DIAGNOSTICO,61984011214,Não deseja informar e-mail,2025-09-08 00:00:00,2025-09-08,CBV
2,8034250,HOSL,OFTA,HOSL PRIMEIRO BLOCO CIRURGICO,81987829575,Não deseja informar e-mail,2019-10-01 00:00:00,2025-09-08,HOSL
3,8139694,FSL,OFTA,FSL TERREO CONSULTORIOS,81984741810,Não deseja informar e-mail,2018-11-01 00:00:00,2025-09-08,HOSL
4,10268326,FSL,OFTA,FSL TERREO CONSULTORIOS,81986183932,naõ@bol.com.br,2025-02-03 00:00:00,2025-09-08,HOSL


In [9]:
# VALIDANDO POR NULOS

null_values_in_A = df_acompFREQ2[df_acompFREQ2['Grupo Hospitalar'].isnull()]

In [10]:
null_values_in_A.head()

,Cód. Paciente,Unidade,Especialidade,Setor,Celular,E-mail,Primeira Admissão,Última Admissão,Grupo Hospitalar
37044,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaT,None
37045,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaT,None
37046,Filtros aplicados:\nAno_Atual é Ano Atual\nMes...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,None


In [11]:
# VALIDANDO

print(df_acompFREQ2[['Unidade', 'Grupo Hospitalar']].value_counts())

Unidade                       Grupo Hospitalar
HPAUL                         V1 SP               32309
HORE                          HOPE                32048
HABC                          V1 SP               22043
CBV MATRIZ                    CBV                 13744
HOSL                          HOSL                11600
SAMARO                        CERPO                7800
FSL                           HOSL                 5881
HOC                           HOC                  5165
UASR                          HOPE                 4647
HORP                          HORP                 3995
HRO (GOLDEN)                  HRO                  3893
SANDRE                        CERPO                3826
CEOFT (CEO)                   HOPE                 3587
HRO (MATRIZ)                  HRO                  3266
OFTALMOS BC                   OFTALMOS             3192
UASG                          HOPE                 3149
LASER                         LASER                2859
N

# OBTER COLUNA DE PRIMEIRA VEZ OU RECORRENTE (ACOMP FREQ PACIENTES)

In [12]:
df_acompFREQ2['Frequencia'] = df_acompFREQ2.apply(
    lambda row: 'Primeira Vez' if row['Primeira Admissão'] == row['Última Admissão'] 
    else 'Recorrente', axis=1)

In [13]:
# VALIDANDO POR NULOS

null_values_in_FREQ = df_acompFREQ2[df_acompFREQ2['Frequencia'].isnull()]

In [14]:
null_values_in_FREQ.head()

,Cód. Paciente,Unidade,Especialidade,Setor,Celular,E-mail,Primeira Admissão,Última Admissão,Grupo Hospitalar,Frequencia


In [15]:
# VALIDANDO

df_acompFREQ2['Primeira Admissão'] = pd.to_datetime(df_acompFREQ2['Primeira Admissão'], errors='coerce')
df_acompFREQ2['Última Admissão'] = pd.to_datetime(df_acompFREQ2['Última Admissão'], errors='coerce')

print(df_acompFREQ2[['Primeira Admissão', 'Última Admissão', 'Frequencia']].value_counts())

Primeira Admissão  Última Admissão  Frequencia  
2025-08-25         2025-08-25       Primeira Vez    1180
2025-08-20         2025-08-20       Primeira Vez    1177
2025-08-21         2025-08-21       Primeira Vez    1135
2025-09-02         2025-09-02       Primeira Vez    1089
2025-08-26         2025-08-26       Primeira Vez    1074
                                                    ... 
2016-09-29         2025-08-05       Recorrente         1
2016-09-27         2025-08-01       Recorrente         1
1900-01-02         2025-08-16       Recorrente         1
2025-09-04         2025-09-07       Recorrente         1
1900-01-02         2025-08-02       Recorrente         1
Name: count, Length: 78819, dtype: int64


In [16]:
df_acompFREQ2.head()

,Cód. Paciente,Unidade,Especialidade,Setor,Celular,E-mail,Primeira Admissão,Última Admissão,Grupo Hospitalar,Frequencia
0,9950184,CBV MATRIZ,OFTA,CBV MATRIZ CENTRO DIAGNOSTICO,61982400323,Não deseja informar e-mail,2024-11-04,2025-09-08,CBV,Recorrente
1,11484765,CBV MATRIZ,OFTA,CBV MATRIZ CENTRO DIAGNOSTICO,61984011214,Não deseja informar e-mail,2025-09-08,2025-09-08,CBV,Primeira Vez
2,8034250,HOSL,OFTA,HOSL PRIMEIRO BLOCO CIRURGICO,81987829575,Não deseja informar e-mail,2019-10-01,2025-09-08,HOSL,Recorrente
3,8139694,FSL,OFTA,FSL TERREO CONSULTORIOS,81984741810,Não deseja informar e-mail,2018-11-01,2025-09-08,HOSL,Recorrente
4,10268326,FSL,OFTA,FSL TERREO CONSULTORIOS,81986183932,naõ@bol.com.br,2025-02-03,2025-09-08,HOSL,Recorrente


# DEFINIR COLUNAS DE INTERESSE (ACOMP FREQ PACIENTES)

# COM ISSO CHEGAMOS NO NOSSO OBJETIVO PARA O ACOMP DA FREQ PACIENTES, QUE ERA:

USAR BASE PARA PEGAR ESPECIALIDADE, DATA PRIMEIRA ADMISSAO, DATA ULTIMA ADMISSAO, FREQUENCIA, SETOR, UNIDADE, GRUPO HOSPITALAR, CELULAR, EMAIL

In [17]:
colunas = ['Cód. Paciente', 'Celular', 'E-mail', 'Primeira Admissão', 'Última Admissão', 'Frequencia', 'Unidade', 
           'Grupo Hospitalar','Setor', 'Especialidade']

df_acompFREQ2 = df_acompFREQ2[colunas]

In [18]:
df_acompFREQ2.head()

,Cód. Paciente,Celular,E-mail,Primeira Admissão,Última Admissão,Frequencia,Unidade,Grupo Hospitalar,Setor,Especialidade
0,9950184,61982400323,Não deseja informar e-mail,2024-11-04,2025-09-08,Recorrente,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA
1,11484765,61984011214,Não deseja informar e-mail,2025-09-08,2025-09-08,Primeira Vez,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA
2,8034250,81987829575,Não deseja informar e-mail,2019-10-01,2025-09-08,Recorrente,HOSL,HOSL,HOSL PRIMEIRO BLOCO CIRURGICO,OFTA
3,8139694,81984741810,Não deseja informar e-mail,2018-11-01,2025-09-08,Recorrente,FSL,HOSL,FSL TERREO CONSULTORIOS,OFTA
4,10268326,81986183932,naõ@bol.com.br,2025-02-03,2025-09-08,Recorrente,FSL,HOSL,FSL TERREO CONSULTORIOS,OFTA


In [19]:
print("Tamanho AcompFreq: ", df_acompFREQ2.shape[0])

Tamanho AcompFreq:  191595


# /////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# /////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# LER BASES (CADASTRO)

In [20]:
# CADASTRO -> GERAL -> PASTA TODA

# Specify the folder path
folder_path = r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\Cadastro"


# Initialize an empty DataFrame to store the data
df_cadastro = pd.DataFrame()

# Loop through all files in the folder
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    try:
        if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
            # Read the Excel file into a DataFrame with specific columns
            df_temp = pd.read_excel(file_path, usecols=['Cód. Paciente', 'Paciente', 'Gênero', 'Nascimento'])
        elif file_name.endswith('.csv'):
            # Read the CSV file into a DataFrame with ';' as the delimiter and specific columns
            df_temp = pd.read_csv(file_path, sep=';', usecols=['Cód. Paciente', 'Paciente', 'Gênero', 'Nascimento'])
        else:
            # Skip files that are not Excel or CSV
            continue
        
        # Concatenate the current DataFrame with the overall DataFrame
        df_cadastro = pd.concat([df_cadastro, df_temp], ignore_index=True)
    except Exception as e:
        print(f"Erro ao processar o arquivo {file_name}: {e}")

df_cadastro = df_cadastro.drop_duplicates()
df_cadastro['Cód. Paciente'] = df_cadastro['Cód. Paciente'].astype(str)
df_cadastro['Cód. Paciente'] = df_cadastro['Cód. Paciente'].str.removesuffix('.0')

# Remover as linhas que contêm "Total", "filtros aplicados" ou são completamente vazias
df_cadastro = df_cadastro.dropna(how='all')  # Remove linhas totalmente nulas

# Remover as últimas linhas que contêm "Total" ou "filtros aplicados"
while (df_cadastro.iloc[-1].astype(str).str.contains('Total|Filtros aplicados', case=False, na=False).any()):
    df_cadastro = df_cadastro.iloc[:-1]  # Remove a última linha

C:\Users\MarceloFridmanMarizd\anaconda3_new\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [21]:
print("\nTamanho Cadastro Completo, Histórico + Atual: " ,df_cadastro.shape[0])


Tamanho Cadastro Completo, Histórico + Atual:  2201115


In [22]:
df_cadastro.head()

,Cód. Paciente,Paciente,Nascimento,Gênero
0,11472714,LAVINIA DIAS FERREIRA DE JESUS,2011-12-13 00:00:00,FEMININO
1,10353916,CAMILA BASSO,1999-10-06 00:00:00,FEMININO
2,11487346,FABIO KENDY YOSHIDA,1985-04-08 00:00:00,MASCULINO
3,8189336,AYRTON ALYSON SILVA DE ANDRADE,1996-07-27 00:00:00,MASCULINO
4,630327,MARIA BETANIA DO NASCIMENTO CANTALICE,1962-09-26 00:00:00,FEMININO


# OBTER IDADE (CADASTRO)

In [23]:
df_cadastro2 = df_cadastro.copy()

In [24]:
df_cadastro2['Nascimento'] = pd.to_datetime(df_cadastro2['Nascimento'], errors='coerce')
current_date = pd.to_datetime('today').normalize()
df_cadastro2['Idade'] = (current_date - df_cadastro2['Nascimento']).dt.days // 365.25

In [25]:
# VALIDANDO POR NULOS

null_values_in_IDADE = df_cadastro2[df_cadastro2['Idade'].isnull()]

In [26]:
null_values_in_IDADE.head()

,Cód. Paciente,Paciente,Nascimento,Gênero,Idade
41618,nan,NaN,NaT,NaN,NaN
1696184,5680662,RODRIGO BACELLAR YACOUB,NaT,MASCULINO,NaN
1726462,990342,RICARDO BRAYNNE GOMES COSTA,NaT,MASCULINO,NaN
3192844,11472714,NaN,NaT,NaN,NaN
3192845,10353916,NaN,NaT,NaN,NaN


In [27]:
# VALIDANDO

print(df_cadastro2[['Nascimento', 'Idade']].value_counts())

Nascimento  Idade 
1981-05-15   44.0     100
1984-03-01   41.0      98
1981-08-06   44.0      94
1979-10-01   45.0      92
1979-11-20   45.0      91
                     ... 
1900-01-04   125.0      1
1870-04-05   155.0      1
2048-01-25  -23.0       1
2025-06-07   0.0        1
2025-04-13   0.0        1
Name: count, Length: 35715, dtype: int64


# DEFINIR COLUNAS DE INTERESSE (CADASTRO)

# COM ISSO CHEGAMOS NO NOSSO OBJETIVO PARA CADASTRO, QUE ERA:

USAR BASE PARA PEGAR INFOS CADASTRAIS: COD PACIENTE, NOME, IDADE, GENERO

In [28]:
colunas = ['Cód. Paciente', 'Paciente', 'Idade', 'Gênero']

df_cadastro2 = df_cadastro2[colunas]

In [29]:
df_cadastro2.head()

,Cód. Paciente,Paciente,Idade,Gênero
0,11472714,LAVINIA DIAS FERREIRA DE JESUS,13.0,FEMININO
1,10353916,CAMILA BASSO,25.0,FEMININO
2,11487346,FABIO KENDY YOSHIDA,40.0,MASCULINO
3,8189336,AYRTON ALYSON SILVA DE ANDRADE,29.0,MASCULINO
4,630327,MARIA BETANIA DO NASCIMENTO CANTALICE,62.0,FEMININO


In [30]:
print("Tamanho Cadastro: ", df_cadastro2.shape[0])

Tamanho Cadastro:  2201115


# /////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# /////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# MERGE DAS DUAS BASES df_acompFREQ2 x df_cadastro2 (JUNTAR DATAS DE ACOMP FREQ COM INFOS CADASTRAIS)

In [31]:
df_cadACOMP = pd.merge(df_acompFREQ2, df_cadastro2, on='Cód. Paciente', how='inner')

In [32]:
df_cadACOMP.head()

,Cód. Paciente,Celular,E-mail,Primeira Admissão,Última Admissão,Frequencia,Unidade,Grupo Hospitalar,Setor,Especialidade,Paciente,Idade,Gênero
0,9950184,61982400323,Não deseja informar e-mail,2024-11-04,2025-09-08,Recorrente,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA,VERA LUCIA DE AREA LEAO NAVARRO,70.0,FEMININO
1,9950184,61982400323,Não deseja informar e-mail,2024-11-04,2025-09-08,Recorrente,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA,VERA LUCIA DE AREA LEAO NAVARRO,70.0,FEMININO
2,9950184,61982400323,Não deseja informar e-mail,2024-11-04,2025-09-08,Recorrente,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA,NaN,NaN,NaN
3,11484765,61984011214,Não deseja informar e-mail,2025-09-08,2025-09-08,Primeira Vez,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA,GERALDO MOACIR SOARES,71.0,MASCULINO
4,11484765,61984011214,Não deseja informar e-mail,2025-09-08,2025-09-08,Primeira Vez,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA,NaN,NaN,NaN


In [33]:
print("Tamanho AcompFreq + Cadastro (Merge): ", df_cadACOMP.shape[0])

Tamanho AcompFreq + Cadastro (Merge):  277519


# /////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# /////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# LER BASES (PRODUCOES)

In [34]:
# PROD CONSULTAS -> MES ATUAL

# Specify the folder path
folder_path = r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - Pasta Pública\ESTRATÉGIA\Marketing\SoluCX\Rodando\Qvis\Produção\Total"


# Initialize an empty DataFrame to store the data
df_consultas = pd.DataFrame()

# Loop through all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
        file_path = os.path.join(folder_path, file_name)
        
        # Read the Excel file into a DataFrame
        df_temp = pd.read_excel(file_path)

        df_temp = df_temp[  (df_temp['Categoria'] == 'Consultas' ) |
                            (df_temp['Categoria'] == 'Outros' )
        ]
        
        # Concatenate the current DataFrame with the overall DataFrame
        df_consultas = pd.concat([df_consultas, df_temp], ignore_index=True)

df_consultas = df_consultas.drop_duplicates()
df_consultas['Cód. Paciente'] = df_consultas['Cód. Paciente'].astype(str)
df_consultas['Cód. Paciente'] = df_consultas['Cód. Paciente'].str.removesuffix('.0')

# Remover as linhas que contêm "Total", "filtros aplicados" ou são completamente vazias
df_consultas = df_consultas.dropna(how='all')  # Remove linhas totalmente nulas

# Remover as últimas linhas que contêm "Total" ou "Filtros aplicados"
while (df_consultas.iloc[-1].astype(str).str.contains('Total|Filtros aplicados', case=False, na=False).any()):
    df_consultas = df_consultas.iloc[:-1]  # Remove a última linha

In [35]:
df_consultas.head()

,Cód. Admissão,Data Admissão,Tipo Produto,Categoria,Subcategoria,Caráter Admissão,Cód. Paciente,Solicitante,Médico
0,37854794,2025-09-13,CONSULTA,Consultas,Eletivas,ELETIVO,8218776,MARINA FREITAS,MARINA FREITAS
1,37854794,2025-09-13,AMBULATORIAL (OFTALMO),Outros,Procedimentos ambulatoriais,ELETIVO,8218776,NaN,MARINA FREITAS
2,37854887,2025-09-13,CONSULTA,Consultas,Eletivas,ELETIVO,8596059,MARINA FREITAS,MARINA FREITAS
3,37854887,2025-09-13,AMBULATORIAL (OFTALMO),Outros,Procedimentos ambulatoriais,ELETIVO,8596059,NaN,MARINA FREITAS
4,37854891,2025-09-13,CONSULTA,Consultas,Eletivas,ELETIVO,7942833,MARINA FREITAS,MARINA FREITAS


# OBTER COLUNA DE JORNADA (PRODUCAO CONSULTAS)

In [36]:
df_consultas2 = df_consultas.copy()

In [37]:
# Definir a função para mapear os valores de "Caráter Admissão" para "Jornada"
def map_jornada_consultas(carater_admissao):
    if carater_admissao == 'ELETIVO':
        return 'Avaliação Ambulatorial'
    elif carater_admissao == 'URGÊNCIA /EMERGÊNCIA':
        return 'Avaliação Urgência'
    return None

# Aplicar a função ao DataFrame
df_consultas2['Jornada'] = df_consultas2['Caráter Admissão'].apply(map_jornada_consultas)

In [38]:
# VALIDANDO POR NULOS

null_values_in_B = df_consultas2[df_consultas2['Jornada'].isnull()]

In [39]:
null_values_in_B.head()

,Cód. Admissão,Data Admissão,Tipo Produto,Categoria,Subcategoria,Caráter Admissão,Cód. Paciente,Solicitante,Médico,Jornada


In [40]:
# VALIDANDO

print(df_consultas2[['Caráter Admissão', 'Jornada']].value_counts())

Caráter Admissão      Jornada               
ELETIVO               Avaliação Ambulatorial    155086
URGÊNCIA /EMERGÊNCIA  Avaliação Urgência         30806
Name: count, dtype: int64


# CRIAR NOVA COLUNA PARA CLASSIFICAR O PRODUTO, com base na Categoria

In [41]:
df_consultas2.head()

,Cód. Admissão,Data Admissão,Tipo Produto,Categoria,Subcategoria,Caráter Admissão,Cód. Paciente,Solicitante,Médico,Jornada
0,37854794,2025-09-13,CONSULTA,Consultas,Eletivas,ELETIVO,8218776,MARINA FREITAS,MARINA FREITAS,Avaliação Ambulatorial
1,37854794,2025-09-13,AMBULATORIAL (OFTALMO),Outros,Procedimentos ambulatoriais,ELETIVO,8218776,NaN,MARINA FREITAS,Avaliação Ambulatorial
2,37854887,2025-09-13,CONSULTA,Consultas,Eletivas,ELETIVO,8596059,MARINA FREITAS,MARINA FREITAS,Avaliação Ambulatorial
3,37854887,2025-09-13,AMBULATORIAL (OFTALMO),Outros,Procedimentos ambulatoriais,ELETIVO,8596059,NaN,MARINA FREITAS,Avaliação Ambulatorial
4,37854891,2025-09-13,CONSULTA,Consultas,Eletivas,ELETIVO,7942833,MARINA FREITAS,MARINA FREITAS,Avaliação Ambulatorial


In [42]:
# Criar a nova coluna 'PRODUTO' e preencher com o valor 'CONSULTA'
df_consultas2['Produto'] = 'CONSULTA'

In [43]:
df_consultas2.head()

,Cód. Admissão,Data Admissão,Tipo Produto,Categoria,Subcategoria,Caráter Admissão,Cód. Paciente,Solicitante,Médico,Jornada,Produto
0,37854794,2025-09-13,CONSULTA,Consultas,Eletivas,ELETIVO,8218776,MARINA FREITAS,MARINA FREITAS,Avaliação Ambulatorial,CONSULTA
1,37854794,2025-09-13,AMBULATORIAL (OFTALMO),Outros,Procedimentos ambulatoriais,ELETIVO,8218776,NaN,MARINA FREITAS,Avaliação Ambulatorial,CONSULTA
2,37854887,2025-09-13,CONSULTA,Consultas,Eletivas,ELETIVO,8596059,MARINA FREITAS,MARINA FREITAS,Avaliação Ambulatorial,CONSULTA
3,37854887,2025-09-13,AMBULATORIAL (OFTALMO),Outros,Procedimentos ambulatoriais,ELETIVO,8596059,NaN,MARINA FREITAS,Avaliação Ambulatorial,CONSULTA
4,37854891,2025-09-13,CONSULTA,Consultas,Eletivas,ELETIVO,7942833,MARINA FREITAS,MARINA FREITAS,Avaliação Ambulatorial,CONSULTA


In [44]:
print("Tamanho Consultas: ", df_consultas2.shape[0])

Tamanho Consultas:  185892


## PRODUCAO EXAMES

In [45]:
# PROD EXAMES -> MES ATUAL

# Specify the folder path
folder_path = r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - Pasta Pública\ESTRATÉGIA\Marketing\SoluCX\Rodando\Qvis\Produção\Total"

# Initialize an empty DataFrame to store the data
df_exames = pd.DataFrame()

# Loop through all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
        file_path = os.path.join(folder_path, file_name)
        
        # Read the Excel file into a DataFrame
        df_temp = pd.read_excel(file_path)

        df_temp = df_temp[  (df_temp['Categoria'] == 'Exames' ) 
        ]
        
        # Concatenate the current DataFrame with the overall DataFrame
        df_exames = pd.concat([df_exames, df_temp], ignore_index=True)

df_exames = df_exames.drop_duplicates()
df_exames['Cód. Paciente'] = df_exames['Cód. Paciente'].astype(str)
df_exames['Cód. Paciente'] = df_exames['Cód. Paciente'].str.removesuffix('.0')

# Remover as linhas que contêm "Total", "filtros aplicados" ou são completamente vazias
df_exames = df_exames.dropna(how='all')  # Remove linhas totalmente nulas

# Remover as últimas linhas que contêm "Total" ou "Filtros aplicados"
while (df_exames.iloc[-1].astype(str).str.contains('Total|Filtros aplicados', case=False, na=False).any()):
    df_exames = df_exames.iloc[:-1]  # Remove a última linha

In [46]:
df_exames.head()

,Cód. Admissão,Data Admissão,Tipo Produto,Categoria,Subcategoria,Caráter Admissão,Cód. Paciente,Solicitante,Médico
0,38002059,2025-09-07,OFTALMO TIPO I,Exames,Exame Médico,URGÊNCIA /EMERGÊNCIA,275409,LUAN FELLIPE BISPO ALMEIDA,LUAN FELLIPE BISPO ALMEIDA
1,38001979,2025-09-07,OFTALMO TIPO I,Exames,Exame Médico,URGÊNCIA /EMERGÊNCIA,8265384,AMANDA HENRIQUES TERRA,AMANDA HENRIQUES TERRA
2,38002171,2025-09-07,OFTALMO TIPO I,Exames,Exame Médico,URGÊNCIA /EMERGÊNCIA,1198725,LUAN FELLIPE BISPO ALMEIDA,JULIANA SOUZA GOMES
3,38001839,2025-09-07,OFTALMO TIPO I,Exames,Exame Médico,URGÊNCIA /EMERGÊNCIA,363511,AMANDA HENRIQUES TERRA,NaN
4,38001670,2025-09-07,OFTALMO TIPO I,Exames,Exame Médico,URGÊNCIA /EMERGÊNCIA,7849612,ALEXANDRE DANTAS,ALEXANDRE DANTAS


# OBTER COLUNA DE JORNADA (PRODUCAO EXAMES)

In [47]:
df_exames2 = df_exames.copy()

In [48]:
# Definir a função para mapear os valores de "Caráter Admissão" para "Jornada"
def map_jornada_exames(carater_admissao):
    if carater_admissao == 'ELETIVO':
        return 'Avaliação Exames'
    elif carater_admissao == 'URGÊNCIA /EMERGÊNCIA':
        return 'Avaliação Urgência'
    return None

# Aplicar a função ao DataFrame
df_exames2['Jornada'] = df_exames2['Caráter Admissão'].apply(map_jornada_exames)

In [49]:
# VALIDANDO POR NULOS

null_values_in_D = df_exames2[df_exames2['Jornada'].isnull()]

In [50]:
null_values_in_D.head()

,Cód. Admissão,Data Admissão,Tipo Produto,Categoria,Subcategoria,Caráter Admissão,Cód. Paciente,Solicitante,Médico,Jornada


In [51]:
# VALIDANDO

print(df_exames2[['Caráter Admissão', 'Jornada']].value_counts())

Caráter Admissão      Jornada           
ELETIVO               Avaliação Exames      119685
URGÊNCIA /EMERGÊNCIA  Avaliação Urgência       521
Name: count, dtype: int64


# CRIAR NOVA COLUNA PARA CLASSIFICAR O PRODUTO, VALOR CONSTANTE (PRODUCAO EXAMES)

In [52]:
# Criar a nova coluna 'PRODUTO' e preencher com o valor 'EXAME'
df_exames2['Produto'] = 'EXAME'

In [53]:
df_exames2.head()

,Cód. Admissão,Data Admissão,Tipo Produto,Categoria,Subcategoria,Caráter Admissão,Cód. Paciente,Solicitante,Médico,Jornada,Produto
0,38002059,2025-09-07,OFTALMO TIPO I,Exames,Exame Médico,URGÊNCIA /EMERGÊNCIA,275409,LUAN FELLIPE BISPO ALMEIDA,LUAN FELLIPE BISPO ALMEIDA,Avaliação Urgência,EXAME
1,38001979,2025-09-07,OFTALMO TIPO I,Exames,Exame Médico,URGÊNCIA /EMERGÊNCIA,8265384,AMANDA HENRIQUES TERRA,AMANDA HENRIQUES TERRA,Avaliação Urgência,EXAME
2,38002171,2025-09-07,OFTALMO TIPO I,Exames,Exame Médico,URGÊNCIA /EMERGÊNCIA,1198725,LUAN FELLIPE BISPO ALMEIDA,JULIANA SOUZA GOMES,Avaliação Urgência,EXAME
3,38001839,2025-09-07,OFTALMO TIPO I,Exames,Exame Médico,URGÊNCIA /EMERGÊNCIA,363511,AMANDA HENRIQUES TERRA,NaN,Avaliação Urgência,EXAME
4,38001670,2025-09-07,OFTALMO TIPO I,Exames,Exame Médico,URGÊNCIA /EMERGÊNCIA,7849612,ALEXANDRE DANTAS,ALEXANDRE DANTAS,Avaliação Urgência,EXAME


In [54]:
print("Tamanho Exames: ", df_exames2.shape[0])

Tamanho Exames:  120206


# PRODUCAO CIRURGIAS (TOTAL)

In [55]:
# PROD CIRURGIAS -> MES ATUAL

# Specify the folder path
folder_path = r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - Pasta Pública\ESTRATÉGIA\Marketing\SoluCX\Rodando\Qvis\Produção\Total"

# Initialize an empty DataFrame to store the data
df_cirurgias = pd.DataFrame()

# Loop through all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
        file_path = os.path.join(folder_path, file_name)
        
        # Read the Excel file into a DataFrame
        df_temp = pd.read_excel(file_path)

        df_temp = df_temp[   df_temp['Categoria'] == 'Cirurgias'  ]
        
        # Concatenate the current DataFrame with the overall DataFrame
        df_cirurgias = pd.concat([df_cirurgias, df_temp], ignore_index=True)

df_cirurgias = df_cirurgias.drop_duplicates()
df_cirurgias['Cód. Paciente'] = df_cirurgias['Cód. Paciente'].astype(str)
df_cirurgias['Cód. Paciente'] = df_cirurgias['Cód. Paciente'].str.removesuffix('.0')

# Remover as linhas que contêm "Total", "filtros aplicados" ou são completamente vazias
df_cirurgias = df_cirurgias.dropna(how='all')  # Remove linhas totalmente nulas

# Remover as últimas linhas que contêm "Total" ou "Filtros aplicados"
while (df_cirurgias.iloc[-1].astype(str).str.contains('Total|Filtros aplicados', case=False, na=False).any()):
    df_cirurgias = df_cirurgias.iloc[:-1]  # Remove a última linha

In [56]:
df_cirurgias.head()

,Cód. Admissão,Data Admissão,Tipo Produto,Categoria,Subcategoria,Caráter Admissão,Cód. Paciente,Solicitante,Médico
0,37997802,2025-09-08,CIRURGICO (OFTALMO),Cirurgias,Plástica,ELETIVO,5245355,FABIANA PIOVEZAN FRANCO,NaN
1,37992958,2025-09-08,CIRURGICO (OFTALMO),Cirurgias,Catarata,ELETIVO,5273087,ELMER SALVIANO,NaN
2,37993027,2025-09-08,CIRURGICO (OFTALMO),Cirurgias,Catarata,ELETIVO,9048633,ELMER SALVIANO,NaN
3,37993156,2025-09-08,CIRURGICO (OFTALMO),Cirurgias,Córnea,ELETIVO,5169328,ARIANE ALESSIO,NaN
4,37995338,2025-09-08,CIRURGICO (OFTALMO),Cirurgias,Plástica,ELETIVO,10333563,FABIANA PIOVEZAN FRANCO,NaN


# RETIRAR 'NAO-CATEGORIZADO' DE SUBCATEGORIA (PRODUCAO CIRURGIAS)

In [57]:
df_cirurgias2 = df_cirurgias.copy()

In [58]:
# Passo 1: Filtrar o DataFrame para remover linhas com "Não Categorizado" e " Cirurgia Médico Externo' na coluna "Subcategoria"
df_cirurgias2 = df_cirurgias2[df_cirurgias2['Subcategoria'] != 'Não Categorizado']
df_cirurgias2 = df_cirurgias2[df_cirurgias2['Subcategoria'] != 'Cirurgia Médico Externo']

In [59]:
# VALIDANDO

print(df_cirurgias2['Subcategoria'].value_counts())

Subcategoria
Catarata            11803
Injeção              4920
Plástica             2874
Córnea               1640
Retina               1372
Refrativa            1265
Outras Cirurgias      908
Glaucoma              435
Otorrino              368
Name: count, dtype: int64


# OBTER COLUNA DE JORNADA (PRODUCAO CIRURGIAS)

In [60]:
# Definir a função para mapear os valores de "Caráter Admissão" e "Subcategoria" para "Jornada"
def map_jornada_cirurgias(carater_admissao, subcategoria):
    if carater_admissao == 'ELETIVO':
        if subcategoria == 'Laser':
            return 'Avaliação Exames'
        elif subcategoria != 'Injeção':
            return 'Avaliação Cirúrgica'
        elif subcategoria == 'Injeção':
            return 'Avaliação Injeção'
    elif carater_admissao == 'URGÊNCIA /EMERGÊNCIA':
        return 'Avaliação Urgência'
    else:
        return None

# Aplicar a função ao DataFrame para criar a nova coluna "Jornada"
df_cirurgias2['Jornada'] = df_cirurgias2.apply(lambda row: map_jornada_cirurgias
                                               (row['Caráter Admissão'], row['Subcategoria']), axis=1)

In [61]:
# VALIDANDO POR NULOS

null_values_in_F = df_cirurgias2[df_cirurgias2['Jornada'].isnull()]

In [62]:
null_values_in_F.head()

,Cód. Admissão,Data Admissão,Tipo Produto,Categoria,Subcategoria,Caráter Admissão,Cód. Paciente,Solicitante,Médico,Jornada


In [63]:
# VALIDANDO

print(df_cirurgias2[['Caráter Admissão', 'Subcategoria', 'Jornada']].value_counts())

Caráter Admissão      Subcategoria      Jornada            
ELETIVO               Catarata          Avaliação Cirúrgica    11787
                      Injeção           Avaliação Injeção       4913
                      Plástica          Avaliação Cirúrgica     2817
                      Córnea            Avaliação Cirúrgica     1621
                      Refrativa         Avaliação Cirúrgica     1263
                      Retina            Avaliação Cirúrgica     1237
                      Outras Cirurgias  Avaliação Cirúrgica      874
                      Glaucoma          Avaliação Cirúrgica      412
                      Otorrino          Avaliação Cirúrgica      350
URGÊNCIA /EMERGÊNCIA  Retina            Avaliação Urgência       135
                      Plástica          Avaliação Urgência        57
                      Outras Cirurgias  Avaliação Urgência        34
                      Glaucoma          Avaliação Urgência        23
                      Córnea            Ava

# CRIAR NOVA COLUNA PARA CLASSIFICAR O PRODUTO, VALOR CONSTANTE (PRODUCAO CIRURGIAS)

# PARA ESSE CASO, QUANDO LASER, SERA PRODUTO EXAME, O CONTRARIO CIRURGIA

In [64]:
# Criar a nova coluna 'Produto' com base no valor da coluna 'Jornada'
df_cirurgias2['Produto'] = df_cirurgias2['Jornada'].apply(lambda x: 'EXAME' if x == 'Avaliação Exames' else 'CIRURGIA')

In [65]:
# VALIDANDO

print(df_cirurgias2[['Jornada', 'Subcategoria', 'Produto']].value_counts())

Jornada              Subcategoria      Produto 
Avaliação Cirúrgica  Catarata          CIRURGIA    11787
Avaliação Injeção    Injeção           CIRURGIA     4913
Avaliação Cirúrgica  Plástica          CIRURGIA     2817
                     Córnea            CIRURGIA     1621
                     Refrativa         CIRURGIA     1263
                     Retina            CIRURGIA     1237
                     Outras Cirurgias  CIRURGIA      874
                     Glaucoma          CIRURGIA      412
                     Otorrino          CIRURGIA      350
Avaliação Urgência   Retina            CIRURGIA      135
                     Plástica          CIRURGIA       57
                     Outras Cirurgias  CIRURGIA       34
                     Glaucoma          CIRURGIA       23
                     Córnea            CIRURGIA       19
                     Otorrino          CIRURGIA       18
                     Catarata          CIRURGIA       16
                     Injeção           C

In [66]:
df_cirurgias2.head()

,Cód. Admissão,Data Admissão,Tipo Produto,Categoria,Subcategoria,Caráter Admissão,Cód. Paciente,Solicitante,Médico,Jornada,Produto
0,37997802,2025-09-08,CIRURGICO (OFTALMO),Cirurgias,Plástica,ELETIVO,5245355,FABIANA PIOVEZAN FRANCO,NaN,Avaliação Cirúrgica,CIRURGIA
1,37992958,2025-09-08,CIRURGICO (OFTALMO),Cirurgias,Catarata,ELETIVO,5273087,ELMER SALVIANO,NaN,Avaliação Cirúrgica,CIRURGIA
2,37993027,2025-09-08,CIRURGICO (OFTALMO),Cirurgias,Catarata,ELETIVO,9048633,ELMER SALVIANO,NaN,Avaliação Cirúrgica,CIRURGIA
3,37993156,2025-09-08,CIRURGICO (OFTALMO),Cirurgias,Córnea,ELETIVO,5169328,ARIANE ALESSIO,NaN,Avaliação Cirúrgica,CIRURGIA
4,37995338,2025-09-08,CIRURGICO (OFTALMO),Cirurgias,Plástica,ELETIVO,10333563,FABIANA PIOVEZAN FRANCO,NaN,Avaliação Cirúrgica,CIRURGIA


In [67]:
print("Tamanho Cirurgias: ", df_cirurgias2.shape[0])

Tamanho Cirurgias:  25585


# PRODUCAO LENTES

In [68]:
# PROD LENTES -> MES ATUAL

# Specify the folder path
folder_path = r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - Pasta Pública\ESTRATÉGIA\Marketing\SoluCX\Rodando\Qvis\Produção\Total"

# Initialize an empty DataFrame to store the data
df_lentes = pd.DataFrame()

# Loop through all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
        file_path = os.path.join(folder_path, file_name)
        
        # Read the Excel file into a DataFrame
        df_temp = pd.read_excel(file_path)

        df_temp = df_temp[   df_temp['Categoria'] == 'Lentes de Contato'  ]
        
        # Concatenate the current DataFrame with the overall DataFrame
        df_lentes = pd.concat([df_lentes, df_temp], ignore_index=True)

#df_lentes = df_lentes.drop_duplicates()
#df_lentes['Cód. Paciente'] = df_lentes['Cód. Paciente'].astype(str)
#df_lentes['Cód. Paciente'] = df_lentes['Cód. Paciente'].str.removesuffix('.0')

# Remover as linhas que contêm "Total", "filtros aplicados" ou são completamente vazias
df_lentes = df_lentes.dropna(how='all')  # Remove linhas totalmente nulas

df_lentes = df_lentes.drop_duplicates()
df_lentes['Cód. Paciente'] = df_lentes['Cód. Paciente'].astype(str)
df_lentes['Cód. Paciente'] = df_lentes['Cód. Paciente'].str.removesuffix('.0')

# Remover as linhas que contêm "Total", "filtros aplicados" ou são completamente vazias
df_lentes = df_lentes.dropna(how='all')  # Remove linhas totalmente nulas

# Remover as últimas linhas que contêm "Total" ou "Filtros aplicados"
while (df_lentes.iloc[-1].astype(str).str.contains('Total|Filtros aplicados', case=False, na=False).any()):
    df_lentes = df_lentes.iloc[:-1]  # Remove a última linha

In [69]:
df_lentes.head()

,Cód. Admissão,Data Admissão,Tipo Produto,Categoria,Subcategoria,Caráter Admissão,Cód. Paciente,Solicitante,Médico
0,37999649,2025-09-06,LENTE DE CONTATO,Lentes de Contato,Lentes de Contato,ELETIVO,6211986,GEISA SILVA,GEISA SILVA
1,37999716,2025-09-06,LENTE DE CONTATO,Lentes de Contato,Lentes de Contato,ELETIVO,5755130,PEDRO POLETTO,PEDRO POLETTO
2,37999729,2025-09-06,LENTE DE CONTATO,Lentes de Contato,Lentes de Contato,ELETIVO,9049237,LILIAN GRESPAN,LILIAN GRESPAN
3,37999734,2025-09-06,LENTE DE CONTATO,Lentes de Contato,Lentes de Contato,ELETIVO,5950696,LEOPOLDO RIBEIRO,LEOPOLDO RIBEIRO
4,37999810,2025-09-06,LENTE DE CONTATO,Lentes de Contato,Lentes de Contato,ELETIVO,5521167,LENTE DE CONTATO,LENTE DE CONTATO


# OBTER COLUNA DE JORNADA (PRODUCAO LENTES)

In [70]:
df_lentes2 = df_lentes.copy()

In [71]:
# Definir a função para mapear os valores de "Caráter Admissão" para "Jornada"
def map_jornada_lentes(carater_admissao):
    if carater_admissao == 'ELETIVO':
        return 'Avaliação Lente de Contato' 
    elif carater_admissao == 'URGÊNCIA /EMERGÊNCIA':
        return 'Avaliação Lente de Contato' 
    else: None

# Aplicar a função ao DataFrame
df_lentes2['Jornada'] = df_lentes2['Caráter Admissão'].apply(map_jornada_lentes)

In [72]:
# VALIDANDO POR NULOS

null_values_in_H = df_lentes2[df_lentes2['Jornada'].isnull()]

In [73]:
null_values_in_H.head()

,Cód. Admissão,Data Admissão,Tipo Produto,Categoria,Subcategoria,Caráter Admissão,Cód. Paciente,Solicitante,Médico,Jornada


In [74]:
# VALIDANDO

print(df_lentes2[['Caráter Admissão', 'Jornada']].value_counts())

Caráter Admissão  Jornada                   
ELETIVO           Avaliação Lente de Contato    3908
Name: count, dtype: int64


# CRIAR NOVA COLUNA PARA CLASSIFICAR O PRODUTO, VALOR CONSTANTE (PRODUCAO LENTES)

In [75]:
# Criar a nova coluna 'PRODUTO' e preencher com o valor 'LENTE'
df_lentes2['Produto'] = 'LENTE'

In [76]:
df_lentes2.head()

,Cód. Admissão,Data Admissão,Tipo Produto,Categoria,Subcategoria,Caráter Admissão,Cód. Paciente,Solicitante,Médico,Jornada,Produto
0,37999649,2025-09-06,LENTE DE CONTATO,Lentes de Contato,Lentes de Contato,ELETIVO,6211986,GEISA SILVA,GEISA SILVA,Avaliação Lente de Contato,LENTE
1,37999716,2025-09-06,LENTE DE CONTATO,Lentes de Contato,Lentes de Contato,ELETIVO,5755130,PEDRO POLETTO,PEDRO POLETTO,Avaliação Lente de Contato,LENTE
2,37999729,2025-09-06,LENTE DE CONTATO,Lentes de Contato,Lentes de Contato,ELETIVO,9049237,LILIAN GRESPAN,LILIAN GRESPAN,Avaliação Lente de Contato,LENTE
3,37999734,2025-09-06,LENTE DE CONTATO,Lentes de Contato,Lentes de Contato,ELETIVO,5950696,LEOPOLDO RIBEIRO,LEOPOLDO RIBEIRO,Avaliação Lente de Contato,LENTE
4,37999810,2025-09-06,LENTE DE CONTATO,Lentes de Contato,Lentes de Contato,ELETIVO,5521167,LENTE DE CONTATO,LENTE DE CONTATO,Avaliação Lente de Contato,LENTE


In [77]:
print("Tamanho Lentes: ", df_lentes2.shape[0])

Tamanho Lentes:  3908


# CONCATENAR DF's DE PRODUCAO

In [78]:
df_producao = pd.concat([df_consultas2, df_exames2, df_cirurgias2, df_lentes2], ignore_index=True)

In [79]:
print("Tamanho Consultas + Exames + Cirurgias + Lentes (Concat): ", df_producao.shape[0])

Tamanho Consultas + Exames + Cirurgias + Lentes (Concat):  335591


In [80]:
df_producao2 = df_producao.copy()

In [81]:
df_producao2.head()

,Cód. Admissão,Data Admissão,Tipo Produto,Categoria,Subcategoria,Caráter Admissão,Cód. Paciente,Solicitante,Médico,Jornada,Produto
0,37854794,2025-09-13,CONSULTA,Consultas,Eletivas,ELETIVO,8218776,MARINA FREITAS,MARINA FREITAS,Avaliação Ambulatorial,CONSULTA
1,37854794,2025-09-13,AMBULATORIAL (OFTALMO),Outros,Procedimentos ambulatoriais,ELETIVO,8218776,NaN,MARINA FREITAS,Avaliação Ambulatorial,CONSULTA
2,37854887,2025-09-13,CONSULTA,Consultas,Eletivas,ELETIVO,8596059,MARINA FREITAS,MARINA FREITAS,Avaliação Ambulatorial,CONSULTA
3,37854887,2025-09-13,AMBULATORIAL (OFTALMO),Outros,Procedimentos ambulatoriais,ELETIVO,8596059,NaN,MARINA FREITAS,Avaliação Ambulatorial,CONSULTA
4,37854891,2025-09-13,CONSULTA,Consultas,Eletivas,ELETIVO,7942833,MARINA FREITAS,MARINA FREITAS,Avaliação Ambulatorial,CONSULTA


In [82]:
# VALIDANDO POR NULOS

null_values_in_J = df_producao2[df_producao2['Jornada'].isnull()]

In [83]:
null_values_in_J.head()

,Cód. Admissão,Data Admissão,Tipo Produto,Categoria,Subcategoria,Caráter Admissão,Cód. Paciente,Solicitante,Médico,Jornada,Produto


In [84]:
# VALIDANDO POR NULOS

null_values_in_L = df_producao2[df_producao2['Produto'].isnull()]

In [85]:
null_values_in_L.head()

,Cód. Admissão,Data Admissão,Tipo Produto,Categoria,Subcategoria,Caráter Admissão,Cód. Paciente,Solicitante,Médico,Jornada,Produto


# DEFINIR COLUNAS DE INTERESSE (PRODUCAO)

# COM ISSO CHEGAMOS NO NOSSO OBJETIVO PARA AS BASES DE PRODUCAO, QUE ERA:

USAR BASE PARA PEGAR COD ADMISSAO, DATA ADMISSAO, JORNADA, SUBCATEGORIA, SOLICITANTE, MEDICO

In [86]:
colunas = ['Cód. Paciente', 'Cód. Admissão', 'Produto', 'Subcategoria', 'Data Admissão', 'Caráter Admissão', 
           'Tipo Produto', 'Solicitante', 'Médico', 'Jornada']

df_producao2 = df_producao2[colunas]

In [87]:
df_producao2.head()

,Cód. Paciente,Cód. Admissão,Produto,Subcategoria,Data Admissão,Caráter Admissão,Tipo Produto,Solicitante,Médico,Jornada
0,8218776,37854794,CONSULTA,Eletivas,2025-09-13,ELETIVO,CONSULTA,MARINA FREITAS,MARINA FREITAS,Avaliação Ambulatorial
1,8218776,37854794,CONSULTA,Procedimentos ambulatoriais,2025-09-13,ELETIVO,AMBULATORIAL (OFTALMO),NaN,MARINA FREITAS,Avaliação Ambulatorial
2,8596059,37854887,CONSULTA,Eletivas,2025-09-13,ELETIVO,CONSULTA,MARINA FREITAS,MARINA FREITAS,Avaliação Ambulatorial
3,8596059,37854887,CONSULTA,Procedimentos ambulatoriais,2025-09-13,ELETIVO,AMBULATORIAL (OFTALMO),NaN,MARINA FREITAS,Avaliação Ambulatorial
4,7942833,37854891,CONSULTA,Eletivas,2025-09-13,ELETIVO,CONSULTA,MARINA FREITAS,MARINA FREITAS,Avaliação Ambulatorial


In [88]:
print("Tamanho Consultas + Exames + Cirurgias + Lentes (Concat) Após Filtrar Dia da Base: ", df_producao2.shape[0])

Tamanho Consultas + Exames + Cirurgias + Lentes (Concat) Após Filtrar Dia da Base:  335591


In [89]:
print('Volumetria Diária QVIS= {}'.format(df_producao2['Cód. Paciente'].nunique()))

Volumetria Diária QVIS= 132285


# /////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# /////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# OBTER DATA MAX (ULTIMA ADM) VERDADEIRA -> PREVINIR ERROS QVIS -> FAZENDO MANUALMENTE

# PRODUCAO

In [90]:
df_prodMAX = df_producao2.copy()
df_prodMAX = df_prodMAX[['Cód. Paciente', 'Data Admissão']]
df_prodMAX['Data Admissão'] = pd.to_datetime(df_prodMAX['Data Admissão'])

# Encontrando a data máxima de admissao para cada paciente
df_prodMAX = df_prodMAX.groupby('Cód. Paciente', as_index=False)['Data Admissão'].max()
df_prodMAX.rename(columns={'Data Admissão': 'Data Admissão MAX'}, inplace=True)

In [91]:
df_prodMAX.head()

,Cód. Paciente,Data Admissão MAX
0,10000044,2025-08-20
1,10000085,2025-08-14
2,10000100,2025-08-06
3,10000169,2025-09-08
4,10000196,2025-08-23


# ACOMP FREQ

In [92]:
df_acompMAX = df_cadACOMP.copy()
df_acompMAX = df_acompMAX[['Cód. Paciente', 'Última Admissão']]
df_acompMAX['Última Admissão'] = pd.to_datetime(df_acompMAX['Última Admissão'])

# Encontrando a data máxima de admissao para cada paciente
df_acompMAX = df_acompMAX.groupby('Cód. Paciente', as_index=False)['Última Admissão'].max()
df_acompMAX.rename(columns={'Última Admissão': 'Data Admissão MAX'}, inplace=True)

In [93]:
df_acompMAX.head()

,Cód. Paciente,Data Admissão MAX
0,10000044,2025-08-20
1,10000085,2025-08-14
2,10000100,2025-08-06
3,10000169,2025-09-08
4,10000196,2025-08-23


# CONCAT PROD COM ACOMP FREQ -> POSTERIORMENTE AGRUPANDO DE NOVO NO COD PACIENTE E MAX DATA, AGORA VERDADEIRA

In [94]:
print("Prod: ", df_prodMAX.shape[0])
print("\nAcomp Freq: ", df_acompMAX.shape[0])

print("\nProd + Acomp Freq: ", df_acompMAX.shape[0] + df_prodMAX.shape[0])

Prod:  132285

Acomp Freq:  78929

Prod + Acomp Freq:  211214


In [95]:
df_dataMaxReal = pd.concat([df_prodMAX, df_acompMAX], axis=0)

In [96]:
df_dataMaxReal.head()

,Cód. Paciente,Data Admissão MAX
0,10000044,2025-08-20
1,10000085,2025-08-14
2,10000100,2025-08-06
3,10000169,2025-09-08
4,10000196,2025-08-23


In [97]:
print("Data Max Real: ", df_dataMaxReal.shape[0])

Data Max Real:  211214


In [98]:
# AGRUPAR POR COD PACIENTE E DATA MAX, PRA DE FATO TER A ULTIMA DATA DE ADMISSAO

# Encontrando a data máxima de admissao para cada paciente
df_dataMaxReal['Data Verdadeira MAX'] = df_dataMaxReal.groupby('Cód. Paciente')['Data Admissão MAX'].transform('max')

df_dataMaxReal = df_dataMaxReal[['Cód. Paciente', 'Data Verdadeira MAX']]

df_dataMaxReal = df_dataMaxReal.rename(columns={'Data Verdadeira MAX': 'Última Admissão'})

In [99]:
df_dataMaxReal.head()

,Cód. Paciente,Última Admissão
0,10000044,2025-08-20
1,10000085,2025-08-14
2,10000100,2025-08-06
3,10000169,2025-09-08
4,10000196,2025-08-23


# MERGE DAS DUAS BASES df_cadACOMP2 x df_producao2 (OBTER BASE FINAL)

# E MERGE NO DF DATA MAX REAL, PRA DE FATO TER A BASE FINAL COM A DATA ULTIMA ADM VERDADEIRA

In [100]:
df_cadACOMP2 = df_cadACOMP.copy()
df_producao3 = df_producao2.copy()

In [101]:
# Renomear a coluna 'Data Admissão' em df_producao2 para 'Última Admissão'
df_producao3 = df_producao3.rename(columns={'Data Admissão': 'Última Admissão'})

# Realizar o merge df_acompFREQ2 com df_producao3
df_PREfinal = pd.merge(df_cadACOMP2, df_producao3, on='Cód. Paciente', how='inner')

# Combinar as colunas 'Última Admissão_x' e 'Última Admissão_y' em uma só
df_PREfinal['Última Admissão'] = df_PREfinal['Última Admissão_x'].combine_first(df_PREfinal['Última Admissão_y'])

# Remover as colunas 'Última Admissão_x' e 'Última Admissão_y'
df_PREfinal = df_PREfinal.drop(columns=['Última Admissão_x', 'Última Admissão_y'])

# Converter as colunas de datas para o tipo datetime
df_PREfinal['Última Admissão'] = pd.to_datetime(df_PREfinal['Última Admissão'])

In [102]:
# Corrigir possível erro de nomes nas colunas do merge
df_PREfinal.columns = df_PREfinal.columns.str.strip()
df_dataMaxReal.columns = df_dataMaxReal.columns.str.strip()

# Tirar duplicatas gerais de linha inteira para o df ficar mais leve e nao dar erro de memoria no merge
df_PREfinal = df_PREfinal.drop_duplicates()
df_dataMaxReal = df_dataMaxReal.drop_duplicates()

# Realizar o merge com múltiplas condições
df_final = pd.merge(df_PREfinal, df_dataMaxReal, on=['Cód. Paciente', 'Última Admissão'], how='inner')

# Deixar o df mais leve para os proximos passos -> tirando duplicatas de linha inteira
df_final = df_final.drop_duplicates()

In [103]:
df_final.head()

,Cód. Paciente,Celular,E-mail,Primeira Admissão,Frequencia,Unidade,Grupo Hospitalar,Setor,Especialidade,Paciente,...,Gênero,Cód. Admissão,Produto,Subcategoria,Caráter Admissão,Tipo Produto,Solicitante,Médico,Jornada,Última Admissão
0,9950184,61982400323,Não deseja informar e-mail,2024-11-04,Recorrente,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA,VERA LUCIA DE AREA LEAO NAVARRO,...,FEMININO,37952725,EXAME,Exame Técnico,ELETIVO,OFTALMO TIPO II,MARIA REGINA CHALITA,MEDICO CBV,Avaliação Exames,2025-09-08
1,9950184,61982400323,Não deseja informar e-mail,2024-11-04,Recorrente,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA,VERA LUCIA DE AREA LEAO NAVARRO,...,FEMININO,37952725,EXAME,Exame Médico,ELETIVO,OFTALMO TIPO I,MARIA REGINA CHALITA,JUSCELINO KUBITSCHEK,Avaliação Exames,2025-09-08
2,9950184,61982400323,Não deseja informar e-mail,2024-11-04,Recorrente,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA,VERA LUCIA DE AREA LEAO NAVARRO,...,FEMININO,37952725,EXAME,Pacote Exames,ELETIVO,PACOTES (OFTALMO),NaN,MEDICO CBV,Avaliação Exames,2025-09-08
3,9950184,61982400323,Não deseja informar e-mail,2024-11-04,Recorrente,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA,NaN,...,NaN,37952725,EXAME,Exame Técnico,ELETIVO,OFTALMO TIPO II,MARIA REGINA CHALITA,MEDICO CBV,Avaliação Exames,2025-09-08
4,9950184,61982400323,Não deseja informar e-mail,2024-11-04,Recorrente,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA,NaN,...,NaN,37952725,EXAME,Exame Médico,ELETIVO,OFTALMO TIPO I,MARIA REGINA CHALITA,JUSCELINO KUBITSCHEK,Avaliação Exames,2025-09-08


In [104]:
print("Tamanho Produções (Concat) + AcompFreq Após Merge: ", df_PREfinal.shape[0])

print("\nTamanho Produções (Concat) + AcompFreq Após Merge e Após Merge com df concatenado que tem a ultima data max verdadeira: ", df_final.shape[0])

Tamanho Produções (Concat) + AcompFreq Após Merge:  916363

Tamanho Produções (Concat) + AcompFreq Após Merge e Após Merge com df concatenado que tem a ultima data max verdadeira:  577368


In [105]:
print('Volumetria Diária QVIS= {}'.format(df_final['Cód. Paciente'].nunique()))

Volumetria Diária QVIS= 78268


# FILTRAR DATA / DIA DA BASE -> D-1

## #################################################### #

In [106]:
from datetime import datetime, timedelta
import pandas as pd

# Convertendo a coluna 'Última Admissão' para datetime
df_final['Última Admissão'] = pd.to_datetime(df_final['Última Admissão'], dayfirst=True)

# Obtendo o dia da semana atual (0 = segunda-feira, 1 = terça-feira, ..., 6 = domingo)
dia_atual = datetime.today().weekday()

# Definindo a quantidade de dias para subtrair, de acordo com o dia atual
if dia_atual == 0:  # Se for segunda-feira
    dias_para_subtrair_inicio = 7 # Filtra de 7 dias atrás
else:
    dias_para_subtrair_inicio = 1  # Para os outros dias, filtra apenas o dia anterior

dias_para_subtrair_fim = 1  # **Sempre deve ser 1 para garantir que não inclua o dia atual**

# Calculando as datas de início e fim do intervalo
data_inicio = (datetime.today() - timedelta(days=dias_para_subtrair_inicio)).date()
data_fim = (datetime.today() - timedelta(days=dias_para_subtrair_fim)).date()

# Verificando as datas de filtro
print(f"\nFiltro de intervalo de {data_inicio} até {data_fim}")

# Filtrando o DataFrame para manter os registros dentro do intervalo de datas
df_final = df_final[(df_final['Última Admissão'].dt.date >= data_inicio) & 
                      (df_final['Última Admissão'].dt.date <= data_fim)]

# Verificando o tamanho após o filtro
print("Tamanho df_final após filtrar D-1 -> D-3: ", df_final.shape[0])

# Verificando os dados mínimos e máximos para conferir
print("\nData MIN: ", df_final['Última Admissão'].min())
print("Data MAX: ", df_final['Última Admissão'].max())


Filtro de intervalo de 2025-09-08 até 2025-09-08
Tamanho df_final após filtrar D-1 -> D-3:  43051

Data MIN:  2025-09-08 00:00:00
Data MAX:  2025-09-08 00:00:00


In [107]:
print("Tamanho df_final apos filtrar d-1: ", df_final.shape[0])

print("\nData MIN: ", df_final['Última Admissão'].min())
print("Data MAX: ", df_final['Última Admissão'].max())

Tamanho df_final apos filtrar d-1:  43051

Data MIN:  2025-09-08 00:00:00
Data MAX:  2025-09-08 00:00:00


## #################################################### #

# SEPARAR COLUNA SOLICITANTE POR ;

# E

# SEPARAR COLUNA MEDICO POR ;

In [108]:
# SOLICITANTE

df_final['Solicitante'] = df_final['Solicitante'].str.split(' ; ')

# Explodir as listas em linhas separadas
df_final = df_final.explode('Solicitante').reset_index(drop=True)

##########################################################################################

# MÉDICO

df_final['Médico'] = df_final['Médico'].str.split(' ; ')

# Explodir as listas em linhas separadas
df_final = df_final.explode('Médico').reset_index(drop=True)

In [109]:
df_final.head()

,Cód. Paciente,Celular,E-mail,Primeira Admissão,Frequencia,Unidade,Grupo Hospitalar,Setor,Especialidade,Paciente,...,Gênero,Cód. Admissão,Produto,Subcategoria,Caráter Admissão,Tipo Produto,Solicitante,Médico,Jornada,Última Admissão
0,9950184,61982400323,Não deseja informar e-mail,2024-11-04,Recorrente,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA,VERA LUCIA DE AREA LEAO NAVARRO,...,FEMININO,37952725,EXAME,Exame Técnico,ELETIVO,OFTALMO TIPO II,MARIA REGINA CHALITA,MEDICO CBV,Avaliação Exames,2025-09-08
1,9950184,61982400323,Não deseja informar e-mail,2024-11-04,Recorrente,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA,VERA LUCIA DE AREA LEAO NAVARRO,...,FEMININO,37952725,EXAME,Exame Médico,ELETIVO,OFTALMO TIPO I,MARIA REGINA CHALITA,JUSCELINO KUBITSCHEK,Avaliação Exames,2025-09-08
2,9950184,61982400323,Não deseja informar e-mail,2024-11-04,Recorrente,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA,VERA LUCIA DE AREA LEAO NAVARRO,...,FEMININO,37952725,EXAME,Pacote Exames,ELETIVO,PACOTES (OFTALMO),NaN,MEDICO CBV,Avaliação Exames,2025-09-08
3,9950184,61982400323,Não deseja informar e-mail,2024-11-04,Recorrente,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA,NaN,...,NaN,37952725,EXAME,Exame Técnico,ELETIVO,OFTALMO TIPO II,MARIA REGINA CHALITA,MEDICO CBV,Avaliação Exames,2025-09-08
4,9950184,61982400323,Não deseja informar e-mail,2024-11-04,Recorrente,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA,NaN,...,NaN,37952725,EXAME,Exame Médico,ELETIVO,OFTALMO TIPO I,MARIA REGINA CHALITA,JUSCELINO KUBITSCHEK,Avaliação Exames,2025-09-08


# TRATAR NOMES DE SOLICITANTE

# E

# TRATAR NOMES DE MÉDICO

In [110]:
import re
from unidecode import unidecode

# SOLICITANTE

df_final['Solicitante'] = df_final['Solicitante'].astype(str)

def extrair_nome(solicitante):
    # Regex para encontrar o primeiro caso (antes do traço)
    match_traco = re.search(r'^(.*?)\s*-\s*', solicitante)
    if match_traco:
        nome = match_traco.group(1)
    else:
        # Regex para encontrar o segundo caso (antes de dois espaços consecutivos)
        match_dois_espacos = re.search(r'^(.*?)\s\s', solicitante)
        if match_dois_espacos:
            nome = match_dois_espacos.group(1)
        else:
            # Se nenhum dos casos for encontrado, retorna o valor original
            nome = solicitante
    
    # Remove acentuações e converte para maiúsculas
    nome = unidecode(nome).upper()
    return nome

# Aplica a função diretamente na coluna 'Solicitante'
df_final['Solicitante'] = df_final['Solicitante'].apply(extrair_nome)

########################################################################################################################

# MÉDICO

df_final['Médico'] = df_final['Médico'].astype(str)

def extrair_nome_med(medico):
    # Regex para encontrar o primeiro caso (antes do traço)
    match_traco = re.search(r'^(.*?)\s*-\s*', medico)
    if match_traco:
        nome = match_traco.group(1)
    else:
        # Regex para encontrar o segundo caso (antes de dois espaços consecutivos)
        match_dois_espacos = re.search(r'^(.*?)\s\s', medico)
        if match_dois_espacos:
            nome = match_dois_espacos.group(1)
        else:
            # Se nenhum dos casos for encontrado, retorna o valor original
            nome = medico
    
    # Remove acentuações e converte para maiúsculas
    nome = unidecode(nome).upper()
    return nome

# Aplica a função diretamente na coluna 'Médico'
df_final['Médico'] = df_final['Médico'].apply(extrair_nome_med)

In [111]:
df_final.head()

,Cód. Paciente,Celular,E-mail,Primeira Admissão,Frequencia,Unidade,Grupo Hospitalar,Setor,Especialidade,Paciente,...,Gênero,Cód. Admissão,Produto,Subcategoria,Caráter Admissão,Tipo Produto,Solicitante,Médico,Jornada,Última Admissão
0,9950184,61982400323,Não deseja informar e-mail,2024-11-04,Recorrente,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA,VERA LUCIA DE AREA LEAO NAVARRO,...,FEMININO,37952725,EXAME,Exame Técnico,ELETIVO,OFTALMO TIPO II,MARIA REGINA CHALITA,MEDICO CBV,Avaliação Exames,2025-09-08
1,9950184,61982400323,Não deseja informar e-mail,2024-11-04,Recorrente,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA,VERA LUCIA DE AREA LEAO NAVARRO,...,FEMININO,37952725,EXAME,Exame Médico,ELETIVO,OFTALMO TIPO I,MARIA REGINA CHALITA,JUSCELINO KUBITSCHEK,Avaliação Exames,2025-09-08
2,9950184,61982400323,Não deseja informar e-mail,2024-11-04,Recorrente,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA,VERA LUCIA DE AREA LEAO NAVARRO,...,FEMININO,37952725,EXAME,Pacote Exames,ELETIVO,PACOTES (OFTALMO),NAN,MEDICO CBV,Avaliação Exames,2025-09-08
3,9950184,61982400323,Não deseja informar e-mail,2024-11-04,Recorrente,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA,NaN,...,NaN,37952725,EXAME,Exame Técnico,ELETIVO,OFTALMO TIPO II,MARIA REGINA CHALITA,MEDICO CBV,Avaliação Exames,2025-09-08
4,9950184,61982400323,Não deseja informar e-mail,2024-11-04,Recorrente,CBV MATRIZ,CBV,CBV MATRIZ CENTRO DIAGNOSTICO,OFTA,NaN,...,NaN,37952725,EXAME,Exame Médico,ELETIVO,OFTALMO TIPO I,MARIA REGINA CHALITA,JUSCELINO KUBITSCHEK,Avaliação Exames,2025-09-08


# RENOMEAR COLUNAS DE ACORDO COM A PLANILHA PEDIDA

In [112]:
df_final2 = df_final[['Cód. Admissão', 'Última Admissão', 'Frequencia', 'Cód. Paciente', 'Paciente', 'Celular', 'E-mail',
                      'Idade', 'Gênero',
                   'Unidade', 'Setor', 'Especialidade',
                   'Grupo Hospitalar', 'Jornada', 'Subcategoria', 'Solicitante', 'Médico', 'Tipo Produto', 
                      'Caráter Admissão', 'Produto']].rename(
                                                   columns={'Cód. Admissão':'id_transacao',
                                                  'Última Admissão':'data_transacao',
                                                            'Frequencia': 'frequencia',
                                                  'Cód. Paciente':'id_cliente',
                                                  'Paciente':'nome_do_cliente',
                                                  'Celular':'telefone',
                                                  'E-mail':'email',
                                                            'Idade': 'idade',
                                                        'Gênero': 'sexo',
                                                  'Unidade':'nome_loja',
                                                  'Setor': 'setores',
                                                  'Especialidade': 'especialidade',
                                                  'Grupo Hospitalar': 'grupo_hospitalar',
                                                  'Jornada': 'jornada',
                                                  'Subcategoria': 'subcategoria', 
                                                  'Solicitante': 'solicitante',
                                                            'Médico': 'medico',
                                                  'Tipo Produto': 'tipo_produto',
                                                   'Caráter Admissão': 'carater_admissao',
                                                    'Produto': 'produto'
                                                 })

df_final2['id_loja'] = df_final2['nome_loja']

df_final2 = df_final2[['id_transacao', 'data_transacao', 'frequencia','id_cliente', 'nome_do_cliente', 'telefone', 'email',
                       'idade', 'sexo',
                   'nome_loja', 'setores', 'especialidade', 'id_loja',
                   'grupo_hospitalar', 'carater_admissao', 'jornada', 'subcategoria', 'tipo_produto', 'solicitante', 'medico', 
                       'produto']]

In [113]:
df_final2.head()

,id_transacao,data_transacao,frequencia,id_cliente,nome_do_cliente,telefone,email,idade,sexo,nome_loja,...,especialidade,id_loja,grupo_hospitalar,carater_admissao,jornada,subcategoria,tipo_produto,solicitante,medico,produto
0,37952725,2025-09-08,Recorrente,9950184,VERA LUCIA DE AREA LEAO NAVARRO,61982400323,Não deseja informar e-mail,70.0,FEMININO,CBV MATRIZ,...,OFTA,CBV MATRIZ,CBV,ELETIVO,Avaliação Exames,Exame Técnico,OFTALMO TIPO II,MARIA REGINA CHALITA,MEDICO CBV,EXAME
1,37952725,2025-09-08,Recorrente,9950184,VERA LUCIA DE AREA LEAO NAVARRO,61982400323,Não deseja informar e-mail,70.0,FEMININO,CBV MATRIZ,...,OFTA,CBV MATRIZ,CBV,ELETIVO,Avaliação Exames,Exame Médico,OFTALMO TIPO I,MARIA REGINA CHALITA,JUSCELINO KUBITSCHEK,EXAME
2,37952725,2025-09-08,Recorrente,9950184,VERA LUCIA DE AREA LEAO NAVARRO,61982400323,Não deseja informar e-mail,70.0,FEMININO,CBV MATRIZ,...,OFTA,CBV MATRIZ,CBV,ELETIVO,Avaliação Exames,Pacote Exames,PACOTES (OFTALMO),NAN,MEDICO CBV,EXAME
3,37952725,2025-09-08,Recorrente,9950184,NaN,61982400323,Não deseja informar e-mail,NaN,NaN,CBV MATRIZ,...,OFTA,CBV MATRIZ,CBV,ELETIVO,Avaliação Exames,Exame Técnico,OFTALMO TIPO II,MARIA REGINA CHALITA,MEDICO CBV,EXAME
4,37952725,2025-09-08,Recorrente,9950184,NaN,61982400323,Não deseja informar e-mail,NaN,NaN,CBV MATRIZ,...,OFTA,CBV MATRIZ,CBV,ELETIVO,Avaliação Exames,Exame Médico,OFTALMO TIPO I,MARIA REGINA CHALITA,JUSCELINO KUBITSCHEK,EXAME


# TRATAR TELEFONE DO DF FINAL

In [114]:
df_final2_cel = df_final2.copy()

df_final2_cel['telefone'] = df_final2_cel['telefone'].str.replace(r'\D', '', regex=True)
df_final2_cel['onzeDigitos'] = df_final2_cel['telefone'].apply(lambda x: len(str(x)) == 11)
df_final2_cel = df_final2_cel[df_final2_cel['onzeDigitos'] == True].reset_index(drop=True)

# df_final2_cel = df_final2_cel.drop_duplicates(subset=['id_cliente'])
df_final2_cel = df_final2_cel.drop_duplicates().drop(columns=['onzeDigitos'])

# Função para formatar os números de telefone
def formatar_telefone(numero):
    ddd = numero[:2]
    celular = numero[2:]
    return f'({ddd}) {celular}'

# Aplicando a função à coluna de telefone
df_final2_cel['telefone'] = df_final2_cel['telefone'].apply(formatar_telefone)

# Função para remover a formatação dos números de telefone
def formatar_telefone(numero):
    return numero.replace("(", "").replace(")", "").replace(" ", "")

# Aplicando a função à coluna de telefone
df_final2_cel['telefone'] = df_final2_cel['telefone'].apply(formatar_telefone)

In [115]:
df_final2_cel.head()

,id_transacao,data_transacao,frequencia,id_cliente,nome_do_cliente,telefone,email,idade,sexo,nome_loja,...,especialidade,id_loja,grupo_hospitalar,carater_admissao,jornada,subcategoria,tipo_produto,solicitante,medico,produto
0,37952725,2025-09-08,Recorrente,9950184,VERA LUCIA DE AREA LEAO NAVARRO,61982400323,Não deseja informar e-mail,70.0,FEMININO,CBV MATRIZ,...,OFTA,CBV MATRIZ,CBV,ELETIVO,Avaliação Exames,Exame Técnico,OFTALMO TIPO II,MARIA REGINA CHALITA,MEDICO CBV,EXAME
1,37952725,2025-09-08,Recorrente,9950184,VERA LUCIA DE AREA LEAO NAVARRO,61982400323,Não deseja informar e-mail,70.0,FEMININO,CBV MATRIZ,...,OFTA,CBV MATRIZ,CBV,ELETIVO,Avaliação Exames,Exame Médico,OFTALMO TIPO I,MARIA REGINA CHALITA,JUSCELINO KUBITSCHEK,EXAME
2,37952725,2025-09-08,Recorrente,9950184,VERA LUCIA DE AREA LEAO NAVARRO,61982400323,Não deseja informar e-mail,70.0,FEMININO,CBV MATRIZ,...,OFTA,CBV MATRIZ,CBV,ELETIVO,Avaliação Exames,Pacote Exames,PACOTES (OFTALMO),NAN,MEDICO CBV,EXAME
3,37952725,2025-09-08,Recorrente,9950184,NaN,61982400323,Não deseja informar e-mail,NaN,NaN,CBV MATRIZ,...,OFTA,CBV MATRIZ,CBV,ELETIVO,Avaliação Exames,Exame Técnico,OFTALMO TIPO II,MARIA REGINA CHALITA,MEDICO CBV,EXAME
4,37952725,2025-09-08,Recorrente,9950184,NaN,61982400323,Não deseja informar e-mail,NaN,NaN,CBV MATRIZ,...,OFTA,CBV MATRIZ,CBV,ELETIVO,Avaliação Exames,Exame Médico,OFTALMO TIPO I,MARIA REGINA CHALITA,JUSCELINO KUBITSCHEK,EXAME


# TRATAR EMAIL DO DF FINAL

In [116]:
df_final2_email = df_final2.copy()

df_final2_email['Contem@'] = df_final2_email['email'].str.contains(r'@', regex=True, na=False)
df_final2_email['Contem.'] = df_final2_email['email'].str.contains(r'\.', regex=True, na=False)

df_final2_email = df_final2_email[(df_final2_email['Contem@']==True) & 
                                  (df_final2_email['Contem.']==True)]

df_final2_email = df_final2_email.drop_duplicates().drop(columns=['Contem@', 'Contem.'])

In [117]:
df_final2_email.head()

,id_transacao,data_transacao,frequencia,id_cliente,nome_do_cliente,telefone,email,idade,sexo,nome_loja,...,especialidade,id_loja,grupo_hospitalar,carater_admissao,jornada,subcategoria,tipo_produto,solicitante,medico,produto
18,38003609,2025-09-08,Recorrente,10000169,MARIA JOSE CONSTANTINO DA SILVA,13996730958,corrida75km@icloud.com,59.0,FEMININO,HPAUL,...,OFTA,HPAUL,V1 SP,ELETIVO,Avaliação Exames,Exame Médico,OFTALMO TIPO I,RAMON OLIVEIRA,RAMON OLIVEIRA,EXAME
19,38003609,2025-09-08,Recorrente,10000169,NaN,13996730958,corrida75km@icloud.com,NaN,NaN,HPAUL,...,OFTA,HPAUL,V1 SP,ELETIVO,Avaliação Exames,Exame Médico,OFTALMO TIPO I,RAMON OLIVEIRA,RAMON OLIVEIRA,EXAME
20,37990446,2025-09-08,Recorrente,10000874,ANTONIO NIVALDO OLIVEIRA LIMA,61984443479,naotem@gmail.com,64.0,MASCULINO,CBV MATRIZ,...,OFTA,CBV MATRIZ,CBV,ELETIVO,Avaliação Cirúrgica,Retina,CIRURGICO (OFTALMO),TIAGO SUZUKI,NAN,CIRURGIA
21,37990446,2025-09-08,Recorrente,10000874,NaN,61984443479,naotem@gmail.com,NaN,NaN,CBV MATRIZ,...,OFTA,CBV MATRIZ,CBV,ELETIVO,Avaliação Cirúrgica,Retina,CIRURGICO (OFTALMO),TIAGO SUZUKI,NAN,CIRURGIA
22,38010433,2025-09-08,Recorrente,10001966,SOPHIE BARBOSA ALVES MOREIRA,11958035401,gabb.bga@gmail.com,0.0,FEMININO,HORE,...,OFTA,HORE,HOPE,ELETIVO,Avaliação Ambulatorial,Eletivas,CONSULTA,DAENA LEAL,DAENA LEAL,CONSULTA


# CONCATENAR DF FINAL TELEFONE com DF FINAL EMAIL - TRATADOS

In [118]:
df_base = pd.concat([df_final2_cel, df_final2_email])

df_base = df_base.drop_duplicates(subset=['id_transacao', 'data_transacao', 'frequencia','id_cliente', 'nome_do_cliente', 'telefone', 'email',
                       'idade', 'sexo',
                   'nome_loja', 'setores', 'especialidade', 'id_loja',
                   'grupo_hospitalar', 'carater_admissao', 'jornada', 'subcategoria', 'tipo_produto', 'solicitante', 'medico', 
                       'produto'])

df_base['id_cliente'] = df_base['id_cliente'].astype(str)

# FINALIZAR PEGANDO PRIMEIRO NOME DO PACIENTE

In [119]:
df_base['nome_do_cliente'] = df_base['nome_do_cliente'].str.split(' ').str[0]

In [120]:
df_base.head()

,id_transacao,data_transacao,frequencia,id_cliente,nome_do_cliente,telefone,email,idade,sexo,nome_loja,...,especialidade,id_loja,grupo_hospitalar,carater_admissao,jornada,subcategoria,tipo_produto,solicitante,medico,produto
0,37952725,2025-09-08,Recorrente,9950184,VERA,61982400323,Não deseja informar e-mail,70.0,FEMININO,CBV MATRIZ,...,OFTA,CBV MATRIZ,CBV,ELETIVO,Avaliação Exames,Exame Técnico,OFTALMO TIPO II,MARIA REGINA CHALITA,MEDICO CBV,EXAME
1,37952725,2025-09-08,Recorrente,9950184,VERA,61982400323,Não deseja informar e-mail,70.0,FEMININO,CBV MATRIZ,...,OFTA,CBV MATRIZ,CBV,ELETIVO,Avaliação Exames,Exame Médico,OFTALMO TIPO I,MARIA REGINA CHALITA,JUSCELINO KUBITSCHEK,EXAME
2,37952725,2025-09-08,Recorrente,9950184,VERA,61982400323,Não deseja informar e-mail,70.0,FEMININO,CBV MATRIZ,...,OFTA,CBV MATRIZ,CBV,ELETIVO,Avaliação Exames,Pacote Exames,PACOTES (OFTALMO),NAN,MEDICO CBV,EXAME
3,37952725,2025-09-08,Recorrente,9950184,NaN,61982400323,Não deseja informar e-mail,NaN,NaN,CBV MATRIZ,...,OFTA,CBV MATRIZ,CBV,ELETIVO,Avaliação Exames,Exame Técnico,OFTALMO TIPO II,MARIA REGINA CHALITA,MEDICO CBV,EXAME
4,37952725,2025-09-08,Recorrente,9950184,NaN,61982400323,Não deseja informar e-mail,NaN,NaN,CBV MATRIZ,...,OFTA,CBV MATRIZ,CBV,ELETIVO,Avaliação Exames,Exame Médico,OFTALMO TIPO I,MARIA REGINA CHALITA,JUSCELINO KUBITSCHEK,EXAME


In [121]:
# Obtendo a data atual formatada
data_atual = datetime.today().strftime('%d/%m/%y')

# Exibindo o título com a data dinâmica
print(f"# VERIFICAÇÕES DE VOLUMETRIA (MÊS ATUAL - REF {data_atual})")

# VERIFICAÇÕES DE VOLUMETRIA (MÊS ATUAL - REF 09/09/25)


In [122]:
print("Base Total: ",df_base.shape[0])
print("Total Cód. Paciente: ",df_base['id_cliente'].nunique())
print("Total Celular:",df_base['telefone'].nunique())
print("Total E-mail:",df_base['email'].nunique())

Base Total:  42499
Total Cód. Paciente:  5373
Total Celular: 5215
Total E-mail: 3662


In [123]:
df_base['data_transacao'].min()

Timestamp('2025-09-08 00:00:00')

In [124]:
df_base['data_transacao'].max()

Timestamp('2025-09-08 00:00:00')

# CLASSIFICAR JORNADAS COM NOME PADRAO SOLUCX

In [125]:
print("Jornadas: \n", df_base['jornada'].value_counts())

print("\nGrupo Hospitalar: \n", df_base['grupo_hospitalar'].value_counts())

Jornadas: 
 jornada
Avaliação Ambulatorial        16354
Avaliação Exames              16122
Avaliação Cirúrgica            5046
Avaliação Urgência             3698
Avaliação Injeção               765
Avaliação Lente de Contato      514
Name: count, dtype: int64

Grupo Hospitalar: 
 grupo_hospitalar
HOPE        14083
V1 SP       12823
HOSL         4952
CBV          4547
CERPO        2255
HOC          1040
OFTALMOS      965
HORP          894
LASER         382
HRO           296
HOA           262
Name: count, dtype: int64


In [126]:
def mapear_jornada(row):
    
    # HOPE
    if row['grupo_hospitalar'] == 'HOPE':
        if row['jornada'] == 'Avaliação Ambulatorial':
            return 'hope_consultas'
        elif row['jornada'] == 'Avaliação Exames':
            return 'hope_exames'
        elif row['jornada'] == 'Avaliação Urgência':
            return 'hope_urgencia'
        elif row['jornada'] == 'Avaliação Cirúrgica':
            return 'hope_centrocirurgico'
        elif row['jornada'] == 'Avaliação Injeção':
            return 'hope_centrocirurgico'
        elif row['jornada'] == 'Avaliação Lente de Contato':
            return 'hope_lentes_de_contato'
    
    # V1 SP
    elif row['grupo_hospitalar'] == 'V1 SP':
        if row['jornada'] == 'Avaliação Ambulatorial':
            return 'holhos_consultas'
        elif row['jornada'] == 'Avaliação Exames':
            return 'holhos_exames'
        elif row['jornada'] == 'Avaliação Urgência':
            return 'holhos_urgencia'
        elif row['jornada'] == 'Avaliação Cirúrgica':
            return 'holhos_centrocirurgico'
        elif row['jornada'] == 'Avaliação Injeção':
            return 'holhos_centrocirurgico'
        elif row['jornada'] == 'Avaliação Lente de Contato':
            return 'holhos_lentes_de_contato'
        
    # HOSL
    elif row['grupo_hospitalar'] == 'HOSL':
        if row['jornada'] == 'Avaliação Ambulatorial':
            return 'santa_luzia_consultas'
        elif row['jornada'] == 'Avaliação Exames':
            return 'santa_luzia_exames'
        elif row['jornada'] == 'Avaliação Urgência':
            return 'santa_luzia_urgencia'
        elif row['jornada'] == 'Avaliação Cirúrgica':
            return 'santa_luzia_centrocirurgico'
        elif row['jornada'] == 'Avaliação Injeção':
            return 'santa_luzia_centrocirurgico'
        elif row['jornada'] == 'Avaliação Lente de Contato':
            return 'santa_luzia_lentes_de_contato'
    
    # CERPO
    elif row['grupo_hospitalar'] == 'CERPO':
        if row['jornada'] == 'Avaliação Ambulatorial':
            return 'cerpo_consultas'
        elif row['jornada'] == 'Avaliação Exames':
            return 'cerpo_exames'
        elif row['jornada'] == 'Avaliação Urgência':
            return 'cerpo_urgencia'
        elif row['jornada'] == 'Avaliação Cirúrgica':
            return 'cerpo_centrocirurgico'
        elif row['jornada'] == 'Avaliação Injeção':
            return 'cerpo_centrocirurgico'
        elif row['jornada'] == 'Avaliação Lente de Contato':
            return 'cerpo_lentes_de_contato'
        
    # HOC
    elif row['grupo_hospitalar'] == 'HOC':
        if row['jornada'] == 'Avaliação Ambulatorial':
            return 'hoc_consultas'
        elif row['jornada'] == 'Avaliação Exames':
            return 'hoc_exames'
        elif row['jornada'] == 'Avaliação Urgência':
            return 'hoc_urgencia'
        elif row['jornada'] == 'Avaliação Cirúrgica':
            return 'hoc_centrocirurgico'
        elif row['jornada'] == 'Avaliação Injeção':
            return 'hoc_centrocirurgico'
        elif row['jornada'] == 'Avaliação Lente de Contato':
            return 'hoc_lentes_de_contato'
        
    # OFTALMOS
    elif row['grupo_hospitalar'] == 'OFTALMOS':
        if row['jornada'] == 'Avaliação Ambulatorial':
            return 'oftalmos_consultas'
        elif row['jornada'] == 'Avaliação Exames':
            return 'oftalmos_exames'
        elif row['jornada'] == 'Avaliação Urgência':
            return 'oftalmos_urgencia'
        elif row['jornada'] == 'Avaliação Cirúrgica':
            return 'oftalmos_centrocirurgico'
        elif row['jornada'] == 'Avaliação Injeção':
            return 'oftalmos_centrocirurgico'
        elif row['jornada'] == 'Avaliação Lente de Contato':
            return 'oftalmos_lentes_de_contato'
    
    # LASER OCULAR
    elif row['grupo_hospitalar'] == 'LASER':
        if row['jornada'] == 'Avaliação Ambulatorial':
            return 'laser_ocular_consultas'
        elif row['jornada'] == 'Avaliação Exames':
            return 'laser_ocular_exames'
        elif row['jornada'] == 'Avaliação Urgência':
            return 'laser_ocular_urgencia'
        elif row['jornada'] == 'Avaliação Cirúrgica':
            return 'laser_ocular_centrocirurgico'
        elif row['jornada'] == 'Avaliação Injeção':
            return 'laser_ocular_centrocirurgico'
        elif row['jornada'] == 'Avaliação Lente de Contato':
            return 'laser_ocular_lentes_de_contato'
    
    # HRO
    elif row['grupo_hospitalar'] == 'HRO':
        if row['jornada'] == 'Avaliação Ambulatorial':
            return 'hro_consultas'
        elif row['jornada'] == 'Avaliação Exames':
            return 'hro_exames'
        elif row['jornada'] == 'Avaliação Urgência':
            return 'hro_urgencia'
        elif row['jornada'] == 'Avaliação Cirúrgica':
            return 'hro_centrocirurgico'
        elif row['jornada'] == 'Avaliação Injeção':
            return 'hro_centrocirurgico'
        elif row['jornada'] == 'Avaliação Lente de Contato':
            return 'hro_lentes_de_contato'
        
    # CBV
    elif row['grupo_hospitalar'] == 'CBV':
        if row['jornada'] == 'Avaliação Ambulatorial':
            return 'cbv_consultas'
        elif row['jornada'] == 'Avaliação Exames':
            return 'cbv_exames'
        elif row['jornada'] == 'Avaliação Urgência':
            return 'cbv_urgencia'
        elif row['jornada'] == 'Avaliação Cirúrgica':
            return 'cbv_centrocirurgico'
        elif row['jornada'] == 'Avaliação Injeção':
            return 'cbv_centrocirurgico'
        elif row['jornada'] == 'Avaliação Lente de Contato':
            return 'cbv_lentes_de_contato'
        
    

    #HOA
    elif row['grupo_hospitalar'] == 'HOA':
        if row['jornada'] == 'Avaliação Ambulatorial':
            return 'hoa_consultas'
        elif row['jornada'] == 'Avaliação Exames':
            return 'hoa_exames'
        elif row['jornada'] == 'Avaliação Urgência':
            return 'hoa_urgencia'
        elif row['jornada'] == 'Avaliação Cirúrgica':
            return 'hoa_centro_cirurgico'
        elif row['jornada'] == 'Avaliação Injeção':
            return 'hoa_centro_cirurgico'
        elif row['jornada'] == 'Avaliação Lente de Contato':
            return 'hoa_lentes_de_contato'

    #HORP
    elif row['grupo_hospitalar'] == 'HORP':
        if row['jornada'] == 'Avaliação Ambulatorial':
            return 'horp_consultas'
        elif row['jornada'] == 'Avaliação Exames':
            return 'horp_exames'
        elif row['jornada'] == 'Avaliação Urgência':
            return 'horp_urgencia'
        elif row['jornada'] == 'Avaliação Cirúrgica':
            return 'horp_centro_cirurgico'
        elif row['jornada'] == 'Avaliação Injeção':
            return 'horp_centro_cirurgico'
        elif row['jornada'] == 'Avaliação Lente de Contato':
            return 'horp_lentes_de_contato'

    else:
        # Se nenhuma condição for satisfeita, retorna NaN ou algum valor padrão
        return None

# Aplica a função ao DataFrame para criar a nova coluna 'jornada2'
df_base['jornada2'] = df_base.apply(mapear_jornada, axis=1)

In [127]:
# VALIDANDO POR NULOS

null_values_in_SOLUCX2 = df_base[df_base['jornada2'].isnull()]

In [128]:
null_values_in_SOLUCX2.head()

,id_transacao,data_transacao,frequencia,id_cliente,nome_do_cliente,telefone,email,idade,sexo,nome_loja,...,id_loja,grupo_hospitalar,carater_admissao,jornada,subcategoria,tipo_produto,solicitante,medico,produto,jornada2


In [129]:
# VALIDANDO

print(df_base[['carater_admissao', 'jornada', 'produto', 'grupo_hospitalar', 'jornada2']].value_counts())

carater_admissao      jornada                     produto   grupo_hospitalar  jornada2              
ELETIVO               Avaliação Exames            EXAME     HOPE              hope_exames               6056
                      Avaliação Ambulatorial      CONSULTA  V1 SP             holhos_consultas          5362
                                                            HOPE              hope_consultas            4916
                      Avaliação Exames            EXAME     V1 SP             holhos_exames             3779
                                                            HOSL              santa_luzia_exames        2284
                                                                                                        ... 
URGÊNCIA /EMERGÊNCIA  Avaliação Urgência          CONSULTA  HOC               hoc_urgencia                 3
ELETIVO               Avaliação Cirúrgica         CIRURGIA  HOA               hoa_centro_cirurgico         2
                      Avali

# FILTRAR HOSPITAL (GRUPO HOSPITALAR) DE ACORDO COM A PESQUISA

# PESQUISA DE AGOSTO -> TODO QVIS

In [130]:
print("Grupo Hospitalar:\n",df_base['grupo_hospitalar'].value_counts())

Grupo Hospitalar:
 grupo_hospitalar
HOPE        14083
V1 SP       12823
HOSL         4952
CBV          4547
CERPO        2255
HOC          1040
OFTALMOS      965
HORP          894
LASER         382
HRO           296
HOA           262
Name: count, dtype: int64


In [131]:
# df_base = df_base[df_base['grupo_hospitalar'].isin(['HOPE', 'HOSL', 'HRO', 'HOC', 'OFTALMOS'])]

In [132]:
# df_base.head()

In [133]:
# print("Hospital: \n", df_base['grupo_hospitalar'].value_counts())

# SEPARAR JORNADAS EM 2 DF's (RESTANTE x CIRURGIA)

# JORNADAS CONSULTAS, EXAMES, URGENCIA E LENTES:
- SEPARAR CADA UM ANTES, DROPAR DUPLICATAS DO PACIENTE, CONCATENAR ELES

# JORNADAS CIRURGICA E INJECAO:
- SEPARAR CADA UM ANTES, DROPAR DUPLICATAS DO PACIENTE, CONCATENAR ELES

In [134]:
# DF 1 -> CONSULTAS, EXAMES, LENTES E URGENCIA

# CONSULTAS
consultas_df = ['Avaliação Ambulatorial']
df_consultas1 = df_base[df_base['jornada'].isin(consultas_df)]
df_consultas1 = df_consultas1.drop_duplicates(subset=['id_cliente'])

# EXAMES
exames_df = ['Avaliação Exames']
df_exames1 = df_base[df_base['jornada'].isin(exames_df)]
df_exames1 = df_exames1.drop_duplicates(subset=['id_cliente'])

# LENTES
lentes_df = ['Avaliação Lente de Contato']
df_lentes1 = df_base[df_base['jornada'].isin(lentes_df)]
df_lentes1 = df_lentes1.drop_duplicates(subset=['id_cliente'])

# URGENCIA
urgencia_df = ['Avaliação Urgência']
df_urgencia1 = df_base[df_base['jornada'].isin(urgencia_df)]
df_urgencia1 = df_urgencia1.drop_duplicates(subset=['id_cliente'])

# CONCATENAR
df1_jornadas = pd.concat([df_consultas1, df_exames1, df_lentes1, df_urgencia1])

###############################################################################################################################

# DF 2 -> CIRURGIA E INJECAO

# CIRURGIA
cirurgias_df = ['Avaliação Cirúrgica']
df_cirurgias1 = df_base[df_base['jornada'].isin(cirurgias_df)]
df_cirurgias1 = df_cirurgias1.drop_duplicates(subset=['id_cliente'])

# INJECAO
injecao_df = ['Avaliação Injeção']
df_injecao1 = df_base[df_base['jornada'].isin(injecao_df)]
df_injecao1 = df_injecao1.drop_duplicates(subset=['id_cliente'])

# CONCATENAR
df2_jornadas = pd.concat([df_cirurgias1, df_injecao1])

In [135]:
df1_jornadas.head()

,id_transacao,data_transacao,frequencia,id_cliente,nome_do_cliente,telefone,email,idade,sexo,nome_loja,...,id_loja,grupo_hospitalar,carater_admissao,jornada,subcategoria,tipo_produto,solicitante,medico,produto,jornada2
8,37755348,2025-09-08,Recorrente,8034250,CLEONICE,81987829575,Não deseja informar e-mail,61.0,FEMININO,HOSL,...,HOSL,HOSL,ELETIVO,Avaliação Ambulatorial,Eletivas,CONSULTA,MARIA EUGENIA MARTINS,MARIA EUGENIA MARTINS,CONSULTA,santa_luzia_consultas
22,38010433,2025-09-08,Recorrente,10001966,SOPHIE,11958035401,gabb.bga@gmail.com,0.0,FEMININO,HORE,...,HORE,HOPE,ELETIVO,Avaliação Ambulatorial,Eletivas,CONSULTA,DAENA LEAL,DAENA LEAL,CONSULTA,hope_consultas
35,37968366,2025-09-08,Recorrente,10003149,IRENE,11982839265,mariainaildeferreiradasilva@gmail.com,82.0,FEMININO,HPAUL,...,HPAUL,V1 SP,ELETIVO,Avaliação Ambulatorial,Eletivas,CONSULTA,FELIPE LEAL DE MORAES BRITO,FELIPE LEAL DE MORAES BRITO,CONSULTA,holhos_consultas
46,37820076,2025-09-08,Recorrente,10003695,KAZUMI,11972078020,kazumiyano2@gmail.com,50.0,FEMININO,HPAUL,...,HPAUL,V1 SP,ELETIVO,Avaliação Ambulatorial,Eletivas,CONSULTA,ANALIA SUEHIRO,ANALIA SUEHIRO,CONSULTA,holhos_consultas
52,37879415,2025-09-08,Recorrente,10004430,SAMUEL,11976241079,vasgoncelosemilia194@gmail.com,1.0,MASCULINO,SANDRE,...,SANDRE,CERPO,ELETIVO,Avaliação Ambulatorial,Eletivas,CONSULTA,GRACIELLA SAKAMITI,GRACIELLA SAKAMITI,CONSULTA,cerpo_consultas


In [136]:
df2_jornadas.head()

,id_transacao,data_transacao,frequencia,id_cliente,nome_do_cliente,telefone,email,idade,sexo,nome_loja,...,id_loja,grupo_hospitalar,carater_admissao,jornada,subcategoria,tipo_produto,solicitante,medico,produto,jornada2
10,37994646,2025-09-08,Recorrente,8034250,CLEONICE,81987829575,Não deseja informar e-mail,61.0,FEMININO,HOSL,...,HOSL,HOSL,ELETIVO,Avaliação Cirúrgica,Catarata,CIRURGICO (OFTALMO),MARIA EUGENIA MARTINS,NAN,CIRURGIA,santa_luzia_centrocirurgico
20,37990446,2025-09-08,Recorrente,10000874,ANTONIO,61984443479,naotem@gmail.com,64.0,MASCULINO,CBV MATRIZ,...,CBV MATRIZ,CBV,ELETIVO,Avaliação Cirúrgica,Retina,CIRURGICO (OFTALMO),TIAGO SUZUKI,NAN,CIRURGIA,cbv_centrocirurgico
123,37779879,2025-09-08,Recorrente,10009850,SEVERINO,81971042460,Não deseja informar e-mail,88.0,MASCULINO,FSL,...,FSL,HOSL,ELETIVO,Avaliação Cirúrgica,Catarata,CIRURGICO (OFTALMO),LORENA GUERRA,NAN,CIRURGIA,santa_luzia_centrocirurgico
134,37994495,2025-09-08,Recorrente,10011647,SALATIEL,81991959735,Não deseja informar e-mail,59.0,MASCULINO,HOSL,...,HOSL,HOSL,ELETIVO,Avaliação Cirúrgica,Catarata,CIRURGICO (OFTALMO),MARIA EUGENIA MARTINS,NAN,CIRURGIA,santa_luzia_centrocirurgico
143,37993119,2025-09-08,Recorrente,10011745,MARLUCE,81973234756,Não deseja informar e-mail,71.0,FEMININO,FSL,...,FSL,HOSL,ELETIVO,Avaliação Cirúrgica,Catarata,CIRURGICO (OFTALMO),MARIA EDUARDA PACHECO,NAN,CIRURGIA,santa_luzia_centrocirurgico


In [137]:
# VERIFICANDO

print("Jornadas 1: ", df1_jornadas['jornada'].value_counts())
print("\nJornadas 1: ", df1_jornadas['jornada2'].value_counts())

print("\nJornadas 2: ", df2_jornadas['jornada'].value_counts())
print("\nJornadas 2: ", df2_jornadas['jornada2'].value_counts())

Jornadas 1:  jornada
Avaliação Ambulatorial        4164
Avaliação Exames              3135
Avaliação Urgência             865
Avaliação Lente de Contato     144
Name: count, dtype: int64

Jornadas 1:  jornada2
holhos_consultas                  1253
hope_consultas                    1160
hope_exames                       1064
holhos_exames                      761
santa_luzia_consultas              445
santa_luzia_exames                 434
cerpo_consultas                    430
holhos_urgencia                    403
cbv_consultas                      367
cbv_exames                         326
hope_urgencia                      251
cerpo_exames                       194
horp_consultas                     158
horp_exames                        133
hoc_consultas                      123
oftalmos_consultas                 108
cbv_urgencia                        88
santa_luzia_urgencia                77
oftalmos_exames                     74
hoc_exames                          69
laser_ocul

# NEGATIVAÇÃO DAS BASES (JORNADA 1 - EXCETO CIRURGIA/INJECAO)

# 7 DIAS DE BLACKOUT -> VER NOTA NA PASTA

In [138]:
# LER PASTA COM BASES A SEREM NEGATIVADAS
# NOTAS DOS BLACKOUTS NA PASTA -> ACOMPANHAR E ATUALIZAR POR LÁ
# BASES JORNADA 1

# Specify the folder path
folder_path = r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\Negativar_Jornada1"

# Initialize an empty DataFrame to store the data
df_negJORNADA1 = pd.DataFrame()

# Loop through all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
        file_path = os.path.join(folder_path, file_name)
        
        # Read the Excel file into a DataFrame
        df_temp = pd.read_excel(file_path)
        
        # Concatenate the current DataFrame with the overall DataFrame
        df_negJORNADA1 = pd.concat([df_negJORNADA1, df_temp], ignore_index=True)
        
df_negJORNADA1 = df_negJORNADA1.drop_duplicates()
df_negJORNADA1['id_cliente'] = df_negJORNADA1['id_cliente'].astype(str)
df_negJORNADA1['id_cliente'] = df_negJORNADA1['id_cliente'].str.removesuffix('.0')

C:\Users\MarceloFridmanMarizd\AppData\Local\Temp\ipykernel_3752\1546742016.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_negJORNADA1 = pd.concat([df_negJORNADA1, df_temp], ignore_index=True)
C:\Users\MarceloFridmanMarizd\AppData\Local\Temp\ipykernel_3752\1546742016.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_negJORNADA1 = pd.concat([df_negJORNADA1, df_temp], ignore_index=True)
C:\Users\MarceloFridmanMarizd\AppData\Local\Temp\ipykernel_3752\1546742016.py:20: FutureWarning: T

In [139]:
df_negJORNADA1.head()

,id_cliente,nome_do_cliente,email,telefone,id_transacao,id_loja,data_transacao,jornada,frequencia,idade,...,nome_loja,setores,especialidade,grupo_hospitalar,carater_admissao,subcategoria,solicitante,medico,produto,tec_med
0,9950270,MARIA,Não deseja informar e-mail,98984821974,37885844,VMAIS,2025-08-30,hro_consultas,Recorrente,74.0,...,VMAIS,VMAIS TERREO ATENDIMENTO,OFTA,HRO,ELETIVO,Volta/Retorno,KENARD MARTINS,KENARD MARTINS,CONSULTA,médico
1,5125662,RENATA,renatamilan1725@gmail.com,67981271380,37751716,HOC,2025-08-30,hoc_consultas,Recorrente,50.0,...,HOC,HOC ATENDIMENTO,OFTA,HOC,ELETIVO,Volta/Retorno,ELMER SALVIANO,ELMER SALVIANO,CONSULTA,médico
2,8189496,ANTONIO,não@gmail.com,81991589763,37733399,FSL,2025-08-30,santa_luzia_consultas,Recorrente,70.0,...,FSL,FSL TERREO CONSULTORIOS,OFTA,HOSL,ELETIVO,Eletivas,MARILIA SIQUEIRA,MARILIA SIQUEIRA,CONSULTA,médico
3,10003042,NaN,elis.aparecidacarneiro@gmail.com,11965394757,37905442,SAMARO,2025-08-30,cerpo_consultas,Primeira Vez,NaN,...,SAMARO,SAMARO PRIMEIRO ANDAR ATENDIMENTO,OFTA,CERPO,ELETIVO,Eletivas,GABRIELA CARNEIRO TEIXEIRA,GABRIELA CARNEIRO TEIXEIRA,CONSULTA,médico
4,10005088,FABIANA,Não deseja informar e-mail,11983321592,37945526,SAMARO,2025-08-30,cerpo_consultas,Recorrente,45.0,...,SAMARO,SAMARO PRIMEIRO ANDAR ATENDIMENTO,OFTA,CERPO,ELETIVO,Eletivas,MYCHELE SZENBERG CARNEIRO CAMPELO,MYCHELE SZENBERG CARNEIRO CAMPELO,CONSULTA,médico


In [140]:
# NEGATIVAR -> BASE DIA ATUAL x DF DE CIMA
# LEMBRAR DE SEMPRE ATUALIZAR O DF DO DIA AQUI
# DF BASE DO DIA JORNADA 1 x DF JORNADAS 1 ANTERIORES

# Data de hoje
hoje = pd.to_datetime(datetime.today().date())

# 🔹 FILTRO DA NEGATIVAÇÃO (últimos 7 dias)
inicio_neg_7 = hoje - timedelta(days=7)  # 7 dias atrás
fim_neg_7 = hoje - timedelta(days=1)  # Até ontem

df_negJORNADA1_filtrado = df_negJORNADA1[
    (df_negJORNADA1['data_transacao'] >= inicio_neg_7) & 
    (df_negJORNADA1['data_transacao'] <= fim_neg_7)
]

# 🔹 NEGATIVAR BASE DO DIA ATUAL
df_final1 = df1_jornadas[~df1_jornadas['id_cliente'].isin(df_negJORNADA1_filtrado['id_cliente'])]
df_final1 = df1_jornadas.copy()

In [141]:
print("Datas que estao sendo negativadas (validar) MIN: ", df_negJORNADA1_filtrado['data_transacao'].min())
print("Datas que estao sendo negativadas (validar) MAX: ", df_negJORNADA1_filtrado['data_transacao'].max())

print("\nAntes do ISIN: ", df1_jornadas['id_cliente'].nunique())
print("Depois do ISIN: ", df_final1['id_cliente'].nunique())

Datas que estao sendo negativadas (validar) MIN:  2025-09-02 00:00:00
Datas que estao sendo negativadas (validar) MAX:  2025-09-07 00:00:00

Antes do ISIN:  5290
Depois do ISIN:  5290


# NEGATIVAÇÃO DAS BASES (JORNADA 2 - APENAS CIRURGIA/INJECAO)

# 14 DIAS DE BLACKOUT -> VER NOTA NA PASTA

In [142]:
# LER PASTA COM BASES A SEREM NEGATIVADAS
# NOTAS DOS BLACKOUTS NA PASTA -> ACOMPANHAR E ATUALIZAR POR LÁ
# BASES JORNADA 2

# Specify the folder path
folder_path = r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\Negativar_Jornada2"

# Initialize an empty DataFrame to store the data
df_negJORNADA2 = pd.DataFrame()

# Loop through all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
        file_path = os.path.join(folder_path, file_name)
        
        # Read the Excel file into a DataFrame
        df_temp = pd.read_excel(file_path)
        
        # Concatenate the current DataFrame with the overall DataFrame
        df_negJORNADA2 = pd.concat([df_negJORNADA2, df_temp], ignore_index=True)
        
df_negJORNADA2 = df_negJORNADA2.drop_duplicates()
df_negJORNADA2['id_cliente'] = df_negJORNADA2['id_cliente'].astype(str)
df_negJORNADA2['id_cliente'] = df_negJORNADA2['id_cliente'].str.removesuffix('.0')

C:\Users\MarceloFridmanMarizd\AppData\Local\Temp\ipykernel_3752\2138428072.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_negJORNADA2 = pd.concat([df_negJORNADA2, df_temp], ignore_index=True)
C:\Users\MarceloFridmanMarizd\AppData\Local\Temp\ipykernel_3752\2138428072.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_negJORNADA2 = pd.concat([df_negJORNADA2, df_temp], ignore_index=True)
C:\Users\MarceloFridmanMarizd\AppData\Local\Temp\ipykernel_3752\2138428072.py:20: FutureWarning: T

In [143]:
df_negJORNADA2.head()

,id_cliente,nome_do_cliente,email,telefone,id_transacao,id_loja,data_transacao,jornada,frequencia,idade,...,nome_loja,setores,especialidade,grupo_hospitalar,carater_admissao,subcategoria,solicitante,medico,produto,tec_med
0,10311280,NaN,Não deseja informar e-mail,98981388128,37639376CirRUY,HRO (GOLDEN),2025-08-30,hro_centrocirurgico,Recorrente,NaN,...,HRO (GOLDEN),HRO (GOLDEN) ATENDIMENTO,OFTA,HRO,ELETIVO,Glaucoma,NAN,NAN,CIRURGIA,médico
1,10347759,NaN,lucianamarchezin@gmail.com,11999069357,37696937CirRUY,HPAUL,2025-08-30,holhos_centrocirurgico,Recorrente,NaN,...,HPAUL,HPAUL HOSPITAL 1º / 2º / 3º ANDAR ATENDIMENTO ...,OFTA,V1 SP,ELETIVO,Catarata,NAN,NAN,CIRURGIA,médico
2,10361880,NaN,beatrizcristina2092@gmail.com,81973205816,37732869CirRUY,HORE,2025-08-30,hope_centrocirurgico,Recorrente,NaN,...,HORE,HORE QUINTO RECEPCAO,OFTA,HOPE,ELETIVO,Retina,VASCO BRAVO FILHO,NAN,CIRURGIA,médico
3,10363605,NaN,Não deseja informar e-mail,11950535408,37820898CirRUY,HPAUL,2025-08-30,holhos_centrocirurgico,Recorrente,NaN,...,HPAUL,HPAUL HOSPITAL 1º / 2º / 3º ANDAR ATENDIMENTO ...,OFTA,V1 SP,ELETIVO,Retina,NAN,NAN,CIRURGIA,médico
4,10364898,NaN,roseligosil@gmail.com,47997371965,37856264CirRUY,HPAUL,2025-08-30,holhos_centrocirurgico,Recorrente,NaN,...,HPAUL,HPAUL HOSPITAL 1º / 2º / 3º ANDAR ATENDIMENTO ...,OFTA,V1 SP,ELETIVO,Catarata,MOACYR CAMPOS,NAN,CIRURGIA,médico


In [144]:
# NEGATIVAR -> BASE DIA ATUAL x DF DE CIMA
# LEMBRAR DE SEMPRE ATUALIZAR O DF DO DIA AQUI
# DF BASE DO DIA JORNADA 2 x DF JORNADAS 2 ANTERIORES

hoje = pd.to_datetime(datetime.today().date())

# 🔹 FILTRO DA NEGATIVAÇÃO (últimos 14 dias)
inicio_neg_14 = hoje - timedelta(days=14)  # 14 dias atrás
fim_neg_14 = hoje - timedelta(days=1)  # Até ontem

df_negJORNADA2_filtrado = df_negJORNADA2[
    (df_negJORNADA2['data_transacao'] >= inicio_neg_14) & 
    (df_negJORNADA2['data_transacao'] <= fim_neg_14)
]

# 🔹 NEGATIVAR BASE DO DIA ATUAL
df_final2 = df2_jornadas[~df2_jornadas['id_cliente'].isin(df_negJORNADA2_filtrado['id_cliente'])]
df_final2 = df2_jornadas.copy()

In [145]:
print("Datas que estao sendo negativadas (validar) MIN: ", df_negJORNADA2_filtrado['data_transacao'].min())
print("Datas que estao sendo negativadas (validar) MAX: ", df_negJORNADA2_filtrado['data_transacao'].max())

print("\nAntes do ISIN: ", df2_jornadas['id_cliente'].nunique())
print("Depois do ISIN: ", df_final2['id_cliente'].nunique())

Datas que estao sendo negativadas (validar) MIN:  2025-08-26 00:00:00
Datas que estao sendo negativadas (validar) MAX:  2025-09-07 00:00:00

Antes do ISIN:  579
Depois do ISIN:  579


# LER BASE MEDICOS -> CLASSIFICAR TEC_MED EM JORNADA 1 (DF_FINAL1)

In [146]:
df_medicos = pd.read_excel(r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\Base Medicos\Medicos VisionOne.xlsx")

In [147]:
df_medicos.head()

,NOME do Médico
0,ADRIANA FALCAO VELOSO LYRA
1,ADRIANA PECORARE XAVIER
2,AFRA DE ABREU E LIMA MONTENEGRO
3,AILA IRINEU DE MOURA FREIRE
4,ALEXANDRE DANTAS SOARES QUINTAS SEGUNDO


# REPRODUZINDO OQUE SERIA O PROCV DO EXCEL, NA COLUNA SOLICITANTES PARA NOME DO MEDICO

In [148]:
# 1️⃣ Merge para 'solicitante' com 'NOME do Médico'
df_final1 = df_final1.merge(df_medicos[['NOME do Médico']], 
                             left_on='solicitante', 
                             right_on='NOME do Médico', 
                             how='left', 
                             suffixes=('', '_solici'))

# Criando a coluna 'solici' com as correspondências e 'NaN' onde não houver correspondência
df_final1['solici'] = df_final1['NOME do Médico'].where(df_final1['NOME do Médico'].notna())

# REPRODUZINDO OQUE SERIA O PROCV DO EXCEL, NA COLUNA MEDICO PARA NOME DO MEDICO

In [149]:
# 2️⃣ Merge para 'medico' com 'NOME do Médico'
df_final1 = df_final1.merge(df_medicos[['NOME do Médico']], 
                             left_on='medico', 
                             right_on='NOME do Médico', 
                             how='left', 
                             suffixes=('', '_medico'))

# Criando a coluna 'medico_match' com as correspondências e 'NaN' onde não houver correspondência
df_final1['med'] = df_final1['NOME do Médico'].where(df_final1['NOME do Médico'].notna())

# REPRODUZINDO A FORMULA QUE USAVAMOS EM EXCEL, PARA CLASSIFICAR O TECNICO E MEDICO

In [150]:
import numpy as np

# Definição das condições para a coluna 'tec_med'
condicoes = [
    df_final1['produto'] != "EXAME",  # Se 'produto' não for "EXAME", classifica como "médico"
    
    (df_final1['tipo_produto'] == "OFTALMO TIPO II") & 
    (df_final1['solici'].isna()) & (df_final1['med'].isna()),  # Caso ambos sejam NaN, classifica como "técnico"
    
    (df_final1['tipo_produto'] == "OFTALMO TIPO II") & 
    (df_final1['solici'].notna()) & (df_final1['med'].isna()),  # 'solici' não é NaN e 'med' é NaN → "técnico"
    
    (df_final1['tipo_produto'] == "OFTALMO TIPO II") & 
    (df_final1['solici'].isna()) & (df_final1['med'].notna()),  # 'solici' é NaN e 'med' não é NaN → "técnico"
    
    (df_final1['tipo_produto'] == "OFTALMO TIPO II") & 
    (df_final1['solici'].notna()) & (df_final1['med'].notna()),  # Ambos preenchidos → "médico"
    
    (df_final1['produto'] == "EXAME") & (df_final1['tipo_produto'] != "OFTALMO TIPO II") & 
    (df_final1['solici'].notna()) & (df_final1['med'].notna())  # Produto = "EXAME", tipo_produto ≠ "OFTALMO TIPO II", ambos preenchidos → "médico"
]

# Resultados correspondentes às condições
resultados = ["médico", "técnico", "técnico", "técnico", "médico", "médico"]

# Aplicação das condições para criar a coluna 'tec_med'
df_final1['tec_med'] = np.select(condicoes, resultados, default="técnico")

In [151]:
df_final1.head()

,id_transacao,data_transacao,frequencia,id_cliente,nome_do_cliente,telefone,email,idade,sexo,nome_loja,...,tipo_produto,solicitante,medico,produto,jornada2,NOME do Médico,solici,NOME do Médico_medico,med,tec_med
0,37755348,2025-09-08,Recorrente,8034250,CLEONICE,81987829575,Não deseja informar e-mail,61.0,FEMININO,HOSL,...,CONSULTA,MARIA EUGENIA MARTINS,MARIA EUGENIA MARTINS,CONSULTA,santa_luzia_consultas,NaN,NaN,NaN,NaN,médico
1,38010433,2025-09-08,Recorrente,10001966,SOPHIE,11958035401,gabb.bga@gmail.com,0.0,FEMININO,HORE,...,CONSULTA,DAENA LEAL,DAENA LEAL,CONSULTA,hope_consultas,NaN,NaN,NaN,NaN,médico
2,37968366,2025-09-08,Recorrente,10003149,IRENE,11982839265,mariainaildeferreiradasilva@gmail.com,82.0,FEMININO,HPAUL,...,CONSULTA,FELIPE LEAL DE MORAES BRITO,FELIPE LEAL DE MORAES BRITO,CONSULTA,holhos_consultas,FELIPE LEAL DE MORAES BRITO,FELIPE LEAL DE MORAES BRITO,FELIPE LEAL DE MORAES BRITO,FELIPE LEAL DE MORAES BRITO,médico
3,37820076,2025-09-08,Recorrente,10003695,KAZUMI,11972078020,kazumiyano2@gmail.com,50.0,FEMININO,HPAUL,...,CONSULTA,ANALIA SUEHIRO,ANALIA SUEHIRO,CONSULTA,holhos_consultas,ANALIA SUEHIRO,ANALIA SUEHIRO,ANALIA SUEHIRO,ANALIA SUEHIRO,médico
4,37879415,2025-09-08,Recorrente,10004430,SAMUEL,11976241079,vasgoncelosemilia194@gmail.com,1.0,MASCULINO,SANDRE,...,CONSULTA,GRACIELLA SAKAMITI,GRACIELLA SAKAMITI,CONSULTA,cerpo_consultas,NaN,NaN,NaN,NaN,médico


# CLASSIFICAR COLUNA TEC_MED NO JORNADA 2

In [152]:
# JORNADA 2 -> DF_FINAL 2

df_final2['tec_med'] = 'médico'

In [153]:
print("tec_med jornada2: ", df_final2['tec_med'].value_counts())

tec_med jornada2:  tec_med
médico    590
Name: count, dtype: int64


In [154]:
df_final2.head()

,id_transacao,data_transacao,frequencia,id_cliente,nome_do_cliente,telefone,email,idade,sexo,nome_loja,...,grupo_hospitalar,carater_admissao,jornada,subcategoria,tipo_produto,solicitante,medico,produto,jornada2,tec_med
10,37994646,2025-09-08,Recorrente,8034250,CLEONICE,81987829575,Não deseja informar e-mail,61.0,FEMININO,HOSL,...,HOSL,ELETIVO,Avaliação Cirúrgica,Catarata,CIRURGICO (OFTALMO),MARIA EUGENIA MARTINS,NAN,CIRURGIA,santa_luzia_centrocirurgico,médico
20,37990446,2025-09-08,Recorrente,10000874,ANTONIO,61984443479,naotem@gmail.com,64.0,MASCULINO,CBV MATRIZ,...,CBV,ELETIVO,Avaliação Cirúrgica,Retina,CIRURGICO (OFTALMO),TIAGO SUZUKI,NAN,CIRURGIA,cbv_centrocirurgico,médico
123,37779879,2025-09-08,Recorrente,10009850,SEVERINO,81971042460,Não deseja informar e-mail,88.0,MASCULINO,FSL,...,HOSL,ELETIVO,Avaliação Cirúrgica,Catarata,CIRURGICO (OFTALMO),LORENA GUERRA,NAN,CIRURGIA,santa_luzia_centrocirurgico,médico
134,37994495,2025-09-08,Recorrente,10011647,SALATIEL,81991959735,Não deseja informar e-mail,59.0,MASCULINO,HOSL,...,HOSL,ELETIVO,Avaliação Cirúrgica,Catarata,CIRURGICO (OFTALMO),MARIA EUGENIA MARTINS,NAN,CIRURGIA,santa_luzia_centrocirurgico,médico
143,37993119,2025-09-08,Recorrente,10011745,MARLUCE,81973234756,Não deseja informar e-mail,71.0,FEMININO,FSL,...,HOSL,ELETIVO,Avaliação Cirúrgica,Catarata,CIRURGICO (OFTALMO),MARIA EDUARDA PACHECO,NAN,CIRURGIA,santa_luzia_centrocirurgico,médico


# JORNADA 1 JÁ CLASSIFICADO TEC_MED -> BAIXAR SEM A COLUNA TIPO PRODUTO PARA NÃO TER DUPLICATAS, E SEM A COLUNA JORNADA COMUM -> USAR JORNADA PADRAO SOLUCX)

In [155]:
df_jornada1FINAL = df_final1.copy()

In [156]:
# LEMBRAR DE SEMPRE ATUALIZAR O DF DO DIA AQUI
# JORNADA 1

df_JD1xx = df_jornada1FINAL[['id_cliente', 'nome_do_cliente', 'email', 'telefone', 
                             'id_transacao', 'id_loja', 'data_transacao', 'jornada2', 'frequencia',
                       'idade', 'sexo',
                   'nome_loja', 'setores', 'especialidade',
                   'grupo_hospitalar', 'carater_admissao', 'subcategoria', 'solicitante', 'medico', 
                       'produto', 'tec_med']]

df_JD1xx = df_JD1xx.rename(columns={'jornada2': 'jornada'})

df_JD1xx = df_JD1xx[['id_cliente', 'nome_do_cliente', 'email', 'telefone', 
                             'id_transacao', 'id_loja', 'data_transacao', 'jornada', 'frequencia',
                       'idade', 'sexo',
                   'nome_loja', 'setores', 'especialidade',
                   'grupo_hospitalar', 'carater_admissao', 'subcategoria', 'solicitante', 'medico', 
                       'produto', 'tec_med']]

df_JD1xx = df_JD1xx.drop_duplicates()

In [157]:
df_JD1xx.head()

,id_cliente,nome_do_cliente,email,telefone,id_transacao,id_loja,data_transacao,jornada,frequencia,idade,...,nome_loja,setores,especialidade,grupo_hospitalar,carater_admissao,subcategoria,solicitante,medico,produto,tec_med
0,8034250,CLEONICE,Não deseja informar e-mail,81987829575,37755348,HOSL,2025-09-08,santa_luzia_consultas,Recorrente,61.0,...,HOSL,HOSL PRIMEIRO BLOCO CIRURGICO,OFTA,HOSL,ELETIVO,Eletivas,MARIA EUGENIA MARTINS,MARIA EUGENIA MARTINS,CONSULTA,médico
1,10001966,SOPHIE,gabb.bga@gmail.com,11958035401,38010433,HORE,2025-09-08,hope_consultas,Recorrente,0.0,...,HORE,HORE TERCEIRO NORTE PEDIATRIA,OFTA,HOPE,ELETIVO,Eletivas,DAENA LEAL,DAENA LEAL,CONSULTA,médico
2,10003149,IRENE,mariainaildeferreiradasilva@gmail.com,11982839265,37968366,HPAUL,2025-09-08,holhos_consultas,Recorrente,82.0,...,HPAUL,HPAUL HOSPITAL DECIMO ANDAR ATENDIMENTO,OFTA,V1 SP,ELETIVO,Eletivas,FELIPE LEAL DE MORAES BRITO,FELIPE LEAL DE MORAES BRITO,CONSULTA,médico
3,10003695,KAZUMI,kazumiyano2@gmail.com,11972078020,37820076,HPAUL,2025-09-08,holhos_consultas,Recorrente,50.0,...,HPAUL,HPAUL HOSPITAL NONO ANDAR ATENDIMENTO,OFTA,V1 SP,ELETIVO,Eletivas,ANALIA SUEHIRO,ANALIA SUEHIRO,CONSULTA,médico
4,10004430,SAMUEL,vasgoncelosemilia194@gmail.com,11976241079,37879415,SANDRE,2025-09-08,cerpo_consultas,Recorrente,1.0,...,SANDRE,SANDRE ATENDIMENTO,OFTA,CERPO,ELETIVO,Eletivas,GRACIELLA SAKAMITI,GRACIELLA SAKAMITI,CONSULTA,médico


# BAIXAR DF_FINAL JORNADA 1 (df_JD1XX) COMPLETO COM TODAS AS JORNADAS PARA USAR NA NEGATIVACAO (FACILITAR)

# ATUALIZAR DIA DA BASE NO NOME

In [158]:
# Filtra o DataFrame removendo a pessoa que tenha um dos valores fornecidos
df_JD1xx = df_JD1xx[
    ~(
        (df_JD1xx['email'] == 'sergiokalili@hotmail.com') |
        (df_JD1xx['telefone'] == '11976059197') |
        (df_JD1xx['id_cliente'] == '5989296')
    )
]

In [159]:
# VERIFICANDO
print(df_JD1xx[
    (df_JD1xx['email'] == 'sergiokalili@hotmail.com') |
    (df_JD1xx['telefone'] == '11976059197') |
    (df_JD1xx['id_cliente'] == '5989296')
])

Empty DataFrame
Columns: [id_cliente, nome_do_cliente, email, telefone, id_transacao, id_loja, data_transacao, jornada, frequencia, idade, sexo, nome_loja, setores, especialidade, grupo_hospitalar, carater_admissao, subcategoria, solicitante, medico, produto, tec_med]
Index: []

[0 rows x 21 columns]


In [160]:
#ALTERAR PARA CAMINHOS DA GABY -> NAS MESMAS PASTAS

import locale

# Definir a localidade para PT-BR
locale.setlocale(locale.LC_TIME, 'Portuguese_Brazil.1252')  # Para Windows
# locale.setlocale(locale.LC_TIME, 'pt_BR.utf8')  # Para Linux/Mac, caso necessário

# Obter a data atual
data_atual = datetime.today()
dia = data_atual.strftime('%d')  # Dia com dois dígitos
mes = data_atual.strftime('%b').capitalize()  # Nome do mês abreviado (Jan -> Jan, Fev -> Fev)
ano = data_atual.strftime('%y')  # Ano com dois dígitos

# Criar nome dinâmico dos arquivos
nome_base = f"SoluCX_QVIS_Jnd1_D{dia}_{mes}{ano}"

df_JD1xx = df_JD1xx.drop_duplicates(subset=['id_transacao'])

# Salvar DataFrame como Excel -> NA PASTA DE JORNADA 1 NORMAL
df_JD1xx.to_excel(fr"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\Bases_Finais_Subir_SoluCX_Jornada1\{nome_base}_EXCEL.xlsx", index=False)

# Salvar DataFrame como CSV -> NA PASTA DE JORNADA 1 NORMAL
df_JD1xx.to_csv(fr"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\Bases_Finais_Subir_SoluCX_Jornada1\{nome_base}_CSV.csv", sep=';', index=False)

# Salvar DataFrame como Excel -> NA PASTA DE JORNADA 1 NEGATIVAR
df_JD1xx.to_excel(fr"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\Negativar_Jornada1\{nome_base}_EXCEL.xlsx", index=False)

# COPIAR DF_FINAL DE JORNADA 1 (df_JD1xx)
# SEPARAR POR JORNADAS (CONSULTAS, EXAMES, LENTES E URGENCIA)
# BAIXAR SEPARADO CADA E SUBIR AS 4 BASES

In [161]:
df_separar = df_JD1xx.copy()

In [162]:
df_separar.head()

,id_cliente,nome_do_cliente,email,telefone,id_transacao,id_loja,data_transacao,jornada,frequencia,idade,...,nome_loja,setores,especialidade,grupo_hospitalar,carater_admissao,subcategoria,solicitante,medico,produto,tec_med
0,8034250,CLEONICE,Não deseja informar e-mail,81987829575,37755348,HOSL,2025-09-08,santa_luzia_consultas,Recorrente,61.0,...,HOSL,HOSL PRIMEIRO BLOCO CIRURGICO,OFTA,HOSL,ELETIVO,Eletivas,MARIA EUGENIA MARTINS,MARIA EUGENIA MARTINS,CONSULTA,médico
1,10001966,SOPHIE,gabb.bga@gmail.com,11958035401,38010433,HORE,2025-09-08,hope_consultas,Recorrente,0.0,...,HORE,HORE TERCEIRO NORTE PEDIATRIA,OFTA,HOPE,ELETIVO,Eletivas,DAENA LEAL,DAENA LEAL,CONSULTA,médico
2,10003149,IRENE,mariainaildeferreiradasilva@gmail.com,11982839265,37968366,HPAUL,2025-09-08,holhos_consultas,Recorrente,82.0,...,HPAUL,HPAUL HOSPITAL DECIMO ANDAR ATENDIMENTO,OFTA,V1 SP,ELETIVO,Eletivas,FELIPE LEAL DE MORAES BRITO,FELIPE LEAL DE MORAES BRITO,CONSULTA,médico
3,10003695,KAZUMI,kazumiyano2@gmail.com,11972078020,37820076,HPAUL,2025-09-08,holhos_consultas,Recorrente,50.0,...,HPAUL,HPAUL HOSPITAL NONO ANDAR ATENDIMENTO,OFTA,V1 SP,ELETIVO,Eletivas,ANALIA SUEHIRO,ANALIA SUEHIRO,CONSULTA,médico
4,10004430,SAMUEL,vasgoncelosemilia194@gmail.com,11976241079,37879415,SANDRE,2025-09-08,cerpo_consultas,Recorrente,1.0,...,SANDRE,SANDRE ATENDIMENTO,OFTA,CERPO,ELETIVO,Eletivas,GRACIELLA SAKAMITI,GRACIELLA SAKAMITI,CONSULTA,médico


In [163]:
df_separar['jornada'].value_counts()

jornada
holhos_consultas                  1253
hope_consultas                    1160
holhos_exames                      547
santa_luzia_consultas              445
cerpo_consultas                    430
holhos_urgencia                    403
hope_exames                        385
cbv_consultas                      367
hope_urgencia                      251
cerpo_exames                       161
horp_consultas                     158
santa_luzia_exames                 145
hoc_consultas                      123
oftalmos_consultas                 108
cbv_urgencia                        88
cbv_exames                          87
santa_luzia_urgencia                77
oftalmos_exames                     65
laser_ocular_consultas              63
hoa_consultas                       49
hro_urgencia                        42
hoc_exames                          37
laser_ocular_exames                 31
holhos_lentes_de_contato            21
horp_exames                         16
cerpo_lentes_de_c

In [164]:
df_separar['grupo_hospitalar'].value_counts()

grupo_hospitalar
V1 SP       2224
HOPE        1805
HOSL         668
CERPO        606
CBV          544
OFTALMOS     175
HORP         174
HOC          161
LASER         96
HOA           57
HRO           56
Name: count, dtype: int64

In [165]:
df_separar['grupo_hospitalar'] = df_separar['grupo_hospitalar'].replace("HOLHOS", "V1 SP")
df_separar.head()

,id_cliente,nome_do_cliente,email,telefone,id_transacao,id_loja,data_transacao,jornada,frequencia,idade,...,nome_loja,setores,especialidade,grupo_hospitalar,carater_admissao,subcategoria,solicitante,medico,produto,tec_med
0,8034250,CLEONICE,Não deseja informar e-mail,81987829575,37755348,HOSL,2025-09-08,santa_luzia_consultas,Recorrente,61.0,...,HOSL,HOSL PRIMEIRO BLOCO CIRURGICO,OFTA,HOSL,ELETIVO,Eletivas,MARIA EUGENIA MARTINS,MARIA EUGENIA MARTINS,CONSULTA,médico
1,10001966,SOPHIE,gabb.bga@gmail.com,11958035401,38010433,HORE,2025-09-08,hope_consultas,Recorrente,0.0,...,HORE,HORE TERCEIRO NORTE PEDIATRIA,OFTA,HOPE,ELETIVO,Eletivas,DAENA LEAL,DAENA LEAL,CONSULTA,médico
2,10003149,IRENE,mariainaildeferreiradasilva@gmail.com,11982839265,37968366,HPAUL,2025-09-08,holhos_consultas,Recorrente,82.0,...,HPAUL,HPAUL HOSPITAL DECIMO ANDAR ATENDIMENTO,OFTA,V1 SP,ELETIVO,Eletivas,FELIPE LEAL DE MORAES BRITO,FELIPE LEAL DE MORAES BRITO,CONSULTA,médico
3,10003695,KAZUMI,kazumiyano2@gmail.com,11972078020,37820076,HPAUL,2025-09-08,holhos_consultas,Recorrente,50.0,...,HPAUL,HPAUL HOSPITAL NONO ANDAR ATENDIMENTO,OFTA,V1 SP,ELETIVO,Eletivas,ANALIA SUEHIRO,ANALIA SUEHIRO,CONSULTA,médico
4,10004430,SAMUEL,vasgoncelosemilia194@gmail.com,11976241079,37879415,SANDRE,2025-09-08,cerpo_consultas,Recorrente,1.0,...,SANDRE,SANDRE ATENDIMENTO,OFTA,CERPO,ELETIVO,Eletivas,GRACIELLA SAKAMITI,GRACIELLA SAKAMITI,CONSULTA,médico


# GERANDO LETRAS ALEATORIAS TODOS OS DIAS PARA CONCATENAR NO ID_TRANSACAO (DESVIAR DOS PROBLEMAS DE DUPLICATAS QUE TEM AO SUBIR NA PLATAFORMA)

In [166]:
import random
import string

# Função para gerar um código aleatório baseado no dia
def gerar_sufixo():
    data_str = datetime.now().strftime("%Y%m%d")  # Converte a data para string (AAAAMMDD)
    random.seed(data_str)  # Usa a string da data como semente
    return ''.join(random.choices(string.ascii_letters, k=3))  # 5 letras aleatórias

sufixo_diario = gerar_sufixo()

In [167]:
# CONSULTAS
consultas11_df = ['santa_luzia_consultas', 
                  'hope_consultas', 
                  'hro_consultas', 
                  'oftalmos_consultas', 
                  'hoc_consultas',
                 'holhos_consultas', 
                  'cerpo_consultas', 
                  'laser_ocular_consultas',
                 'cbv_consultas', 'hoa_consultas', 'horp_consultas']
df_consultas11 = df_separar[df_separar['jornada'].isin(consultas11_df)]
df_consultas11 = df_consultas11.drop_duplicates(subset=['id_cliente'])
df_consultas11['id_transacao'] = df_consultas11['id_transacao'].astype(str) + 'Cons' + sufixo_diario

# EXAMES
exames11_df = ['santa_luzia_exames', 
               'hope_exames', 
               'hro_exames', 
               'oftalmos_exames', 
               'hoc_exames',
              'holhos_exames', 
               'cerpo_exames', 
               'laser_ocular_exames',
              'cbv_exames', 'hoa_exames', 'horp_exames']
df_exames11 = df_separar[df_separar['jornada'].isin(exames11_df)]
df_exames11 = df_exames11.drop_duplicates(subset=['id_cliente'])
df_exames11['id_transacao'] = df_exames11['id_transacao'].astype(str) + 'Exa' + sufixo_diario

# LENTES
lentes11_df = ['santa_luzia_lentes_de_contato', 
               'hro_lentes_de_contato', 
               'hope_lentes_de_contato', 
               'oftalmos_lentes_de_contato', 
               'hoc_lentes_de_contato',
              'holhos_lentes_de_contato', 
               'cerpo_lentes_de_contato', 
               'laser_ocular_lentes_de_contato',
              'cbv_lentes_de_contato',  'hoa_lentes_de_contato', 'horp_lentes_de_contato']
df_lentes11 = df_separar[df_separar['jornada'].isin(lentes11_df)]
df_lentes11 = df_lentes11.drop_duplicates(subset=['id_cliente'])
df_lentes11['id_transacao'] = df_lentes11['id_transacao'].astype(str) + 'Len' + sufixo_diario

# URGENCIA
urgencia11_df = ['santa_luzia_urgencia', 
                 'hro_urgencia', 
                 'hope_urgencia', 
                 'oftalmos_urgencia', 
                 'hoc_urgencia',
                'holhos_urgencia', 
                 'cerpo_urgencia', 
                 'laser_ocular_urgencia',
                'cbv_urgencia', 'hoa_urgencia', 'horp_urgencia']
df_urgencia11 = df_separar[df_separar['jornada'].isin(urgencia11_df)]
df_urgencia11 = df_urgencia11.drop_duplicates(subset=['id_cliente'])
df_urgencia11['id_transacao'] = df_urgencia11['id_transacao'].astype(str) + 'Urg' + sufixo_diario

In [168]:
print("Jornadas Consultas: ", df_consultas11['jornada'].value_counts())
print("\nJornadas Exames: ", df_exames11['jornada'].value_counts())
print("\nJornadas Lentes: ", df_lentes11['jornada'].value_counts())
print("\nJornadas Urgencia: ", df_urgencia11['jornada'].value_counts())

Jornadas Consultas:  jornada
holhos_consultas          1253
hope_consultas            1160
santa_luzia_consultas      445
cerpo_consultas            430
cbv_consultas              367
horp_consultas             158
hoc_consultas              123
oftalmos_consultas         108
laser_ocular_consultas      63
hoa_consultas               49
hro_consultas                8
Name: count, dtype: int64

Jornadas Exames:  jornada
holhos_exames          547
hope_exames            385
cerpo_exames           161
santa_luzia_exames     145
cbv_exames              87
oftalmos_exames         65
hoc_exames              37
laser_ocular_exames     31
horp_exames             16
hoa_exames               8
hro_exames               6
Name: count, dtype: int64

Jornadas Lentes:  jornada
holhos_lentes_de_contato          21
cerpo_lentes_de_contato           12
hope_lentes_de_contato             9
laser_ocular_lentes_de_contato     2
oftalmos_lentes_de_contato         2
cbv_lentes_de_contato              2
santa

# ATUALIZAR DIA DA BASE NO NOME

In [169]:
##ALTERAR PARA CAMINHOS DA GABY -> NAS MESMAS PASTAS

import locale
from datetime import datetime

# Definir a localidade para PT-BR
locale.setlocale(locale.LC_TIME, 'Portuguese_Brazil.1252')  # Para Windows
# locale.setlocale(locale.LC_TIME, 'pt_BR.utf8')  # Para Linux/Mac, caso necessário

# Obter a data atual
data_atual = datetime.today()
dia = data_atual.strftime('%d')  # Dia com dois dígitos
mes = data_atual.strftime('%b').capitalize()  # Nome do mês abreviado (Jan -> Jan, Fev -> Fev)
ano = data_atual.strftime('%y')  # Ano com dois dígitos

# Criar nome dinâmico dos arquivos
nome_base = f"SoluCX_QVIS_D{dia}_{mes}{ano}"

# JORNADA 1

# CONSULTAS -> DIRETO NA PASTA DO DRIVE DE CONSULTAS
df_consultas11 = df_consultas11.drop_duplicates(subset=['id_transacao'])
df_consultas11.to_excel(fr"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\Base_Finais_Subir_SoluCX_Consultas\{nome_base}_Cons_EXCEL.xlsx", index=False)
df_consultas11.to_csv(fr"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\Base_Finais_Subir_SoluCX_Consultas\{nome_base}_Cons_CSV.csv", sep=';', index=False)

################################################################################################################################

# EXAMES -> DIRETO NA PASTA DO DRIVE DE EXAMES
df_exames11 = df_exames11.drop_duplicates(subset=['id_transacao'])
df_exames11.to_excel(fr"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\Base_Finais_Subir_SoluCX_Exames\{nome_base}_Exa_EXCEL.xlsx", index=False)
df_exames11.to_csv(fr"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\Base_Finais_Subir_SoluCX_Exames\{nome_base}_Exa_CSV.csv", sep=';', index=False)

################################################################################################################################

# LENTES -> DIRETO NA PASTA DO DRIVE DE LENTES
df_lentes11 = df_lentes11.drop_duplicates(subset=['id_transacao'])
df_lentes11.to_excel(fr"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\Base_Finais_Subir_SoluCX_Lentes\{nome_base}_Len_EXCEL.xlsx", index=False)
df_lentes11.to_csv(fr"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\Base_Finais_Subir_SoluCX_Lentes\{nome_base}_Len_CSV.csv", sep=';', index=False)

################################################################################################################################

# URGENCIA -> DIRETO NA PASTA DO DRIVE DE URGENCIA
df_urgencia11 = df_urgencia11.drop_duplicates(subset=['id_transacao'])
df_urgencia11.to_excel(fr"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\Base_Finais_Subir_SoluCX_Urgencia\{nome_base}_Urge_EXCEL.xlsx", index=False)
df_urgencia11.to_csv(fr"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\Base_Finais_Subir_SoluCX_Urgencia\{nome_base}_Urge_CSV.csv", sep=';', index=False)

In [170]:
df_consultas11.sort_values(by=['id_cliente']).head(15)

,id_cliente,nome_do_cliente,email,telefone,id_transacao,id_loja,data_transacao,jornada,frequencia,idade,...,nome_loja,setores,especialidade,grupo_hospitalar,carater_admissao,subcategoria,solicitante,medico,produto,tec_med
1,10001966,SOPHIE,gabb.bga@gmail.com,11958035401,38010433ConsjGY,HORE,2025-09-08,hope_consultas,Recorrente,0.0,...,HORE,HORE TERCEIRO NORTE PEDIATRIA,OFTA,HOPE,ELETIVO,Eletivas,DAENA LEAL,DAENA LEAL,CONSULTA,médico
2,10003149,IRENE,mariainaildeferreiradasilva@gmail.com,11982839265,37968366ConsjGY,HPAUL,2025-09-08,holhos_consultas,Recorrente,82.0,...,HPAUL,HPAUL HOSPITAL DECIMO ANDAR ATENDIMENTO,OFTA,V1 SP,ELETIVO,Eletivas,FELIPE LEAL DE MORAES BRITO,FELIPE LEAL DE MORAES BRITO,CONSULTA,médico
3,10003695,KAZUMI,kazumiyano2@gmail.com,11972078020,37820076ConsjGY,HPAUL,2025-09-08,holhos_consultas,Recorrente,50.0,...,HPAUL,HPAUL HOSPITAL NONO ANDAR ATENDIMENTO,OFTA,V1 SP,ELETIVO,Eletivas,ANALIA SUEHIRO,ANALIA SUEHIRO,CONSULTA,médico
4,10004430,SAMUEL,vasgoncelosemilia194@gmail.com,11976241079,37879415ConsjGY,SANDRE,2025-09-08,cerpo_consultas,Recorrente,1.0,...,SANDRE,SANDRE ATENDIMENTO,OFTA,CERPO,ELETIVO,Eletivas,GRACIELLA SAKAMITI,GRACIELLA SAKAMITI,CONSULTA,médico
5,10005028,ANA,Não deseja informar e-mail,61992778704,37996574ConsjGY,CBV AGUAS CLARAS,2025-09-08,cbv_consultas,Primeira Vez,64.0,...,CBV AGUAS CLARAS,CBV AGUAS CLARAS TERREO ATENDIMENTO,OFTA,CBV,ELETIVO,Eletivas,RODRIGO CASTRO,RODRIGO CASTRO,CONSULTA,médico
6,10005684,MATHEUS,matheuspsqueiroz @gmail.com,11912920368,38001347ConsjGY,HABC,2025-09-08,holhos_consultas,Recorrente,20.0,...,HABC,HABC HOSPITAL OITAVO ANDAR ATENDIMENTO,OFTA,V1 SP,ELETIVO,Eletivas,MARILIA ROCHA COSTA,MARILIA ROCHA COSTA,CONSULTA,médico
7,1000576,ALEXANDRA,Não deseja informar e-mail,81999212133,37951662ConsjGY,UASR,2025-09-08,hope_consultas,Recorrente,62.0,...,UASR,UASR CONSULTORIOS,OFTA,HOPE,ELETIVO,Eletivas,PEDRO LEONARDO SORIANO DA SILVA,PEDRO LEONARDO SORIANO DA SILVA,CONSULTA,médico
8,10006697,JULIANA,july_melo@hotmail.com,98991020037,36708552ConsjGY,CBV MATRIZ,2025-09-08,cbv_consultas,Recorrente,40.0,...,CBV MATRIZ,CBV MATRIZ ATENDIMENTO,OFTA,CBV,ELETIVO,Eletivas,FABRICIO TADEU,FABRICIO TADEU,CONSULTA,médico
9,10006839,MAITE,vivianfalopa@gmail.com,11986806779,37998686ConsjGY,HPAUL,2025-09-08,holhos_consultas,Recorrente,9.0,...,HPAUL,HPAUL HOSPITAL OITAVO ANDAR ATENDIMENTO,OFTA,V1 SP,ELETIVO,Eletivas,CAIO CONTI,CAIO CONTI,CONSULTA,médico
10,10007464,JOAO,Não deseja informar e-mail,61983437810,37768641ConsjGY,HPAUL,2025-09-08,holhos_consultas,Recorrente,25.0,...,HPAUL,HPAUL HOSPITAL QUINTO ANDAR ATENDIMENTO,OFTA,V1 SP,ELETIVO,Eletivas,THAIS BORGES,THAIS BORGES,CONSULTA,médico


# JORNADA 2 JÁ CLASSIFICADO TEC_MED E BAIXAR DE NOVO SEM A COLUNA TIPO PRODUTO PARA NÃO TER DUPLICATAS,  E SEM A COLUNA JORNADA COMUM -> USAR JORNADA PADRAO SOLUCX)

In [171]:
# LEMBRAR DE SEMPRE ATUALIZAR O DF DO DIA AQUI
# JORNADA 2

df_jornada2FINAL = df_final2.copy()

In [172]:
# LEMBRAR DE SEMPRE ATUALIZAR O DF DO DIA AQUI
# JORNADA 2

df_JD2yy = df_jornada2FINAL[['id_cliente', 'nome_do_cliente', 'email', 'telefone', 
                             'id_transacao', 'id_loja', 'data_transacao', 'jornada2', 'frequencia',
                       'idade', 'sexo',
                   'nome_loja', 'setores', 'especialidade',
                   'grupo_hospitalar', 'carater_admissao', 'subcategoria', 'solicitante', 'medico', 
                       'produto', 'tec_med']]

df_JD2yy = df_JD2yy.rename(columns={'jornada2': 'jornada'})

df_JD2yy = df_JD2yy[['id_cliente', 'nome_do_cliente', 'email', 'telefone', 
                             'id_transacao', 'id_loja', 'data_transacao', 'jornada', 'frequencia',
                       'idade', 'sexo',
                   'nome_loja', 'setores', 'especialidade',
                   'grupo_hospitalar', 'carater_admissao', 'subcategoria', 'solicitante', 'medico', 
                       'produto', 'tec_med']]

df_JD2yy = df_JD2yy.drop_duplicates()

In [173]:
# Filtra o DataFrame removendo a pessoa que tenha um dos valores fornecidos
df_JD2yy = df_JD2yy[
    ~(
        (df_JD2yy['email'] == 'sergiokalili@hotmail.com') |
        (df_JD2yy['telefone'] == '11976059197') |
        (df_JD2yy['id_cliente'] == '5989296')
    )
]

In [174]:
# VERIFICANDO
print(df_JD2yy[
    (df_JD2yy['email'] == 'sergiokalili@hotmail.com') |
    (df_JD2yy['telefone'] == '11976059197') |
    (df_JD2yy['id_cliente'] == '5989296')
])

Empty DataFrame
Columns: [id_cliente, nome_do_cliente, email, telefone, id_transacao, id_loja, data_transacao, jornada, frequencia, idade, sexo, nome_loja, setores, especialidade, grupo_hospitalar, carater_admissao, subcategoria, solicitante, medico, produto, tec_med]
Index: []

[0 rows x 21 columns]


In [175]:
##ALTERAR PARA CAMINHOS DA GABY -> NAS MESMAS PASTAS

import random
import string
import locale
from datetime import datetime

# Definir a localidade para PT-BR
locale.setlocale(locale.LC_TIME, 'Portuguese_Brazil.1252')  # Para Windows
# locale.setlocale(locale.LC_TIME, 'pt_BR.utf8')  # Para Linux/Mac, caso necessário

# Função para gerar um código aleatório baseado na data
def gerar_sufixo():
    data_str = datetime.now().strftime("%Y%m%d")  # Converte a data para string (AAAAMMDD)
    random.seed(data_str)  # Usa a string da data como semente
    return ''.join(random.choices(string.ascii_letters, k=3))  # 3 letras aleatórias

# Obter a data atual
data_atual = datetime.today()
dia = data_atual.strftime('%d')  # Dia com dois dígitos
mes = data_atual.strftime('%b').capitalize()  # Nome do mês abreviado (Jan -> Jan, Fev -> Fev)
ano = data_atual.strftime('%y')  # Ano com dois dígitos

# Criar nome dinâmico dos arquivos
nome_base = f"SoluCX_QVIS_Jnd2_D{dia}_{mes}{ano}"

# JORNADA 2

df_JD2yy = df_JD2yy.drop_duplicates(subset=['id_transacao'])

# Gerar sufixo aleatório para o id_transacao
sufixo_aleatorio = gerar_sufixo()

# Adicionando o sufixo 'Cir' e o código aleatório ao id_transacao
df_JD2yy['id_transacao'] = df_JD2yy['id_transacao'].astype(str) + 'Cir' + sufixo_aleatorio

# Salvar DataFrame como Excel -> NA PASTA DE JORNADA 2 NORMAL
df_JD2yy.to_excel(fr"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\Bases_Final_Subir_SoluCX_Jornada2\{nome_base}_EXCEL.xlsx", index=False)

# Salvar DataFrame como CSV -> NA PASTA DE JORNADA 2 NORMAL
df_JD2yy.to_csv(fr"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\Bases_Final_Subir_SoluCX_Jornada2\{nome_base}_CSV.csv", sep=';', index=False)

# Salvar DataFrame como Excel -> NA PASTA DE JORNADA 2 NEGATIVAR
df_JD2yy.to_excel(fr"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\Negativar_Jornada2\{nome_base}_EXCEL.xlsx", index=False)

# /////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# /////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# ENVIAR BASES DIARIAS PELA API

# 1o) DF CONSULTAS (CÓDIGO OK)

df_consultas11

In [176]:
df_consultas11.shape[0]

4164

In [177]:
df_consultas11 = df_consultas11.drop_duplicates()
print(df_consultas11.shape[0])

4164


In [178]:
df_consultas11 = df_consultas11.drop_duplicates(subset=['id_transacao'])
print(df_consultas11.shape[0])
print(df_consultas11['id_transacao'].nunique())

4164
4164


In [179]:
df_consultas11['data_transacao'].min()

Timestamp('2025-09-08 00:00:00')

In [180]:
df_consultas11['data_transacao'].max()

Timestamp('2025-09-08 00:00:00')

In [181]:
df_consultas11.head(20)

,id_cliente,nome_do_cliente,email,telefone,id_transacao,id_loja,data_transacao,jornada,frequencia,idade,...,nome_loja,setores,especialidade,grupo_hospitalar,carater_admissao,subcategoria,solicitante,medico,produto,tec_med
0,8034250,CLEONICE,Não deseja informar e-mail,81987829575,37755348ConsjGY,HOSL,2025-09-08,santa_luzia_consultas,Recorrente,61.0,...,HOSL,HOSL PRIMEIRO BLOCO CIRURGICO,OFTA,HOSL,ELETIVO,Eletivas,MARIA EUGENIA MARTINS,MARIA EUGENIA MARTINS,CONSULTA,médico
1,10001966,SOPHIE,gabb.bga@gmail.com,11958035401,38010433ConsjGY,HORE,2025-09-08,hope_consultas,Recorrente,0.0,...,HORE,HORE TERCEIRO NORTE PEDIATRIA,OFTA,HOPE,ELETIVO,Eletivas,DAENA LEAL,DAENA LEAL,CONSULTA,médico
2,10003149,IRENE,mariainaildeferreiradasilva@gmail.com,11982839265,37968366ConsjGY,HPAUL,2025-09-08,holhos_consultas,Recorrente,82.0,...,HPAUL,HPAUL HOSPITAL DECIMO ANDAR ATENDIMENTO,OFTA,V1 SP,ELETIVO,Eletivas,FELIPE LEAL DE MORAES BRITO,FELIPE LEAL DE MORAES BRITO,CONSULTA,médico
3,10003695,KAZUMI,kazumiyano2@gmail.com,11972078020,37820076ConsjGY,HPAUL,2025-09-08,holhos_consultas,Recorrente,50.0,...,HPAUL,HPAUL HOSPITAL NONO ANDAR ATENDIMENTO,OFTA,V1 SP,ELETIVO,Eletivas,ANALIA SUEHIRO,ANALIA SUEHIRO,CONSULTA,médico
4,10004430,SAMUEL,vasgoncelosemilia194@gmail.com,11976241079,37879415ConsjGY,SANDRE,2025-09-08,cerpo_consultas,Recorrente,1.0,...,SANDRE,SANDRE ATENDIMENTO,OFTA,CERPO,ELETIVO,Eletivas,GRACIELLA SAKAMITI,GRACIELLA SAKAMITI,CONSULTA,médico
5,10005028,ANA,Não deseja informar e-mail,61992778704,37996574ConsjGY,CBV AGUAS CLARAS,2025-09-08,cbv_consultas,Primeira Vez,64.0,...,CBV AGUAS CLARAS,CBV AGUAS CLARAS TERREO ATENDIMENTO,OFTA,CBV,ELETIVO,Eletivas,RODRIGO CASTRO,RODRIGO CASTRO,CONSULTA,médico
6,10005684,MATHEUS,matheuspsqueiroz @gmail.com,11912920368,38001347ConsjGY,HABC,2025-09-08,holhos_consultas,Recorrente,20.0,...,HABC,HABC HOSPITAL OITAVO ANDAR ATENDIMENTO,OFTA,V1 SP,ELETIVO,Eletivas,MARILIA ROCHA COSTA,MARILIA ROCHA COSTA,CONSULTA,médico
7,1000576,ALEXANDRA,Não deseja informar e-mail,81999212133,37951662ConsjGY,UASR,2025-09-08,hope_consultas,Recorrente,62.0,...,UASR,UASR CONSULTORIOS,OFTA,HOPE,ELETIVO,Eletivas,PEDRO LEONARDO SORIANO DA SILVA,PEDRO LEONARDO SORIANO DA SILVA,CONSULTA,médico
8,10006697,JULIANA,july_melo@hotmail.com,98991020037,36708552ConsjGY,CBV MATRIZ,2025-09-08,cbv_consultas,Recorrente,40.0,...,CBV MATRIZ,CBV MATRIZ ATENDIMENTO,OFTA,CBV,ELETIVO,Eletivas,FABRICIO TADEU,FABRICIO TADEU,CONSULTA,médico
9,10006839,MAITE,vivianfalopa@gmail.com,11986806779,37998686ConsjGY,HPAUL,2025-09-08,holhos_consultas,Recorrente,9.0,...,HPAUL,HPAUL HOSPITAL OITAVO ANDAR ATENDIMENTO,OFTA,V1 SP,ELETIVO,Eletivas,CAIO CONTI,CAIO CONTI,CONSULTA,médico


In [182]:
import requests

In [183]:
df_consultas11['id_loja'].value_counts()

id_loja
HORE                            620
HPAUL                           601
HABC                            498
CBV MATRIZ                      326
HOSL                            267
SAMARO                          199
FSL                             178
HORP                            158
UASR                            133
SANDRE                          125
HOC                             123
UASRM                            97
UASG                             91
OFTALMOS BC                      76
UASP                             73
NEO                              69
LASER                            63
CLINOFT                          60
CEOSP                            59
HOA                              49
DIAD                             38
SCSUL                            36
MAUA                             32
OFTALMOS ITA                     32
CEOFT (DIFUSORA)                 28
MOLI                             24
CBV AGUAS CLARAS                 23
CEOFT               

import pandas as pd
import requests
import math
import json

df_consultas11 = df_consultas11.fillna("NULL")

# Links, Tokens e Chaves API
url = "https://api.solucx.com.br/public/transaction"
headers = {
    "x-solucx-api-key": "39e480bfa1003c605ed15bec3e95e657",
    "x-solucx-user-token": "V3GVYx14g2KNmjpkGoe0",
    "Content-Type": "application/json"
}

# Inicialização de contadores
contagem_cons_total = 0
contagem_cons_erros = 0

# Iterar linha por linha (sem lotes)
for index, row in df_consultas11.iterrows():
    data_consultas = {
        "customer_integration": row['id_cliente'],
        "name": row['nome_do_cliente'],           
        "email": row['email'],            
        "phone": row['telefone'],
        "transaction_id": row['id_transacao'],
        "store_integration": row['id_loja'],
        "register_date": row['data_transacao'].strftime('%Y-%m-%d') if isinstance(row['data_transacao'], pd.Timestamp) else row['data_transacao'],
        "journey": row['jornada'],        
        "param_frequencia": row['frequencia'],
        "param_idade": row['idade'],
        "param_sexo": row['sexo'],                 
        "param_nome_loja": row['nome_loja'],
        "param_setores": row['setores'],
        "param_especialidade": row['especialidade'],
        "param_grupo_hospitalar": row['grupo_hospitalar'],
        "param_carater_admissao": row['carater_admissao'],
        "param_subcategoria": row['subcategoria'],
        "param_solicitante": row['solicitante'],
        "param_medico": row['medico'],
        "param_produto": row['produto'],
        "param_tec_med": row['tec_med']
    }

    try:
        # Enviar requisição individual
        response = requests.post(url, headers=headers, json=data_consultas)
        contagem_cons_total += 1

        if response.status_code not in [200, 201, 202, 204]:
            contagem_cons_erros += 1
            #print(f"\n⚠️ Erro no envio do registro {index}")
            #print(f"Status Code: {response.status_code}")
            #try:
            #    print(f"Resposta JSON: {response.json()}")
            #except Exception:
            #    print("⚠️ Não foi possível interpretar a resposta JSON.")
            #print(f"Registro com erro:\n{json.dumps(data_consultas, indent=2)}")
    
    except requests.exceptions.RequestException as e:
        contagem_cons_erros += 1
        print(f"\n❌ Erro na requisição do registro {index}: {e}")
        print(f"Registro com erro:\n{json.dumps(data_consultas, indent=2)}")

# Total de erros
print(f"\n❗ Total de erros nas transações: {contagem_cons_erros}")
# Total de transações enviadas
print(f"✅ Total de transações enviadas: {contagem_cons_total}")

# Verificação final
if contagem_cons_total == df_consultas11.shape[0]:
    print("✔️ Todos os dados foram enviados com sucesso.") 
else:
    print("⚠️ Envio incompleto!")



OTIMIZADO AQUI!

In [184]:
import pandas as pd
import requests
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- Mapeamento dos grupos hospitalares ---
map_hospitalar = {
    'V1 SP': ['CLINOFT', 'HABC', 'HPAUL', 'MOLI', 'VISO', 'ULC', 'CEOSP'],
    'CERPO': ['DIAD', 'SAMARO', 'SANDRE', 'SCSUL', 'MAUA'],
    'LASER': ['LASER'],
    'HOPE': ['CEOFT', 'CEOFT (CEO)', 'CEOFT (DIFUSORA)', 'HORE', 'NEO', 'UASG', 'UASP', 
             'UASPT', 'UASR', 'UASRM', 'COVC', 'HOPE'],
    'HRO': ['HRO (GOLDEN)', 'HRO (ILHA)', 'HRO (MATRIZ)', 'HRO (RIO ANIL)', 'HRO (SUS)', 'VMAIS'],
    'OFTALMOS': ['OFTALMOS BC', 'OFTALMOS ITA', 'OFTALMOS MK'],
    'HOSL': ['FSL', 'HOSL'],
    'CBV': ['CBV MATRIZ', 'CBV AGUAS CLARAS', 'CBV TAGUATINGA', 'CBV LUMINAR', 'CBV HOSPITAL DE AGUAS CLARAS'],
    'HOA': ['HOA', 'HOSPITAL ARARAQUARA'],
    'HORP': ['HORP', 'TOKUNAGA']
}

# --- Mapeia cada unidade para seu grupo hospitalar ---
unidade_para_grupo = {
    unidade: grupo for grupo, unidades in map_hospitalar.items() for unidade in unidades
}

# --- Caminho do arquivo de reenvio ---
caminho_txt = r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\reenvio_consultas.txt"

# --- Preenchendo valores nulos e aplicando mapeamento ---
df_consultas11 = df_consultas11.fillna("")
df_consultas11['grupo_hospitalar'] = df_consultas11['grupo_hospitalar'].map(unidade_para_grupo).fillna(df_consultas11['grupo_hospitalar'])

# --- Configuração da API ---
url = "https://api.solucx.com.br/public/transaction"
headers = {
    "x-solucx-api-key": "39e480bfa1003c605ed15bec3e95e657",
    "x-solucx-user-token": "V3GVYx14g2KNmjpkGoe0",
    "Content-Type": "application/json"
}

# --- Contadores globais ---
contagem_cons_total = 0
contagem_cons_erros = 0

# --- Lista para armazenar payloads com erro ---
payloads_com_erro = []

# --- Carregar payloads antigos do TXT (se houver) ---
payloads_antigos = []
try:
    with open(caminho_txt, "r", encoding="utf-8") as f:
        for linha in f:
            if linha.strip():
                try:
                    payloads_antigos.append(json.loads(linha.strip()))
                except json.JSONDecodeError:
                    pass
    if payloads_antigos:
        print(f"📂 {len(payloads_antigos)} payload(s) recuperado(s) do reenvio.txt")
except FileNotFoundError:
    pass  # se não existe ainda, ignora

# --- Função para montar payload a partir de um row do dataframe ---
def montar_payload(row):
    return {
        "customer_integration": str(row['id_cliente']),
        "name": row['nome_do_cliente'],           
        "email": row['email'],            
        "phone": row['telefone'],
        "transaction_id": str(row['id_transacao']),
        "store_integration": row['id_loja'],
        "register_date": row['data_transacao'].strftime('%Y-%m-%d') 
                         if isinstance(row['data_transacao'], pd.Timestamp) 
                         else row['data_transacao'],
        "journey": row['jornada'],        
        "param_frequencia": row['frequencia'],
        "param_idade": row['idade'],
        "param_sexo": row['sexo'],                 
        "param_nome_loja": row['nome_loja'],
        "param_setores": row['setores'],
        "param_especialidade": row['especialidade'],
        "param_grupo_hospitalar": row['grupo_hospitalar'],
        "param_carater_admissao": row['carater_admissao'],
        "param_subcategoria": row['subcategoria'],
        "param_solicitante": row['solicitante'],
        "param_medico": row['medico'],
        "param_produto": row['produto'],
        "param_tec_med": row['tec_med']
    }

# --- Função para enviar um payload ---
def enviar_payload(payload):
    try:
        response = requests.post(url, headers=headers, json=payload, timeout=30)
        if response.status_code in [201]:
            return True
        else:
            payloads_com_erro.append(payload)
            return False
    except requests.exceptions.RequestException:
        payloads_com_erro.append(payload)
        return False

# --- Criar lista de payloads a enviar ---
payloads_atuais = [montar_payload(row) for _, row in df_consultas11.iterrows()]
#todos_payloads = payloads_antigos + payloads_atuais
todos_payloads = payloads_atuais

print(f"🚀 Enviando {len(todos_payloads)} payloads (atuais + pendentes anteriores)...")

# --- Envio com ThreadPool ---
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(enviar_payload, p): p for p in todos_payloads}
    for future in as_completed(futures):
        if future.result():
            contagem_cons_total += 1
        else:
            contagem_cons_erros += 1

# --- Totais finais ---
print(f"\n❗ Total de erros: {contagem_cons_erros}")
print(f"✅ Total enviados: {contagem_cons_total}")

# --- Atualizar TXT com falhas ---
with open(caminho_txt, "w", encoding="utf-8") as f:
    for payload in payloads_com_erro:
        f.write(json.dumps(payload, ensure_ascii=False) + "\n")

print(f"📂 Arquivo de reenvio atualizado: {caminho_txt}")



📂 9 payload(s) recuperado(s) do reenvio.txt
🚀 Enviando 4164 payloads (atuais + pendentes anteriores)...

❗ Total de erros: 0
✅ Total enviados: 4164
📂 Arquivo de reenvio atualizado: C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\reenvio_consultas.txt


In [185]:
df_consultas11['grupo_hospitalar'].value_counts()

grupo_hospitalar
V1 SP       1253
HOPE        1160
HOSL         445
CERPO        430
CBV          367
HORP         158
HOC          123
OFTALMOS     108
LASER         63
HOA           49
HRO            8
Name: count, dtype: int64

# 2o) DF EXAMES (CÓDIGO OK)

df_exames11

In [186]:
df_exames11.shape[0]

1488

In [187]:
import pandas as pd
import requests
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- Mapeamento dos grupos hospitalares ---
map_hospitalar = {
    'V1 SP': ['CLINOFT', 'HABC', 'HPAUL', 'MOLI', 'VISO', 'ULC', 'CEOSP'],
    'CERPO': ['DIAD', 'SAMARO', 'SANDRE', 'SCSUL', 'MAUA'],
    'LASER': ['LASER'],
    'HOPE': ['CEOFT', 'CEOFT (CEO)', 'CEOFT (DIFUSORA)', 'HORE', 'NEO', 'UASG', 'UASP', 
             'UASPT', 'UASR', 'UASRM', 'COVC', 'HOPE'],
    'HRO': ['HRO (GOLDEN)', 'HRO (ILHA)', 'HRO (MATRIZ)', 'HRO (RIO ANIL)', 'HRO (SUS)', 'VMAIS'],
    'OFTALMOS': ['OFTALMOS BC', 'OFTALMOS ITA', 'OFTALMOS MK'],
    'HOSL': ['FSL', 'HOSL'],
    'CBV': ['CBV MATRIZ', 'CBV AGUAS CLARAS', 'CBV TAGUATINGA', 'CBV LUMINAR', 'CBV HOSPITAL DE AGUAS CLARAS'],
    'HOA': ['HOA', 'HOSPITAL ARARAQUARA'],
    'HORP': ['HORP', 'TOKUNAGA']
}

# --- Mapeia cada unidade para seu grupo hospitalar ---
unidade_para_grupo = {
    unidade: grupo for grupo, unidades in map_hospitalar.items() for unidade in unidades
}

# --- Caminho do arquivo de reenvio para exames ---
caminho_txt = r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\reenvioexames.txt"

# --- Preenchendo valores nulos e aplicando mapeamento ---
df_exames11 = df_exames11.fillna("")
df_exames11['grupo_hospitalar'] = df_exames11['grupo_hospitalar'].map(unidade_para_grupo).fillna(df_exames11['grupo_hospitalar'])

# --- Configuração da API ---
url = "https://api.solucx.com.br/public/transaction"
headers = {
    "x-solucx-api-key": "39e480bfa1003c605ed15bec3e95e657",
    "x-solucx-user-token": "V3GVYx14g2KNmjpkGoe0",
    "Content-Type": "application/json"
}

# --- Contadores globais ---
contagem_exames_total = 0
contagem_exames_erros = 0

# --- Lista para armazenar payloads com erro ---
payloads_com_erro = []

# --- Carregar payloads antigos do TXT (se houver) ---
payloads_antigos = []
try:
    with open(caminho_txt, "r", encoding="utf-8") as f:
        for linha in f:
            if linha.strip():
                try:
                    payloads_antigos.append(json.loads(linha.strip()))
                except json.JSONDecodeError:
                    pass
    if payloads_antigos:
        print(f"📂 {len(payloads_antigos)} payload(s) recuperado(s) do reenvio.txt")
except FileNotFoundError:
    pass  # se não existe ainda, ignora

# --- Função para montar payload a partir de um row do dataframe ---
def montar_payload(row):
    return {
        "customer_integration": str(row['id_cliente']),
        "name": row['nome_do_cliente'],           
        "email": row['email'],            
        "phone": row['telefone'],
        "transaction_id": str(row['id_transacao']),
        "store_integration": row['id_loja'],
        "register_date": row['data_transacao'].strftime('%Y-%m-%d') 
                         if isinstance(row['data_transacao'], pd.Timestamp) 
                         else row['data_transacao'],
        "journey": row['jornada'],        
        "param_frequencia": row['frequencia'],
        "param_idade": row['idade'],
        "param_sexo": row['sexo'],                 
        "param_nome_loja": row['nome_loja'],
        "param_setores": row['setores'],
        "param_especialidade": row['especialidade'],
        "param_grupo_hospitalar": row['grupo_hospitalar'],
        "param_carater_admissao": row['carater_admissao'],
        "param_subcategoria": row['subcategoria'],
        "param_solicitante": row['solicitante'],
        "param_medico": row['medico'],
        "param_produto": row['produto'],
        "param_tec_med": row['tec_med']
    }

# --- Função para enviar um payload ---
def enviar_payload(payload):
    try:
        response = requests.post(url, headers=headers, json=payload, timeout=10)
        if response.status_code in [201]:
            return True
        else:
            payloads_com_erro.append(payload)
            return False
    except requests.exceptions.RequestException:
        payloads_com_erro.append(payload)
        return False

# --- Criar lista de payloads a enviar ---
payloads_atuais = [montar_payload(row) for _, row in df_exames11.iterrows()]
#todos_payloads = payloads_antigos + payloads_atuais
todos_payloads = payloads_atuais
print(f"🚀 Enviando {len(todos_payloads)} payloads (atuais + pendentes anteriores)...")

# --- Envio com ThreadPool ---
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(enviar_payload, p): p for p in todos_payloads}
    for future in as_completed(futures):
        if future.result():
            contagem_exames_total += 1
        else:
            contagem_exames_erros += 1

# --- Totais finais ---
print(f"\n❗ Total de erros: {contagem_exames_erros}")
print(f"✅ Total enviados: {contagem_exames_total}")

# --- Atualizar TXT com falhas ---
with open(caminho_txt, "w", encoding="utf-8") as f:
    for payload in payloads_com_erro:
        f.write(json.dumps(payload, ensure_ascii=False) + "\n")

print(f"📂 Arquivo de reenvio atualizado: {caminho_txt}")


🚀 Enviando 1488 payloads (atuais + pendentes anteriores)...

❗ Total de erros: 1
✅ Total enviados: 1487
📂 Arquivo de reenvio atualizado: C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\reenvioexames.txt


# 3o) DF LENTES (CÓDIGO OK)

df_lentes11

In [188]:
df_lentes11.shape[0]

49

In [189]:
import pandas as pd
import requests
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- Mapeamento dos grupos hospitalares ---
map_hospitalar = {
    'V1 SP': ['CLINOFT', 'HABC', 'HPAUL', 'MOLI', 'VISO', 'ULC', 'CEOSP'],
    'CERPO': ['DIAD', 'SAMARO', 'SANDRE', 'SCSUL', 'MAUA'],
    'LASER': ['LASER'],
    'HOPE': ['CEOFT', 'CEOFT (CEO)', 'CEOFT (DIFUSORA)', 'HORE', 'NEO', 'UASG', 'UASP', 
             'UASPT', 'UASR', 'UASRM', 'COVC', 'HOPE'],
    'HRO': ['HRO (GOLDEN)', 'HRO (ILHA)', 'HRO (MATRIZ)', 'HRO (RIO ANIL)', 'HRO (SUS)', 'VMAIS'],
    'OFTALMOS': ['OFTALMOS BC', 'OFTALMOS ITA', 'OFTALMOS MK'],
    'HOSL': ['FSL', 'HOSL'],
    'CBV': ['CBV MATRIZ', 'CBV AGUAS CLARAS', 'CBV TAGUATINGA', 'CBV LUMINAR', 'CBV HOSPITAL DE AGUAS CLARAS'],
    'HOA': ['HOA', 'HOSPITAL ARARAQUARA'],
    'HORP': ['HORP', 'TOKUNAGA']
}

# --- Mapeia cada unidade para seu grupo hospitalar ---
unidade_para_grupo = {
    unidade: grupo for grupo, unidades in map_hospitalar.items() for unidade in unidades
}

# --- Caminho do arquivo de reenvio para lentes ---
caminho_txt = r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\reenviolentes.txt"

# --- Preenchendo valores nulos e aplicando mapeamento ---
df_lentes11 = df_lentes11.fillna("")
df_lentes11['grupo_hospitalar'] = df_lentes11['grupo_hospitalar'].map(unidade_para_grupo).fillna(df_lentes11['grupo_hospitalar'])

# --- Configuração da API ---
url = "https://api.solucx.com.br/public/transaction"
headers = {
    "x-solucx-api-key": "39e480bfa1003c605ed15bec3e95e657",
    "x-solucx-user-token": "V3GVYx14g2KNmjpkGoe0",
    "Content-Type": "application/json"
}

# --- Contadores globais ---
contagem_lentes_total = 0
contagem_lentes_erros = 0

# --- Lista para armazenar payloads com erro ---
payloads_com_erro = []

# --- Carregar payloads antigos do TXT (se houver) ---
payloads_antigos = []
try:
    with open(caminho_txt, "r", encoding="utf-8") as f:
        for linha in f:
            if linha.strip():
                try:
                    payloads_antigos.append(json.loads(linha.strip()))
                except json.JSONDecodeError:
                    pass
    if payloads_antigos:
        print(f"📂 {len(payloads_antigos)} payload(s) recuperado(s) do reenvio.txt")
except FileNotFoundError:
    pass  # se não existe ainda, ignora

# --- Função para montar payload a partir de um row do dataframe ---
def montar_payload(row):
    return {
        "customer_integration": row['id_cliente'],
        "name": row['nome_do_cliente'],           
        "email": row['email'],            
        "phone": row['telefone'],
        "transaction_id": str(row['id_transacao']),
        "store_integration": row['id_loja'],
        "register_date": row['data_transacao'].strftime('%Y-%m-%d') 
                         if isinstance(row['data_transacao'], pd.Timestamp) 
                         else row['data_transacao'],
        "journey": row['jornada'],        
        "param_frequencia": row['frequencia'],
        "param_idade": row['idade'],
        "param_sexo": row['sexo'],                 
        "param_nome_loja": row['nome_loja'],
        "param_setores": row['setores'],
        "param_especialidade": row['especialidade'],
        "param_grupo_hospitalar": row['grupo_hospitalar'],
        "param_carater_admissao": row['carater_admissao'],
        "param_subcategoria": row['subcategoria'],
        "param_solicitante": row['solicitante'],
        "param_medico": row['medico'],
        "param_produto": row['produto'],
        "param_tec_med": row['tec_med']
    }

# --- Função para enviar um payload ---
def enviar_payload(payload):
    try:
        response = requests.post(url, headers=headers, json=payload, timeout=10)
        if response.status_code in [201]:
            return True
        else:
            payloads_com_erro.append(payload)
            return False
    except requests.exceptions.RequestException:
        payloads_com_erro.append(payload)
        return False

# --- Criar lista de payloads a enviar ---
payloads_atuais = [montar_payload(row) for _, row in df_lentes11.iterrows()]
#todos_payloads = payloads_antigos + payloads_atuais
todos_payloads = payloads_atuais

print(f"🚀 Enviando {len(todos_payloads)} payloads (atuais + pendentes anteriores)...")

# --- Envio com ThreadPool ---
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(enviar_payload, p): p for p in todos_payloads}
    for future in as_completed(futures):
        if future.result():
            contagem_lentes_total += 1
        else:
            contagem_lentes_erros += 1

# --- Totais finais ---
print(f"\n❗ Total de erros: {contagem_lentes_erros}")
print(f"✅ Total enviados: {contagem_lentes_total}")

# --- Atualizar TXT com falhas ---
with open(caminho_txt, "w", encoding="utf-8") as f:
    for payload in payloads_com_erro:
        f.write(json.dumps(payload, ensure_ascii=False) + "\n")

print(f"📂 Arquivo de reenvio atualizado: {caminho_txt}")


🚀 Enviando 49 payloads (atuais + pendentes anteriores)...

❗ Total de erros: 0
✅ Total enviados: 49
📂 Arquivo de reenvio atualizado: C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\reenviolentes.txt


# 4o) DF URGENCIA (CÓDIGO OK)

df_urgencia11

In [190]:
df_urgencia11.shape[0]

865

In [191]:
import pandas as pd
import requests
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- Mapeamento dos grupos hospitalares ---
map_hospitalar = {
    'V1 SP': ['CLINOFT', 'HABC', 'HPAUL', 'MOLI', 'VISO', 'ULC', 'CEOSP'],
    'CERPO': ['DIAD', 'SAMARO', 'SANDRE', 'SCSUL', 'MAUA'],
    'LASER': ['LASER'],
    'HOPE': ['CEOFT', 'CEOFT (CEO)', 'CEOFT (DIFUSORA)', 'HORE', 'NEO', 'UASG', 'UASP', 
             'UASPT', 'UASR', 'UASRM', 'COVC', 'HOPE'],
    'HRO': ['HRO (GOLDEN)', 'HRO (ILHA)', 'HRO (MATRIZ)', 'HRO (RIO ANIL)', 'HRO (SUS)', 'VMAIS'],
    'OFTALMOS': ['OFTALMOS BC', 'OFTALMOS ITA', 'OFTALMOS MK'],
    'HOSL': ['FSL', 'HOSL'],
    'CBV': ['CBV MATRIZ', 'CBV AGUAS CLARAS', 'CBV TAGUATINGA', 'CBV LUMINAR', 'CBV HOSPITAL DE AGUAS CLARAS'],
    'HOA': ['HOA', 'HOSPITAL ARARAQUARA'],
    'HORP': ['HORP', 'TOKUNAGA']
}

# --- Mapeia cada unidade para seu grupo hospitalar ---
unidade_para_grupo = {
    unidade: grupo for grupo, unidades in map_hospitalar.items() for unidade in unidades
}

# --- Caminho do arquivo de reenvio para urgência ---
caminho_txt = r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\reenviourgencia.txt"

# --- Preenchendo valores nulos e aplicando mapeamento ---
df_urgencia11 = df_urgencia11.fillna("")
df_urgencia11['grupo_hospitalar'] = df_urgencia11['grupo_hospitalar'].map(unidade_para_grupo).fillna(df_urgencia11['grupo_hospitalar'])

# --- Configuração da API ---
url = "https://api.solucx.com.br/public/transaction"
headers = {
    "x-solucx-api-key": "39e480bfa1003c605ed15bec3e95e657",
    "x-solucx-user-token": "V3GVYx14g2KNmjpkGoe0",
    "Content-Type": "application/json"
}

# --- Contadores globais ---
contagem_urgencia_total = 0
contagem_urgencia_erros = 0

# --- Lista para armazenar payloads com erro ---
payloads_com_erro = []

# --- Carregar payloads antigos do TXT (se houver) ---
payloads_antigos = []
try:
    with open(caminho_txt, "r", encoding="utf-8") as f:
        for linha in f:
            if linha.strip():
                try:
                    payloads_antigos.append(json.loads(linha.strip()))
                except json.JSONDecodeError:
                    pass
    if payloads_antigos:
        print(f"📂 {len(payloads_antigos)} payload(s) recuperado(s) do reenvio.txt")
except FileNotFoundError:
    pass  # se não existe ainda, ignora

# --- Função para montar payload a partir de um row do dataframe ---
def montar_payload(row):
    return {
        "customer_integration": row['id_cliente'],
        "name": row['nome_do_cliente'],           
        "email": row['email'],            
        "phone": row['telefone'],
        "transaction_id": str(row['id_transacao']),
        "store_integration": row['id_loja'],
        "register_date": row['data_transacao'].strftime('%Y-%m-%d') 
                         if isinstance(row['data_transacao'], pd.Timestamp) 
                         else row['data_transacao'],
        "journey": row['jornada'],        
        "param_frequencia": row['frequencia'],
        "param_idade": row['idade'],
        "param_sexo": row['sexo'],                 
        "param_nome_loja": row['nome_loja'],
        "param_setores": row['setores'],
        "param_especialidade": row['especialidade'],
        "param_grupo_hospitalar": row['grupo_hospitalar'],
        "param_carater_admissao": row['carater_admissao'],
        "param_subcategoria": row['subcategoria'],
        "param_solicitante": row['solicitante'],
        "param_medico": row['medico'],
        "param_produto": row['produto'],
        "param_tec_med": row['tec_med']
    }

# --- Função para enviar um payload ---
def enviar_payload(payload):
    try:
        response = requests.post(url, headers=headers, json=payload, timeout=10)
        if response.status_code in [201]:
            return True
        else:
            payloads_com_erro.append(payload)
            return False
    except requests.exceptions.RequestException:
        payloads_com_erro.append(payload)
        return False

# --- Criar lista de payloads a enviar ---
payloads_atuais = [montar_payload(row) for _, row in df_urgencia11.iterrows()]
#todos_payloads = payloads_antigos + payloads_atuais
todos_payloads = payloads_atuais

print(f"🚀 Enviando {len(todos_payloads)} payloads (atuais + pendentes anteriores)...")

# --- Envio com ThreadPool ---
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(enviar_payload, p): p for p in todos_payloads}
    for future in as_completed(futures):
        if future.result():
            contagem_urgencia_total += 1
        else:
            contagem_urgencia_erros += 1

# --- Totais finais ---
print(f"\n❗ Total de erros: {contagem_urgencia_erros}")
print(f"✅ Total enviados: {contagem_urgencia_total}")

# --- Atualizar TXT com falhas ---
with open(caminho_txt, "w", encoding="utf-8") as f:
    for payload in payloads_com_erro:
        f.write(json.dumps(payload, ensure_ascii=False) + "\n")

print(f"📂 Arquivo de reenvio atualizado: {caminho_txt}")


🚀 Enviando 865 payloads (atuais + pendentes anteriores)...

❗ Total de erros: 0
✅ Total enviados: 865
📂 Arquivo de reenvio atualizado: C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\reenviourgencia.txt


# 5o) DF CIRURGIAS (CÓDIGO OK)

df_JD2yy

In [192]:
df_JD2yy.shape[0]

590

In [193]:
import pandas as pd
import requests
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- Mapeamento dos grupos hospitalares ---
map_hospitalar = {
    'HOC': ['HOC', 'HOVG'],
    'V1 SP': ['CLINOFT', 'HABC', 'HPAUL', 'MOLI', 'VISO', 'ULC', 'CEOSP'],
    'CERPO': ['DIAD', 'SAMARO', 'SANDRE', 'SCSUL', 'MAUA'],
    'LASER': ['LASER'],
    'HOPE': ['CEOFT', 'CEOFT (CEO)', 'CEOFT (DIFUSORA)', 'HORE', 'NEO', 'UASG', 'UASP', 
             'UASPT', 'UASR', 'UASRM', 'COVC', 'HOPE'],
    'HRO': ['HRO (GOLDEN)', 'HRO (ILHA)', 'HRO (MATRIZ)', 'HRO (RIO ANIL)', 'HRO (SUS)', 'VMAIS'],
    'OFTALMOS': ['OFTALMOS BC', 'OFTALMOS ITA', 'OFTALMOS MK'],
    'HOSL': ['FSL', 'HOSL'],
    'CBV': ['CBV MATRIZ', 'CBV AGUAS CLARAS', 'CBV TAGUATINGA', 'CBV LUMINAR', 'CBV HOSPITAL DE AGUAS CLARAS'],
    'HOA': ['HOA', 'HOSPITAL ARARAQUARA'],
    'HORP': ['HORP', 'TOKUNAGA']
}

# --- Mapeia cada unidade para seu grupo hospitalar ---
unidade_para_grupo = {
    unidade: grupo for grupo, unidades in map_hospitalar.items() for unidade in unidades
}

# --- Caminho do arquivo de reenvio para cirurgias ---
caminho_txt = r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\reenviocirurgias.txt"

# --- Preenchendo valores nulos e aplicando mapeamento ---
df_JD2yy = df_JD2yy.fillna("")
df_JD2yy['grupo_hospitalar'] = df_JD2yy['grupo_hospitalar'].map(unidade_para_grupo).fillna(df_JD2yy['grupo_hospitalar'])

# --- Configuração da API ---
url = "https://api.solucx.com.br/public/transaction"
headers = {
    "x-solucx-api-key": "39e480bfa1003c605ed15bec3e95e657",
    "x-solucx-user-token": "V3GVYx14g2KNmjpkGoe0",
    "Content-Type": "application/json"
}

# --- Contadores globais ---
contagem_cirurgias_total = 0
contagem_cirurgias_erros = 0

# --- Lista para armazenar payloads com erro ---
payloads_com_erro = []

# --- Carregar payloads antigos do TXT (se houver) ---
payloads_antigos = []
try:
    with open(caminho_txt, "r", encoding="utf-8") as f:
        for linha in f:
            if linha.strip():
                try:
                    payloads_antigos.append(json.loads(linha.strip()))
                except json.JSONDecodeError:
                    pass
    if payloads_antigos:
        print(f"📂 {len(payloads_antigos)} payload(s) recuperado(s) do reenvio.txt")
except FileNotFoundError:
    pass  # se não existe ainda, ignora

# --- Função para montar payload a partir de um row do dataframe ---
def montar_payload(row):
    return {
        "customer_integration": row['id_cliente'],
        "name": row['nome_do_cliente'],           
        "email": row['email'],            
        "phone": row['telefone'],
        "transaction_id": str(row['id_transacao']),
        "store_integration": row['id_loja'],
        "register_date": row['data_transacao'].strftime('%Y-%m-%d') 
                         if isinstance(row['data_transacao'], pd.Timestamp) 
                         else row['data_transacao'],
        "journey": row['jornada'],        
        "param_frequencia": row['frequencia'],
        "param_idade": row['idade'],
        "param_sexo": row['sexo'],                 
        "param_nome_loja": row['nome_loja'],
        "param_setores": row['setores'],
        "param_especialidade": row['especialidade'],
        "param_grupo_hospitalar": row['grupo_hospitalar'],
        "param_carater_admissao": row['carater_admissao'],
        "param_subcategoria": row['subcategoria'],
        "param_solicitante": row['solicitante'],
        "param_medico": row['medico'],
        "param_produto": row['produto'],
        "param_tec_med": row['tec_med']
    }

# --- Função para enviar um payload ---
def enviar_payload(payload):
    try:
        response = requests.post(url, headers=headers, json=payload, timeout=10)
        if response.status_code in [201]:
            return True
        else:
            payloads_com_erro.append(payload)
            return False
    except requests.exceptions.RequestException:
        payloads_com_erro.append(payload)
        return False

# --- Criar lista de payloads a enviar ---
payloads_atuais = [montar_payload(row) for _, row in df_JD2yy.iterrows()]
#todos_payloads = payloads_antigos + payloads_atuais
todos_payloads = payloads_atuais

print(f"🚀 Enviando {len(todos_payloads)} payloads (atuais + pendentes anteriores)...")

# --- Envio com ThreadPool ---
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(enviar_payload, p): p for p in todos_payloads}
    for future in as_completed(futures):
        if future.result():
            contagem_cirurgias_total += 1
        else:
            contagem_cirurgias_erros += 1

# --- Totais finais ---
print(f"\n❗ Total de erros: {contagem_cirurgias_erros}")
print(f"✅ Total enviados: {contagem_cirurgias_total}")

# --- Atualizar TXT com falhas ---
with open(caminho_txt, "w", encoding="utf-8") as f:
    for payload in payloads_com_erro:
        f.write(json.dumps(payload, ensure_ascii=False) + "\n")

print(f"📂 Arquivo de reenvio atualizado: {caminho_txt}")


🚀 Enviando 590 payloads (atuais + pendentes anteriores)...

❗ Total de erros: 0
✅ Total enviados: 590
📂 Arquivo de reenvio atualizado: C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\reenviocirurgias.txt


REENVIO - 1h depois

In [194]:
time.sleep(3600)

import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

# --- Configuração da API ---
URL = "https://api.solucx.com.br/public/transaction"
HEADERS = {
    "x-solucx-api-key": "39e480bfa1003c605ed15bec3e95e657",
    "x-solucx-user-token": "V3GVYx14g2KNmjpkGoe0",
    "Content-Type": "application/json"
}

# --- Arquivos de falha por jornada ---
TXT_FALHAS = {
    "consultas": r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\reenvioconsultas.txt",
    "exames":    r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\reenvioexames.txt",
    "lentes":    r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\reenviolentes.txt",
    "cirurgias": r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\reenviocirurgias.txt",
    "urgencia":  r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\reenviourgencia.txt"
}

# --- Funções de leitura e escrita dos TXT de falhas ---
def ler_falhas_txt(caminho):
    if os.path.exists(caminho):
        with open(caminho, "r", encoding="utf-8") as f:
            return [l.strip() for l in f.readlines() if l.strip()]
    return []

def salvar_falhas_txt(caminho, falhas):
    with open(caminho, "w", encoding="utf-8") as f:
        # remove duplicados e salva ordenado
        f.write("\n".join(sorted(set(falhas))))

# --- Monta payload para envio à API ---
def montar_payload(row, falhas_anteriores=None):
    payload = {
        "customer_integration": row.get('id_cliente', ''),
        "name": row.get('nome_do_cliente', ''),
        "email": row.get('email', ''),
        "phone": row.get('telefone', ''),
        "transaction_id": row.get('id_transacao', ''),
        "store_integration": row.get('id_loja', ''),
        "register_date": row['data_transacao'].strftime('%Y-%m-%d') 
                         if isinstance(row.get('data_transacao'), pd.Timestamp) 
                         else row.get('data_transacao', ''),
        "journey": row.get('jornada', ''),
        "param_frequencia": row.get('frequencia', ''),
        "param_idade": row.get('idade', ''),
        "param_sexo": row.get('sexo', ''),
        "param_nome_loja": row.get('nome_loja', ''),
        "param_setores": row.get('setores', ''),
        "param_especialidade": row.get('especialidade', ''),
        "param_grupo_hospitalar": row.get('grupo_hospitalar', ''),
        "param_carater_admissao": row.get('carater_admissao', ''),
        "param_subcategoria": row.get('subcategoria', ''),
        "param_solicitante": row.get('solicitante', ''),
        "param_medico": row.get('medico', ''),
        "param_produto": row.get('produto', ''),
        "param_tec_med": row.get('tec_med', '')
    }
    if falhas_anteriores:
        payload["falhas_anteriores"] = falhas_anteriores
    return payload

# --- Envio de um registro ---
def enviar_registro(row, falhas_anteriores):
    payload = montar_payload(row, falhas_anteriores)
    try:
        resp = requests.post(URL, headers=HEADERS, json=payload, timeout=10)
        if resp.status_code in [200]:
            return True, None
        else:
            return False, row.get('id_transacao', str(payload))
    except requests.exceptions.RequestException:
        return False, row.get('id_transacao', str(payload))

# --- Processamento de uma jornada ---
def processar_jornada(nome_jornada, df):
    if df.empty:
        print(f"⚠️ Nenhum registro para {nome_jornada}")
        return

    caminho_txt = TXT_FALHAS[nome_jornada]
    falhas_existentes = ler_falhas_txt(caminho_txt)
    falhas_atualizadas = list(falhas_existentes)

    enviados, erros = 0, 0

    print(f"\n🚀 Enviando {len(df)} registros de {nome_jornada}...")

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(enviar_registro, row, falhas_existentes): idx for idx, row in df.iterrows()}
        for future in as_completed(futures):
            sucesso, falha_id = future.result()
            if sucesso:
                enviados += 1
                # remove do TXT se estava lá
                if falha_id and falha_id in falhas_atualizadas:
                    falhas_atualizadas.remove(falha_id)
            else:
                erros += 1
                if falha_id not in falhas_atualizadas:
                    falhas_atualizadas.append(falha_id)

    # Atualiza TXT
    salvar_falhas_txt(caminho_txt, falhas_atualizadas)

    print(f"📊 {nome_jornada}: enviados={enviados}, erros={erros}, pendentes em TXT={len(falhas_atualizadas)}")

# --- Dicionário com todas as jornadas e seus DataFrames ---
jornadas = {
    "consultas": df_consultas11,
    "exames": df_exames11,
    "lentes": df_lentes11,
    "urgencia": df_urgencia11,
    "cirurgias": df_JD2yy
}

# --- Processa todas as jornadas ---
for nome, df in jornadas.items():
    processar_jornada(nome, df)

print("\n✅ Finalizado o envio de todas as jornadas!")


In [195]:
import os
import json
import requests

# --- Configuração da API ---
URL = "https://api.solucx.com.br/public/transaction"
HEADERS = {
    "x-solucx-api-key": "39e480bfa1003c605ed15bec3e95e657",
    "x-solucx-user-token": "V3GVYx14g2KNmjpkGoe0",
    "Content-Type": "application/json"
}

# --- Caminhos dos arquivos .txt ---
TXT_FALHAS = {
    "consultas": r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\reenvioconsultas.txt",
    "exames":    r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\reenvioexames.txt",
    "lentes":    r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\reenviolentes.txt",
    "cirurgias": r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\reenviocirurgias.txt",
    "urgencia":  r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\reenviourgencia.txt"
}

# Caminho do arquivo de falhas detalhadas
FALHAS_DETALHADAS = r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\TODO QVIS\falhas_detalhadas.txt"

# --- Reenvia registros de um arquivo e atualiza o arquivo com as falhas restantes ---
def tentar_reenvio(caminho_txt, nome_arquivo):
    if not os.path.exists(caminho_txt):
        print(f"⚠️ Arquivo não encontrado: {caminho_txt}")
        return

    with open(caminho_txt, "r", encoding="utf-8") as f:
        linhas = f.readlines()

    restantes = []
    enviados = 0
    erros = 0
    falhas_detalhadas = []

    for linha in linhas:
        linha = linha.strip()
        if not linha:
            continue
        try:
            payload = json.loads(linha)
            resp = requests.post(URL, headers=HEADERS, json=payload, timeout=15)

            if resp.status_code == 201:
                enviados += 1
            else:
                restantes.append(linha)
                erros += 1
                falhas_detalhadas.append({
                    "arquivo": nome_arquivo,
                    "payload": payload,
                    "status_code": resp.status_code,
                    "response_text": resp.text.strip()
                })
        except Exception as e:
            restantes.append(linha)
            erros += 1
            falhas_detalhadas.append({
                "arquivo": nome_arquivo,
                "payload": linha,
                "erro": str(e)
            })

    # Salva falhas detalhadas (append no arquivo geral)
    if falhas_detalhadas:
        with open(FALHAS_DETALHADAS, "a", encoding="utf-8") as f:
            for falha in falhas_detalhadas:
                f.write(json.dumps(falha, ensure_ascii=False) + "\n")

    # Sobrescreve o arquivo original com as falhas restantes
    with open(caminho_txt, "w", encoding="utf-8") as f:
        for linha in restantes:
            f.write(linha.strip() + "\n")

    print(f"📁 {os.path.basename(caminho_txt)} - Enviados: {enviados} | Falhas: {erros}")


# --- Executa reenvio para todos os arquivos ---
if __name__ == "__main__":
    # Limpa o arquivo de falhas detalhadas (opcional — comente se quiser acumular histórico)
    with open(FALHAS_DETALHADAS, "w", encoding="utf-8") as f:
        f.write("")

    for nome, caminho in TXT_FALHAS.items():
        tentar_reenvio(caminho, nome)

    print("\n✅ Finalizado o reenvio dos arquivos.")


📁 reenvioconsultas.txt - Enviados: 0 | Falhas: 1101
📁 reenvioexames.txt - Enviados: 1 | Falhas: 0
📁 reenviolentes.txt - Enviados: 0 | Falhas: 0
📁 reenviocirurgias.txt - Enviados: 0 | Falhas: 0
📁 reenviourgencia.txt - Enviados: 0 | Falhas: 0

✅ Finalizado o reenvio dos arquivos.


In [196]:
# --- Payload único para envio direto ---
payload_unico = {
    "customer_integration": "M10004986",
    "name": "MARCELOTESTE20260826",
    "email": "mfmcarvalho0@gmail.com",
    "phone": "81996909573",
    "transaction_id": "37909240ConssRzbw",
    "store_integration": "HORE",
    "register_date": "2025-08-25",
    "journey": "hope_consultas",
    "param_frequencia": "Recorrente",
    "param_idade": 4.0,
    "param_sexo": "MASCULINO",
    "param_nome_loja": "HORE",
    "param_setores": "HORE SEGUNDO SUL OTORRINO",
    "param_especialidade": "OTOR",
    "param_grupo_hospitalar": "HOPE",
    "param_carater_admissao": "ELETIVO",
    "param_subcategoria": "Eletivas",
    "param_solicitante": "MILTON S L S JUNIOR",
    "param_medico": "MILTON S L S JUNIOR",
    "param_produto": "CONSULTA",
    "param_tec_med": "médico"
}

# --- Envio direto ---
response = requests.post(url, headers=headers, json=payload_unico)

# --- Ver resultado ---
if response.status_code == 201:
    print("✅ Payload enviado com sucesso.")
else:
    print(f"❌ Erro ao enviar payload. Código: {response.status_code}")
    print(f"🔍 Detalhes: {response.text}")


❌ Erro ao enviar payload. Código: 400
🔍 Detalhes: {"error":["Transaction already created"]}


import requests
from datetime import datetime

url = "https://api.solucx.com.br/public/transaction"
headers = {
    "x-solucx-api-key": "39e480bfa1003c605ed15bec3e95e657",
    "x-solucx-user-token": "V3GVYx14g2KNmjpkGoe0",
    "Content-Type": "application/json"
}

# --- Dados fictícios ---
fake_data = {
    "customer_integration": "12345670",
    "name": "Marcelo Carvalho",
    "email": "mfmcarvalho0@gmail.com",
    "phone": "11987654321",
    "transaction_id": "TX987654",
    "store_integration": "LOJA001",
    "register_date": datetime.today().strftime('%Y-%m-%d'),
    "journey": "holhos_consultas",
    "param_frequencia": "Primeira vez",
    "param_idade": "45",
    "param_sexo": "M",
    "param_nome_loja": "Hospital de Olhos Exemplo",
    "param_setores": "Oftalmologia",
    "param_especialidade": "Catarata",
    "param_grupo_hospitalar": "HOC",
    "param_carater_admissao": "Eletiva",
    "param_subcategoria": "Oftalmo",
    "param_solicitante": "Dr. João da Silva",
    "param_medico": "Dr. José Oftalmo",
    "param_produto": "Lente Intraocular Premium",
    "param_tec_med": "Facoemulsificação"
}

# --- Fazendo request ---
try:
    response = requests.post(url, headers=headers, json=fake_data, timeout=10)
    print("Status:", response.status_code)
    print("Resposta:", response.text)
except requests.exceptions.RequestException as e:
    print("Erro ao enviar:", e)


In [197]:
#!/usr/bin/env python
# coding: utf-8

import smtplib
import ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Função para enviar e-mails via Gmail
def enviar_email(remetente, senha, destinatarios, assunto, corpo):
    try:
        message = MIMEMultipart()
        message['Subject'] = assunto
        message['From'] = remetente
        message['To'] = ', '.join(destinatarios)
        
        # Adicionando o corpo do e-mail
        message.add_header('Content-Type', 'text/html')
        message.set_payload(corpo)

        # Conexão com o servidor SMTP do Gmail
        context = ssl.create_default_context()
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls(context=context)  # Habilita TLS (criptografia)
            server.login(remetente, senha)    # Login com a senha de app
            server.sendmail(
                remetente, destinatarios,
                message.as_string().encode('utf_8')
            )
        print(f"E-mail enviado com sucesso de {remetente}.")
        return True
    except Exception as e:
        print(f"Falha ao enviar e-mail com {remetente}. Erro: {e}")
        return False

# Definir destinatários
email_to = ['marcelo.carvalho@visionone.com.br', 'gabrielly.vetore@visionone.com.br']

# Corpo do e-mail (mensagem fixa)
corpo = r'''
    <p>Prezado(a),</p>
    <p>SUCESSO AO ENVIAR PELA API QVIS</p>
    <p>Esta é uma mensagem enviada automaticamente. Por favor, não a responda.</p>
'''

# Assunto do e-mail
assunto = 'SUCESSO AO ENVIAR PELA API QVIS'

# Remetente e senha de aplicativo para Gmail
remetente_gmail = 'mcc.visionone@gmail.com'
senha_gmail = 'bqwk zcgm vgeh bcgp'  # Substitua com sua senha de aplicativo gerada

# Tentar enviar com Gmail
sucesso = enviar_email(remetente_gmail, senha_gmail, email_to, assunto, corpo)

if sucesso:
    print("E-mail enviado com sucesso!")
else:
    print("Falha ao enviar e-mail.")

E-mail enviado com sucesso de mcc.visionone@gmail.com.
E-mail enviado com sucesso!


In [198]:
end_time = time.time()
total_time = end_time - start_time
print('Tempo total (min) = {}'.format(total_time/60))

Tempo total (min) = 74.22591703335443


In [199]:
import psutil

ram_total = psutil.virtual_memory().total / (1024 ** 3)
ram_disponivel = psutil.virtual_memory().available / (1024 ** 3)

print(f"RAM total: {ram_total:.2f} GB")
print(f"RAM disponível: {ram_disponivel:.2f} GB")


RAM total: 63.69 GB
RAM disponível: 44.40 GB
